In [ ]:
# create/activate venv (ypd_venv)

# create
# python -m venv ypd_venv
# 
# activate
# ypd_venv\Scripts\activate


In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

DB_PATH = "pitch_kinematics.db"

def init_db():
    """Create the pitch_data table if it doesn't exist."""
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    # Adjust columns as needed for your data
    c.execute("""
        CREATE TABLE IF NOT EXISTS pitch_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            participant_name TEXT,
            height REAL,
            weight REAL,
            pitch_date TEXT,
            pitch_type TEXT,
            filename TEXT,
            pitch_stability_score REAL,

            -- Example columns for Ulnar Dev:
            rel_u_dev REAL,
            frame2_u_dev REAL,
            frame4_u_dev REAL,
            frame6_u_dev REAL,
            frame8_u_dev REAL,
            frame10_u_dev REAL,

            -- Example columns for Pronation:
            rel_pronation REAL,
            frame2_pronation REAL,
            frame4_pronation REAL,
            frame6_pronation REAL,
            frame8_pronation REAL,
            frame10_pronation REAL

            -- Add more columns if needed, e.g. flexion or acceleration
        )
    """)

    conn.commit()
    conn.close()


def find_text(element, tag):
    """Helper to get text from an XML tag safely."""
    found = element.find(tag)
    return found.text if found is not None else None

def parse_xml_for_profile():
    """
    1) Use tkinter to prompt for a folder.
    2) Find the 'sessionXXX.xml' inside that folder.
    3) Extract participant_name, height, weight, creation_date (or however you name it).
    4) Return them as a dict.
    """
    root = tk.Tk()
    root.withdraw()
    selected_folder = filedialog.askdirectory(initialdir='D:/Youth Pitch Design/Data/')
    if not selected_folder:
        print("No folder selected. Exiting...")
        return None

    xml_file_path = ''
    for root_dir, _, files in os.walk(selected_folder):
        for file in files:
            if file.lower().startswith('session') and file.lower().endswith('.xml'):
                xml_file_path = os.path.join(root_dir, file)
                break
        if xml_file_path:
            break
    
    if not xml_file_path:
        print("No XML file found. Exiting...")
        return None

    # Parse
    tree = ET.parse(xml_file_path)
    xml_root = tree.getroot()
    session_fields = xml_root.find(".//Session/Fields")

    participant_name = find_text(session_fields, "Name")    
    height = find_text(session_fields, "Height")
    weight = find_text(session_fields, "Weight")
    creation_date = find_text(session_fields, "Creation_date")  


    if None in [participant_name, height, weight, creation_date]:
        print("Missing data in XML file. Exiting...")
        return None

    return {
        "participant_name": participant_name,
        "height": float(height),
        "weight": float(weight),
        "pitch_date": creation_date,  # store in pitch_data table as pitch_date
        "folder": selected_folder     # Might need this for searching pitch files
    }


def compute_pitch_stability_score(row):
    """
    Compute a 0..100 pitch stability score from
    a single row in all_offsets_metrics (which holds
    columns for ulnar dev & pronation around release).
    
    - We gather 5 frames before release + 5 frames after release
      for both Ulnar Dev and Pronation.
    - Then compute an 'angle score' (lower absolute angles => higher score)
    - Then apply a 'variation penalty' based on std dev
    - Combine them for a final 0..100 result
    """
    # -----------------------------------------------------------------------
    # 1) Identify your columns (these are example names; adjust to match real CSV)
    # Suppose 'rel_minus10','rel_minus8','rel_minus6','rel_minus4','rel_minus2',
    #         'rel0','rel_plus2','rel_plus4','rel_plus6','rel_plus8'
    # are the columns for *ulnar deviation*,
    # while you might have 'pron_minus10','pron_minus8',... for pronation
    # If your file merges them, rename accordingly.
    # -----------------------------------------------------------------------
    ulnar_cols = [
        "u_dev_minus10", "u_dev_minus8", "u_dev_minus6", "u_dev_minus4", "u_dev_minus2",
        "u_dev_0",       "u_dev_plus2",  "u_dev_plus4",  "u_dev_plus6",  "u_dev_plus8"
    ]
    pron_cols = [
        "pron_minus10", "pron_minus8", "pron_minus6", "pron_minus4", "pron_minus2",
        "pron_0",       "pron_plus2",  "pron_plus4",  "pron_plus6",  "pron_plus8"
    ]

    # Extract the numeric arrays for each series
    ulnar_values = []
    for col in ulnar_cols:
        if col in row:        # Make sure column exists
            ulnar_values.append(row[col])
    
    pron_values = []
    for col in pron_cols:
        if col in row:
            pron_values.append(row[col])

    # If the row doesn't have enough columns, or the data is incomplete:
    if len(ulnar_values) < 5 or len(pron_values) < 5:
        return 0.0  # or some fallback

    # Convert to numpy arrays
    ulnar_dev_series = np.array(ulnar_values, dtype=float)
    pron_series      = np.array(pron_values,  dtype=float)

    # -----------------------------------------------------------------------
    # 2) "Angle Score": We want smaller absolute angles => bigger score
    # For example, an angle of 0 => 100 points, angle of 40 => 0 points
    # For pronation, you might allow up to 80 => 0 points, etc.
    # -----------------------------------------------------------------------
    def angle_score(angle, max_angle):
        sc = 100.0 - (abs(angle)/max_angle)*100.0
        return np.clip(sc, 0, 100)

    # Example max angles:
    max_u = 40.0   # typical threshold for ulnar dev
    max_p = 80.0   # typical threshold for pronation

    u_scores = [angle_score(a, max_u) for a in ulnar_dev_series]
    p_scores = [angle_score(a, max_p) for a in pron_series]

    mu_u = np.mean(u_scores)
    mu_p = np.mean(p_scores)

    # -----------------------------------------------------------------------
    # 3) Variation penalty: smaller std dev => higher score
    # We define a max std for each, e.g. 20 for ulnar dev, 30 for pronation
    # -----------------------------------------------------------------------
    max_std_u = 20.0
    max_std_p = 30.0

    std_u = np.std(ulnar_dev_series)
    std_p = np.std(pron_series)

    def var_score(stdv, mxs):
        sc = 100.0 - (stdv/mxs)*100.0
        return np.clip(sc, 0, 100)

    vs_u = var_score(std_u, max_std_u)
    vs_p = var_score(std_p, max_std_p)

    # Average magnitude score + average variation score
    mag_mean = np.mean([mu_u, mu_p])
    var_mean = np.mean([vs_u, vs_p])

    # Weighted combo: 60% magnitude, 40% consistency
    final_score = 0.6*mag_mean + 0.4*var_mean

    return round(final_score, 2)


def parse_all_offsets_metrics(all_offsets_path):
    """
    Example: reads all_offsets_metrics.txt,
    extracts columns, and returns a DataFrame you can merge with your pitch_data inserts.
    This is just a skeleton. Adjust columns/logic to match your real file.
    """
    # Suppose the file has columns: [trial_id, rel_u_dev, frame2_u_dev, ... rel_pronation, etc.]
    df = pd.read_csv(all_offsets_path, sep="\t")  # or appropriate delimiter

    # If you compute pitch_stability_score from columns here:
    df["pitch_stability_score"] = df.apply(lambda row: compute_pitch_stability_score(row), axis=1)

    return df

def parse_link_model_based(file_path):
    """
    Reads the 'link_model_based.txt' file which has:
      - Row1 (index=0): blank col in col0, then full filepaths in col1..N
      - Row2..Row4: 'Wrist_Angle', 'LINK_MODEL_BASED', 'PROCESSED'
      - Row5 (index=4): 'ITEM', then repeating X/Y/Z axis labels
      - Row6 (index=5) onward: the actual numeric data, with col0 = 1,2,3,... frame index

    We will:
      1) Manually read the first 5 lines to build column names.
      2) Read the remaining lines into DataFrame, ignoring the first column (the ITEM).
      3) Return a wide DataFrame whose columns might look like:
            ["Frame", "Trial1_X", "Trial1_Y", "Trial1_Z", "Trial2_X", "Trial2_Y", "Trial2_Z", ...]
         plus we keep track of which file each “trial” corresponds to from Row1.
    """

    with open(file_path, "r", encoding="utf-8") as f:
        # --- Read the first 5 header lines ---
        line1 = next(f).rstrip("\n").split("\t")  # e.g. ['', 'D:\Path1.c3d', 'D:\Path2.c3d', ...]
        line2 = next(f).rstrip("\n").split("\t")  # e.g. ['', 'Wrist_Angle','Wrist_Angle','Wrist_Angle',...]
        line3 = next(f).rstrip("\n").split("\t")  # e.g. ['', 'LINK_MODEL_BASED','LINK_MODEL_BASED',...]
        line4 = next(f).rstrip("\n").split("\t")  # e.g. ['', 'PROCESSED','PROCESSED','PROCESSED',...]
        line5 = next(f).rstrip("\n").split("\t")  # e.g. ['ITEM','X','Y','Z','X','Y','Z',...]

        # The number of trials is len(line1) - 1, ignoring the blank first column
        filenames = line1[1:]  # This is the row containing the c3d file paths
        num_trials = len(filenames)

        # The 5th row has: ['ITEM', 'X','Y','Z','X','Y','Z', ...] repeated
        # each trial has 3 columns X/Y/Z. So total = 1 + 3*num_trials
        # The first cell is 'ITEM', so we ignore that. The rest are axis labels.
        axis_labels = line5[1:]  # e.g. ['X','Y','Z','X','Y','Z',...]

        # --- Construct column names for the final DataFrame ---
        # We'll call the first column "Frame" (or "ITEM"), then each trial gets X/Y/Z columns
        # e.g. for trial i: Trial i _X,  ..._Y,  ..._Z
        col_names = ["Frame"]  # or "ITEM"
        for i, filepath in enumerate(filenames):
            trial_id = f"Trial{i+1}"
            col_names.append(f"{trial_id}_X")
            col_names.append(f"{trial_id}_Y")
            col_names.append(f"{trial_id}_Z")

        # -- Now read the rest of the lines into a DataFrame (the numeric data) --
        # We already consumed 5 lines, so we can pass skiprows=5 to pd.read_csv,
        # but we must handle the fact that the first column is "Frame" (ITEM) now.
        #
        # Because row5 had 1 + 3*num_trials columns, the data lines also have that many columns.
        # We'll supply `names=col_names` so pandas uses that as the final column set.
        #
        # We do NOT want to treat row5 as a header, so let's use `header=None`.
        #
        # Also note that row5's "ITEM" name would become col 0 if we skip exactly 4 lines,
        # but we actually want to skip 5 lines total so that the row with "ITEM" is not read as data.
        #
        # So the data starts at line6 (index=5). Therefore skiprows=5.

    df_data = pd.read_csv(
        file_path,
        sep="\t",
        header=None,
        skiprows=5,       # skip the first 5 lines
        names=col_names,  # use our custom column list
        engine="python"   # or "c", but "python" is often safer with odd tabs
    )

    # The "Frame" column is numeric (1,2,3...), but might be read as float if blank lines exist
    # Try to convert it to int:
    df_data["Frame"] = pd.to_numeric(df_data["Frame"], errors="coerce")

    # Optionally drop rows where "Frame" is NaN (e.g. if you have trailing blank lines)
    df_data.dropna(subset=["Frame"], inplace=True)

    # df_data is now "wide" with columns: Frame, Trial1_X, Trial1_Y, Trial1_Z, Trial2_X, ...
    # If you prefer to "melt" it to a long format, we can do that:
    #   df_long = df_data.melt(id_vars="Frame", var_name="TrialAxis", value_name="Angle")
    #   Then parse "TrialAxis" to separate "Trial1" from "_X", etc.

    return df_data, filenames

def ingest_data_into_db(profile, offsets_df):
    """
    Insert rows into the single pitch_data table.
    'offsets_df' should have one row per pitch (trial_id or filename),
    plus columns for pitch_type, etc.
    We'll guess the pitch_type from the filename if needed.
    """
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    for idx, row in offsets_df.iterrows():
        # Suppose the CSV has a column named "filename" instead
        filename = row["filename"]  
        pitch_type = "Fastball" if "Fast" in filename else ...

        # Example columns:
        rel_u_dev = row.get("rel_u_dev", None)
        frame2_u_dev = row.get("frame2_u_dev", None)
        # ... etc for pronation

        # Insert
        c.execute("""
            INSERT INTO pitch_data (
                participant_name,
                height, weight,
                pitch_date,
                pitch_type,
                filename,
                pitch_stability_score,
                rel_u_dev,
                frame2_u_dev
                -- etc for the other columns
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            profile["participant_name"],
            profile["height"],
            profile["weight"],
            profile["pitch_date"],
            pitch_type,
            filename,
            row["pitch_stability_score"],
            rel_u_dev,
            frame2_u_dev,
        ))
    conn.commit()
    conn.close()


if __name__ == "__main__":
    # 1) Initialize DB
    init_db()

    # 2) Parse the XML to get participant profile
    profile_info = parse_xml_for_profile()
    if not profile_info:
        exit()  # No valid profile or user canceled

    # 3) Parse your "all_offsets_metrics.txt" (or whichever path)
    all_offsets_path = "D:\\Youth Pitch Design\\Exports\\all_offsets_metrics.txt"
    offsets_df = parse_all_offsets_metrics(all_offsets_path)

    # 4) Insert into DB
    ingest_data_into_db(profile_info, offsets_df)

    print("Data ingestion complete!")


DB_PATH = "pitch_kinematics.db"

# ----------------- HELPER FUNCTIONS -----------------
def get_dropdown_options():
    """Return participant_name options from pitch_data."""
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    options = [{"label": p, "value": p} for p in sorted(participants)]
    return options

def get_date_options(selected_participant):
    """Return pitch_date options (from creation_date or pitch_date field) for that participant."""
    conn = sqlite3.connect(DB_PATH)
    query = f"""
        SELECT DISTINCT pitch_date 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    """Return pitch_type options from pitch_data for that participant + date, plus an 'All' option."""
    conn = sqlite3.connect(DB_PATH)
    query = f"""
        SELECT DISTINCT pitch_type 
        FROM pitch_data
        WHERE participant_name = '{selected_participant}'
        AND pitch_date = '{selected_date}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": pt, "value": pt} for pt in sorted(df["pitch_type"].unique())]
    # Insert "All" at the top
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    """
    Return the 'filename' options for the given participant/date/pitch_type.
    Insert 'All' as well.
    """
    conn = sqlite3.connect(DB_PATH)
    query = f"""
        SELECT DISTINCT filename 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}' 
        AND pitch_date = '{selected_date}'
    """
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()

    options = [{"label": fn, "value": fn} for fn in sorted(df["filename"].unique())]
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """
    Example approach to gather data from pitch_data, 
    group by pitch_type, compute comparisons vs. some reference_data if you have it, 
    then produce the same table structure you had before.
    """
    conn = sqlite3.connect(DB_PATH)
    query = f"""
        SELECT * 
        FROM pitch_data
        WHERE participant_name = '{selected_participant}'
        AND pitch_date = '{selected_date}'
    """
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    pitch_df = pd.read_sql_query(query, conn)

    # If you have a separate reference_data table, you can load it:
    # ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    conn.close()

    # For demonstration, let's pretend we have a reference DataFrame in code:
    # Provide some dummy reference values for each pitch_type. Adjust as needed.
    dummy_ref = {
        "Fastball": {
            "rel_u_dev": 5, "frame2_u_dev": 4, "frame4_u_dev": 3, "frame6_u_dev": 2, "frame8_u_dev": 1, "frame10_u_dev": 0,
            "rel_pronation": 10, "frame2_pronation": 8, "frame4_pronation": 6, "frame6_pronation": 4, "frame8_pronation": 2, "frame10_pronation": 0
        },
        "Curve": {
            "rel_u_dev": 6, "frame2_u_dev": 5, "frame4_u_dev": 4, "frame6_u_dev": 3, "frame8_u_dev": 2, "frame10_u_dev": 1,
            "rel_pronation": 12, "frame2_pronation": 9, "frame4_pronation": 7, "frame6_pronation": 5, "frame8_pronation": 3, "frame10_pronation": 1
        },
    }
    # Convert pitch_df to group means
    group_cols = [
        "rel_u_dev", "frame2_u_dev", "frame4_u_dev", "frame6_u_dev", "frame8_u_dev", "frame10_u_dev",
        "rel_pronation", "frame2_pronation", "frame4_pronation", "frame6_pronation", "frame8_pronation", "frame10_pronation"
    ]

    pitch_summary = pitch_df.groupby("pitch_type")[group_cols].mean(numeric_only=True).reset_index()

    # Build a "fake" reference DF in the same shape
    # (In real code, you'd read from a reference_data table or similar)
    ref_list = []
    for pt in pitch_summary["pitch_type"].unique():
        if pt in dummy_ref:
            ref_list.append({"pitch_type": pt, **dummy_ref[pt]})
        else:
            # If not in dummy_ref, create zeros or something
            ref_list.append({"pitch_type": pt, **{k:0 for k in group_cols}})
    ref_df = pd.DataFrame(ref_list)

    merged = pitch_summary.merge(ref_df, on="pitch_type", suffixes=("_selected", "_reference"))

    # Example "acceleration" columns
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    # Build the final comparison rows
    rows = []
    for _, row in merged.iterrows():
        pt = row["pitch_type"]
        # Ulnar Deviation row
        rows.append({
            "pitch_type": pt,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 1)
        })
        # Ulnar Deviation comparison
        rows.append({
            "pitch_type": f"{pt} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 1)
        })
        # Pronation row
        rows.append({
            "pitch_type": f"{pt} Pronation/Supination",
            "rel_u_dev": round(row["rel_pronation_selected"], 1),  # store pronation at release in this column
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 1)
        })
        # Pronation comparison
        rows.append({
            "pitch_type": f"{pt} Pronation/Supination Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 1)
        })

    comp_df = pd.DataFrame(rows)
    return comp_df

def get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename):
    """
    In the old code, you had multiple time-series sets: ulnar_dev_series, pronation, flexion.
    If your single pitch_data table does NOT store the entire time-series but only summary frames,
    you either need:
      1) Another table for the full time series, OR
      2) Reconstruct the time series from the summary columns (which is unusual).
    
    For demonstration, we’ll just return some FAKE time series data to plot.
    Replace this with your real logic if you store per-frame data somewhere.
    """
    # We'll return a dict of pitch_type -> dict("ulnar_dev_series" -> [list_of_series], "pronation" -> [list_of_series], "flexion"-> [...])
    # For now, just return random or dummy data:
    example_data = {}
    key_pt = selected_pitch_type if selected_pitch_type != "All" else "Fastball"
    frames = list(range(60))  # 60 frames
    ulnar_data = [np.sin(0.1*i) * 10 for i in frames]
    pron_data  = [np.cos(0.1*i) * 20 for i in frames]
    flex_data  = [np.sin(0.2*i) * -15 for i in frames]

    example_data[key_pt] = {
        "ulnar_dev_series": [ulnar_data],
        "pronation": [pron_data],
        "flexion": [flex_data]
    }
    return example_data

def get_curve_reference_on_the_fly():
    """
    If you have reference data for the time series (for each pitch_type),
    return a dict structure similar to get_time_series.
    Here we just do a dummy placeholder for "Curve".
    """
    frames = list(range(60))
    ref_ulnar  = [np.sin(0.1*i) * 8 for i in frames]
    ref_pron   = [np.cos(0.1*i) * 18 for i in frames]
    ref_flex   = [np.sin(0.2*i) * -12 for i in frames]
    return {
        "Curve": {
            "ulnar_dev_series": [ref_ulnar],
            "pronation": [ref_pron],
            "flexion": [ref_flex]
        }
    }

# ----------------- DASH APP SETUP -----------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# For demonstration, pick some default participant/date
DEFAULT_PARTICIPANT = None
DEFAULT_DATE = None
# You could query your DB to find the last processed participant & date.

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=get_dropdown_options(),
                value=DEFAULT_PARTICIPANT,
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                options=[],  # callback will populate
                value=DEFAULT_DATE,
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative", 
                        "width": "440px",
                        "height": "auto",
                        "padding": "10px",  
                        "margin-left": "140px",
                        "backgroundColor": "black"  
                    }
                )
            ], style={"text-align": "right"})
        ], width=2)
    ], className="mt-3", align="center"),       

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[
                            {"name": i, "id": i} 
                            for i in [
                                "pitch_type", "rel_u_dev",
                                "frame2_u_dev", "frame4_u_dev",
                                "frame6_u_dev", "frame8_u_dev",
                                "frame10_u_dev", "accel_u_dev"
                            ]
                        ],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=9
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Average Stability Score"),
                dbc.CardBody([
                    html.H3(id="average-score", style={
                        "fontSize": "82px", 
                        "textAlign": "center", 
                        "color": "lime"
                    }),
                    html.P("Higher = Better Wrist Stability", style={
                        "textAlign": "center", 
                        "color": "white"
                    })
                ])
            ]),
            width=3
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody([
                    dcc.Graph(id="ulnar-dev-graph"),
                    html.P([
                        "Ulnar deviation measures how much the wrist flexes toward the ulnar side of the forearm or 'flicks' as we release the ball.",
                        html.Br(),
                        html.Strong("Moving in the negative (-) direction represents ulnar deviation "),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "24px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody([
                    dcc.Graph(id="acceleration-graph"),
                    html.P([
                        "Acceleration in the transverse and frontal planes shows how rapidly the wrist angles are changing around release, which can correlate with injury risk. ",
                        html.Br(),
                        html.Strong("Ideally, it’s kept minimal "),
                        "to reduce stress during throwing."
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="pronation-graph")
                ),
                html.P([
                    "Pronation and Supination measure how much you 'twist' the wrist. Supination through ball release is associated with the same 'flick' motion we are trying to avoid.",
                    html.Br(),
                    html.Strong("Negative (-) values correspond with supination"),
                ], style={
                    "marginTop": "10px", 
                    "fontSize": "22px", 
                    "lineHeight": "2.4"
                })
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="flexion-graph")
                ),
                html.P([
                    "Flexion at the wrist can be associated with the same 'flick' that occurs with excessive ulnar deviation",
                    html.Br(),
                    html.Strong("Negative (-) values correspond with flexion"),
                ], style={
                    "marginTop": "10px", 
                    "fontSize": "22px", 
                    "lineHeight": "2.4"
                })
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ----------------- DASH CALLBACKS -----------------

# 1) Populate the date-dropdown whenever participant changes
@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    options = get_date_options(selected_participant)
    possible_values = [o["value"] for o in options]
    # If we have a DEFAULT_DATE or something similar:
    default_date = possible_values[0] if possible_values else None
    return options, default_date

# 2) Populate the pitch-type dropdown whenever participant or date changes
@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None
    options = get_pitch_type_options(selected_participant, selected_date)
    pitch_types = [o["value"] for o in options]
    if "Curve" in pitch_types:
        default_value = "Curve"
    else:
        default_value = "All" if "All" in pitch_types else (pitch_types[0] if pitch_types else None)
    return options, default_value

# 3) Populate the filename-dropdown based on participant, date, pitch_type
@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    options = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    value = options[0]["value"] if options else None
    return options, value

# 4) Update average score, comparison table, and graphs
@app.callback(
    Output("average-score", "children"),
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    # -------------- A) Average Score --------------
    conn = sqlite3.connect(DB_PATH)
    base_query = f"""
        SELECT AVG(pitch_stability_score) AS avg_score
        FROM pitch_data
        WHERE participant_name = '{selected_participant}'
          AND pitch_date = '{selected_date}'
    """
    # If pitch_type not "All", filter
    if selected_pitch_type != "All":
        base_query += f" AND pitch_type = '{selected_pitch_type}'"
    # If filename not "All", filter
    if selected_filename != "All":
        base_query += f" AND filename = '{selected_filename}'"

    df_avg = pd.read_sql_query(base_query, conn)
    conn.close()

    avg_score_val = df_avg.iloc[0,0]
    avg_score_str = f"{avg_score_val:.2f}" if avg_score_val else "N/A"

    # -------------- B) Comparison Table --------------
    comp_df = get_comparison_table(
        selected_participant, 
        selected_date, 
        selected_pitch_type, 
        selected_filename
    )
    table_data = comp_df.to_dict("records")

    # -------------- C) 4 Graphs --------------
    ts_data = get_time_series(
        selected_participant, 
        selected_date, 
        selected_pitch_type, 
        selected_filename
    )
    ref_data = get_curve_reference_on_the_fly()

    # A quick way to guess average release index
    release_frames = []
    for pt, data_dict in ts_data.items():
        for series in data_dict["ulnar_dev_series"]:
            # Suppose last 20 frames is post-release
            release_idx = len(series) - 20
            release_frames.append(release_idx)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    # Vertical line at release
    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # ------------ 1) Ulnar Deviation ------------
    ulnar_fig = go.Figure()
    plotted_pitch_types = set()

    # If reference data for the selected pitch_type
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # Plot actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = (pt not in plotted_pitch_types)
        for series in data_dict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
        plotted_pitch_types.add(pt)

    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline],
        autosize=True
    )

    # ------------ 2) Acceleration ------------
    accel_fig = go.Figure()

    def compute_acceleration(angle_series):
        return np.diff(angle_series, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    FRAMES_BEFORE = 30
    FRAMES_AFTER  = 40

    release_line = dict(
        type="line",
        x0=30, x1=30,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    plotted_pitch_types = set()
    for pt, data_dict in ts_data.items():
        color_ulnar = "#bb6a74"
        color_pron  = "#2c99d4"
        show_legend = (pt not in plotted_pitch_types)

        for series in data_dict["ulnar_dev_series"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]

            acc_u = compute_acceleration(sub_series)
            rms_u = compute_rms(acc_u)
            x_vals = list(range(len(rms_u)))

            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color=color_ulnar, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))

        for series in data_dict["pronation"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]

            acc_p = compute_acceleration(sub_series)
            rms_p = compute_rms(acc_p)
            x_vals = list(range(len(rms_p)))

            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color=color_pron, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))

        plotted_pitch_types.add(pt)

    accel_fig.update_layout(
        title="Acceleration: ±30 Frames Before to +40 After Release",
        xaxis_title="Index (local sub-window)",
        yaxis_title="Accel (°/frame²)",
        shapes=[release_line],
        autosize=True
    )

    # ------------ 3) Pronation ------------
    pronation_fig = go.Figure()
    plotted_pitch_types = set()

    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = (pt not in plotted_pitch_types)
        for series in data_dict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
        plotted_pitch_types.add(pt)

    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    # ------------ 4) Flexion ------------
    flexion_fig = go.Figure()
    plotted_pitch_types = set()

    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = (pt not in plotted_pitch_types)
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
        plotted_pitch_types.add(pt)

    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return avg_score_str, table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


if __name__ == '__main__':
    app.run_server(debug=True)


KeyError: 'filename'

In [ ]:
import os
import glob
import sqlite3
import pandas as pd
import numpy as np

import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

# ----------------------------------------------------------
# 1) SETUP: 
#     - DB path 
#     - function to read Visual3D exports
#     - function to parse them, store in DB
# ----------------------------------------------------------
DB_PATH = 'pitch_kinematics.db'
EXPORT_FOLDER = r"D:\Youth Pronation\Exports"  # Where Visual3D writes .txt or .csv

def init_db():
    """
    Create necessary tables if they don't exist.
    We'll have:
      - trials (trial_id PRIMARY KEY, pitch_type, stability_score, etc.)
      - timeseries (trial_id, frame, time, angle_x, angle_y, angle_z, etc.)
    """
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    # trial_id could be the original file name, e.g. 'Fastball_001'
    c.execute("""
        CREATE TABLE IF NOT EXISTS trials (
            trial_id TEXT PRIMARY KEY,
            pitch_type TEXT,
            stability_score REAL
        )
    """)

    # store the time-series
    c.execute("""
        CREATE TABLE IF NOT EXISTS timeseries (
            trial_id TEXT,
            frame INTEGER,
            time_s REAL,
            angle_x REAL,
            angle_y REAL,
            angle_z REAL,
            FOREIGN KEY (trial_id) REFERENCES trials(trial_id),
            PRIMARY KEY (trial_id, frame)
        )
    """)

    conn.commit()
    conn.close()

def parse_visual3d_export(file_path):
    """
    Read the ASCII file exported by Visual3D that has
    1) Possibly multiple lines of metadata
    2) Column headers including: TIME, Wrist_Angle_X, Wrist_Angle_Y, Wrist_Angle_Z, ...
       (We need to confirm actual column naming from Visual3D's output.)
    3) Data for frames between RelMinus20 and RelPlus30 inclusive.

    We'll attempt to parse using pandas, skipping lines until we find the header row that has 'TIME'.

    Return a DataFrame with columns: frame, time_s, angle_x, angle_y, angle_z
    """
    # We'll try to guess the skiprows by reading lines until we see the column name row
    # If you know the exact # of lines to skip, you can do skiprows=<int>.
    # For demonstration, let's do a small search:
    with open(file_path, 'r') as f:
        lines = f.readlines()

    header_index = None
    for i, line in enumerate(lines):
        if 'TIME' in line.upper() and 'Wrist_Angle' in line:
            header_index = i
            break

    if header_index is None:
        raise ValueError(f"Could not find a header row with TIME / Wrist_Angle in {file_path}")

    # now read with pandas
    df = pd.read_csv(file_path, sep='\t', skiprows=header_index, engine='python')
    # If the columns are space separated or comma separated, adjust 'sep'. 
    # If it's multiple spaces, we might do sep='\s+' or delim_whitespace=True

    # Possibly the columns are: TIME, [some other], Wrist_Angle_X, Wrist_Angle_Y, Wrist_Angle_Z
    # We need to rename them to consistent names
    # Let's see what columns we have:
    print("Columns found in {} => {}".format(os.path.basename(file_path), df.columns.tolist()))

    # We'll guess:
    #   TIME => we rename to 'Time'
    #   Wrist_Angle_X => angle_x
    #   Wrist_Angle_Y => angle_y
    #   Wrist_Angle_Z => angle_z
    # Adjust if actual names differ
    colmap = {}
    for col in df.columns:
        if col.upper().startswith("TIME"):
            colmap[col] = "Time"
        elif "Wrist_Angle_X" in col:
            colmap[col] = "angle_x"
        elif "Wrist_Angle_Y" in col:
            colmap[col] = "angle_y"
        elif "Wrist_Angle_Z" in col:
            colmap[col] = "angle_z"

    df = df.rename(columns=colmap)
    # drop columns we don't care about
    keepcols = ["Time","angle_x","angle_y","angle_z"]
    df = df[[c for c in df.columns if c in keepcols]]

    # Now, Time might be frames or actual seconds. Let's assume frames:
    # If it's frames, we do frame = Time, time_s = frame / FRAME_RATE (we can guess 240 or read from c3d param).
    # For demonstration, let's keep it as 'frame' for now, 
    #   or if it's actual seconds from Visual3D, we'll keep it as time_s.
    # We'll do the simplest approach: treat Time as frames:
    df["frame"] = df["Time"].astype(int)
    # If we want actual seconds, do df["time_s"] = df["frame"] / 240, for example

    df.drop(columns=["Time"], inplace=True)
    # We won't know the real sample rate from the export, so let's do time_s = frame * (1/240) as a guess
    # Or just store time_s = frame for now
    df["time_s"] = df["frame"].astype(float)

    # reorder columns
    df = df[["frame","time_s","angle_x","angle_y","angle_z"]]

    return df

def compute_stability_score(df):
    """
    Example placeholder function that computes a 'stability score'
    from the wrist angles or something else.
    We'll just do a silly measure here:
      - measure the standard deviation of angle_y from frames around release
    """
    # df has columns frame, angle_x, angle_y, angle_z
    # We'll say the last 20 frames is "post-release", we measure how stable angle_y is:
    if len(df) < 20:
        return 0.0
    last_20 = df.iloc[-20:]["angle_y"]
    stdev = last_20.std()
    # define a 'score' as 100 - stdev if stdev < 100, else 0
    score = max(0.0, 100.0 - stdev)
    return round(score,2)

def import_to_db(file_path, pitch_type):
    """
    Parse the Visual3D ASCII/CSV file, compute or define a trial_id from the filename,
    and store the data in 'timeseries' table. Also store pitch_type, stability_score in 'trials' table.
    """
    df = parse_visual3d_export(file_path)
    trial_id = os.path.splitext(os.path.basename(file_path))[0]  # e.g. "Fastball_01_Wrist"
    # or parse pitch_type from the file name if needed

    # For demonstration, let's compute a stability score
    score = compute_stability_score(df)

    # Insert into DB
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()

    # Insert row in trials table
    c.execute("INSERT OR REPLACE INTO trials (trial_id, pitch_type, stability_score) VALUES (?,?,?)",
              (trial_id, pitch_type, score))

    # Insert timeseries
    # We'll do it row by row. Alternatively we can do df.to_sql('timeseries', conn, ...)
    # but we want to assign trial_id for each row
    for _, row in df.iterrows():
        fr = int(row["frame"])
        t_s = float(row["time_s"])
        ax = float(row["angle_x"])
        ay = float(row["angle_y"])
        az = float(row["angle_z"])
        c.execute("""
            INSERT OR REPLACE INTO timeseries 
            (trial_id, frame, time_s, angle_x, angle_y, angle_z)
            VALUES (?,?,?,?,?,?)
        """, (trial_id, fr, t_s, ax, ay, az))
    conn.commit()
    conn.close()
    print(f"Imported {file_path} => trial_id={trial_id}, stability_score={score}")


# ----------------------------------------------------------
# 2) MAIN: 
#    - Initialize DB
#    - Find exported files
#    - Import them
# ----------------------------------------------------------
def main_import():
    init_db()

    # find exported files in EXPORT_FOLDER
    pattern = os.path.join(EXPORT_FOLDER, "*Wrist*.txt")  # or .csv, or something
    all_files = glob.glob(pattern)
    # We'll guess pitch_type from the file name
    # e.g. "Fastball_001_Wrist.txt" => pitch_type="Fastball"
    for f in all_files:
        base = os.path.basename(f).lower()
        if 'fastball' in base:
            pt = 'Fastball'
        elif 'curve' in base:
            pt = 'Curve'
        elif 'slider' in base:
            pt = 'Slider'
        elif 'changeup' in base:
            pt = 'Changeup'
        else:
            pt = 'Unknown'
        # import
        import_to_db(f, pt)


# ----------------------------------------------------------
# 3) DASH APP
#    - We'll replicate the dashboard: table, graphs, stability
# ----------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Pitch Analysis Dashboard"

# Layout:
app.layout = dbc.Container([
    html.H1("Pitch Analysis Dashboard", className="mt-3"),

    # (A) Dropdown to pick a trial
    dbc.Row([
        dbc.Col([
            html.Label("Select Trial:", style={"font-weight": "bold"}),
            dcc.Dropdown(id="trial-dropdown", options=[], value=None, style={"width":"100%"})
        ], width=3),
        dbc.Col([
            html.Label("Stability Score:", style={"font-weight": "bold"}),
            html.Div(id="stability-score", style={"fontSize":"24px", "color":"blue"})
        ], width=3),
    ], className="mb-3"),

    # (B) Table
    dbc.Row([
        dbc.Col([
            html.H4("Comparison Table"),
            dash_table.DataTable(
                id="comparison-table",
                columns=[{"name": "Metric", "id": "metric"},
                         {"name": "Value", "id": "value"}],
                data=[]
            )
        ])
    ]),

    # (C) Graphs
    dbc.Row([
        dbc.Col([dcc.Graph(id="ulnar-graph")], width=6),
        dbc.Col([dcc.Graph(id="pronation-graph")], width=6),
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(id="flexion-graph")], width=6),
        dbc.Col([dcc.Graph(id="accel-graph")], width=6),
    ]),

], fluid=True)


# Callback to populate the trial dropdown
@app.callback(
    Output("trial-dropdown", "options"),
    Output("trial-dropdown", "value"),
    Input("trial-dropdown", "value")  # just for triggering once
)
def populate_trial_dropdown(_):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("SELECT trial_id FROM trials", conn)
    conn.close()
    opts = [{"label": tid, "value": tid} for tid in df["trial_id"].unique()]
    val = opts[0]["value"] if opts else None
    return opts, val

# Main callback to update graphs, table, stability when user selects a trial
@app.callback(
    Output("stability-score", "children"),
    Output("comparison-table", "data"),
    Output("ulnar-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    Output("accel-graph", "figure"),
    Input("trial-dropdown", "value")
)
def update_dashboard(selected_trial):
    if not selected_trial:
        return "N/A", [], go.Figure(), go.Figure(), go.Figure(), go.Figure()

    conn = sqlite3.connect(DB_PATH)
    # 1) fetch stability from trials
    c = conn.cursor()
    c.execute("SELECT stability_score FROM trials WHERE trial_id=?", (selected_trial,))
    row = c.fetchone()
    stability = row[0] if row else None

    # 2) fetch timeseries
    df_ts = pd.read_sql_query("""
        SELECT frame, time_s, angle_x, angle_y, angle_z
        FROM timeseries
        WHERE trial_id=?
        ORDER BY frame
    """, conn, params=(selected_trial,))
    conn.close()

    if df_ts.empty:
        return "N/A", [], go.Figure(), go.Figure(), go.Figure(), go.Figure()

    # 3) build comparison table
    # For demonstration, let's define some metrics
    # e.g. "Max Ulnar Dev" from angle_y
    max_uld = df_ts["angle_y"].max()
    min_uld = df_ts["angle_y"].min()
    final_uld = df_ts["angle_y"].iloc[-1]
    table_data = [
        {"metric": "Max Ulnar Dev (deg)", "value": f"{max_uld:.1f}"},
        {"metric": "Min Ulnar Dev (deg)", "value": f"{min_uld:.1f}"},
        {"metric": "Final Ulnar Dev (deg)", "value": f"{final_uld:.1f}"}
    ]

    # 4) build the four plots:
    #    (A) Ulnar => angle_y
    fig_ulnar = go.Figure()
    fig_ulnar.add_trace(go.Scatter(
        x=df_ts["frame"], 
        y=df_ts["angle_y"], 
        mode="lines", 
        name="Ulnar Deviation"
    ))
    fig_ulnar.update_layout(title="Ulnar Deviation (Y)", xaxis_title="Frame", yaxis_title="Degrees")

    #    (B) Pronation => angle_z
    fig_pron = go.Figure()
    fig_pron.add_trace(go.Scatter(
        x=df_ts["frame"], 
        y=df_ts["angle_z"], 
        mode="lines",
        name="Pronation/Supination"
    ))
    fig_pron.update_layout(title="Pronation/Supination (Z)", xaxis_title="Frame", yaxis_title="Degrees")

    #    (C) Flexion => angle_x
    fig_flex = go.Figure()
    fig_flex.add_trace(go.Scatter(
        x=df_ts["frame"], 
        y=df_ts["angle_x"], 
        mode="lines",
        name="Flexion/Extension"
    ))
    fig_flex.update_layout(title="Flexion/Extension (X)", xaxis_title="Frame", yaxis_title="Degrees")

    #    (D) Accel => numeric derivative of angle_y or angle_z, just as example
    #        We'll do the second derivative or something. Let's do the 1st derivative of angle_y
    #        Then maybe a rolling RMS
    df_ts["diff_y"] = df_ts["angle_y"].diff().fillna(0) * 240.0  # if 240 Hz, approximate deg/s
    fig_accel = go.Figure()
    fig_accel.add_trace(go.Scatter(
        x=df_ts["frame"], 
        y=df_ts["diff_y"], 
        mode="lines",
        name="d(angle_y)/dt"
    ))
    fig_accel.update_layout(title="Acceleration-ish (Ulnar Dev Derivative)", xaxis_title="Frame", yaxis_title="deg/s")

    # Return
    stability_display = f"{stability:.2f}" if stability is not None else "N/A"
    return stability_display, table_data, fig_ulnar, fig_pron, fig_flex, fig_accel


if __name__ == "__main__":
    # 1) import the data if needed
    main_import()
    # 2) run dash
    app.run_server(debug=True)


In [52]:
import os
import glob
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import medfilt

import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio

import os
import glob
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from scipy.signal import butter, filtfilt, medfilt
import ezc3d

# For your Dash app
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go


# ----------------------------------------------------------------
# 1) GLOBAL DB PATH
# ----------------------------------------------------------------
db_path = "pitch_analysis_v5.sqlite"


# ----------------------------------------------------------------
# 2) FILTERING & MARKER PROCESSING
# ----------------------------------------------------------------
def lowpass_filter(data, cutoff, fs, order=2):
    """
    6 Hz Butterworth lowpass filter, forward-backward filtfilt.
    data shape is (nFrames,) for 1D signals.
    """
    from scipy.signal import butter, filtfilt
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

def filter_marker_data(c3d_obj):
    """
    Filter all marker coordinates in the c3d file at ~6 Hz.
    Mutates the c3d_obj 'points' array in-place.
    """
    points = c3d_obj['data']['points']
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    nMarkers = points.shape[1]
    nFrames = points.shape[2]

    if nFrames < 10:
        print("Not enough frames to lowpass filter; skipping filtering.")
        return

    for m in range(nMarkers):
        for coord in range(3):
            raw_signal = points[coord, m, :]
            filtered = lowpass_filter(raw_signal, cutoff=6.0, fs=frame_rate, order=2)
            points[coord, m, :] = filtered

def resolve_marker_indices(marker_labels):
    """
    Map marker names to indices for:
      - Lateral_Elbow
      - Medial_Elbow
      - Wrist_Radius
      - Wrist_Ulna
      - Hand
    Return { 'Lateral_Elbow': i, ...} if found, else {}
    """
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    clean = [lab.replace("Right_","").replace("Left_","") for lab in marker_labels]
    out = {}
    for mk in req:
        if mk in clean:
            out[mk] = clean.index(mk)
        else:
            return {}
    return out


# ----------------------------------------------------------------
# 3) VIRTUAL HAND OFFSET (STATIC TRIAL)
# ----------------------------------------------------------------
def create_virtual_hand_offset(c3d_obj):
    """
    From the static trial, compute an offset vector from the wrist center to 'Hand'.
    We'll replicate that offset in dynamic trials for consistent hand positioning.
    """
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Static trial missing required markers.")

    nFrames = points.shape[2]
    offsets = []
    for f in range(nFrames):
        R = points[:3, markers["Wrist_Radius"], f]
        U = points[:3, markers["Wrist_Ulna"], f]
        H = points[:3, markers["Hand"], f]
        wrist_center = (R + U)/2.0
        offsets.append(H - wrist_center)
    return np.mean(offsets, axis=0)

def apply_virtual_hand_marker(points, marker_indices, hand_offset):
    """
    Override the 'Hand' marker with a virtual location:
      Hand_virtual = wrist_center + offset
    """
    nFrames = points.shape[2]
    for f in range(nFrames):
        R = points[:3, marker_indices["Wrist_Radius"], f]
        U = points[:3, marker_indices["Wrist_Ulna"], f]
        wrist_center = (R + U)/2.0
        new_hand = wrist_center + hand_offset
        points[:3, marker_indices["Hand"], f] = new_hand


# ----------------------------------------------------------------
# 4) EVENT DETECTION
# ----------------------------------------------------------------
def find_local_events(c3d_obj, frame_rate, total_frames):
    if "EVENT" not in c3d_obj["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    ev_lab = c3d_obj["parameters"]["EVENT"]["LABELS"]["value"]
    ev_tim = c3d_obj["parameters"]["EVENT"]["TIMES"]["value"][1]

    if "Foot Contact" not in ev_lab or "Release" not in ev_lab:
        raise ValueError("Required events (Foot Contact, Release) not found.")

    i_foot = ev_lab.index("Foot Contact")
    i_rel  = ev_lab.index("Release")

    foot_sec = ev_tim[i_foot]
    rel_sec  = ev_tim[i_rel]

    foot_global = int(round(foot_sec * frame_rate))
    rel_global  = int(round(rel_sec * frame_rate))

    earliest = min(foot_global, rel_global)
    foot_local= foot_global - earliest
    rel_local = rel_global - earliest

    if foot_local<0 or foot_local>= total_frames: 
        raise ValueError("Foot Contact out of range.")
    if rel_local<0  or rel_local>= total_frames:
        raise ValueError("Release out of range.")
    return foot_local, rel_local


# ----------------------------------------------------------------
# 5) ANGLE CALCULATIONS (+ UNWRAP to avoid ±180° flips)
# ----------------------------------------------------------------
def detect_left_handed(filename_noext):
    """
    If 'LH' or 'Left' in filename => left-handed => return True
    """
    name_low = filename_noext.lower()
    if 'lh' in name_low or 'left' in name_low:
        return True
    return False


def project_to_plane(vec, normal):
    return vec - np.dot(vec, normal)*normal

def compute_deviation_angles(points, markers, start_frame, end_frame, is_left, do_median=False):
    """
    Compute 3 angle series (ulnar_dev, pronation, flexion) from frames [start_frame..end_frame).
    Also includes an 'unwrap' step to prevent flipping from +179° to -179°.
    Optionally apply median filter if do_median=True.
    """
    # We'll build each array, then unwrap
    ulnar_series = []
    pron_series  = []
    flex_series  = []

    # We'll keep track of last angle to unwrap
    prev_u = None
    prev_p = None
    prev_f = None

    for fr in range(start_frame, end_frame):
        u = compute_ulnar_deviation(points, markers, fr, is_left)
        p = compute_pronation(points, markers, fr, is_left)
        f = compute_wrist_flexion(points, markers, fr, is_left)

        # Unwrap each angle so there's no abrupt 360° jump
        if prev_u is not None:
            diff_u = u - prev_u
            if diff_u>150:
                u -= 360
            elif diff_u<-150:
                u += 360
        if prev_p is not None:
            diff_p = p - prev_p
            if diff_p>150:
                p -= 360
            elif diff_p<-150:
                p += 360
        if prev_f is not None:
            diff_f = f - prev_f
            if diff_f>150:
                f -= 360
            elif diff_f<-150:
                f += 360

        ulnar_series.append(u)
        pron_series.append(p)
        flex_series.append(f)

        prev_u = u
        prev_p = p
        prev_f = f

    # Optionally apply a median filter to reduce spikes
    if do_median:
        ulnar_series = medfilt(ulnar_series, kernel_size=5)
        pron_series  = medfilt(pron_series, kernel_size=5)
        flex_series  = medfilt(flex_series, kernel_size=5)

    return np.array(ulnar_series), np.array(pron_series), np.array(flex_series)

def compute_ulnar_deviation(points, marker_indices, frame, is_left_handed=False):
    """
    EXACT same approach for 1 frame. 
    We'll unify in compute_deviation_angles.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    wcent = (R+U)/2.0
    ecent = (E_lat+E_med)/2.0
    forearm_vec  = wcent - ecent
    forearm_unit = forearm_vec/(np.linalg.norm(forearm_vec)+1e-9)

    ref_vec = R - wcent
    ref_proj= project_to_plane(ref_vec, forearm_unit)
    h_proj  = project_to_plane(H - wcent, forearm_unit)

    ref_n = ref_proj/(np.linalg.norm(ref_proj)+1e-9)
    h_n   = h_proj  /(np.linalg.norm(h_proj)+1e-9)

    cross_ = np.cross(ref_n, h_n)
    dot_   = np.dot(ref_n, h_n)
    ang_rad= np.arctan2(np.linalg.norm(cross_), dot_)
    sign   = np.sign(np.dot(cross_, forearm_unit))
    angle  = np.degrees(ang_rad)*sign

    if is_left_handed:
        angle = -angle
    return angle

def compute_pronation(points, marker_indices, frame, is_left_handed=False):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat= points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med= points[:3, marker_indices["Medial_Elbow"], frame]

    wcent= (R+U)/2.0
    ecent= (E_lat+E_med)/2.0
    fore= wcent - ecent
    fore_n= fore/(np.linalg.norm(fore)+1e-9)

    RU = U - R
    Evec= E_med - E_lat

    RU_proj= project_to_plane(RU, fore_n)
    E_proj = project_to_plane(Evec,fore_n)

    RU_n= RU_proj/(np.linalg.norm(RU_proj)+1e-9)
    E_n= E_proj/(np.linalg.norm(E_proj)+1e-9)

    cross_=np.cross(E_n,RU_n)
    dot_= np.dot(E_n,RU_n)
    rad_= np.arccos(np.clip(dot_,-1.0,1.0))
    sign= np.sign(np.dot(cross_, fore_n))
    deg = np.degrees(rad_)*sign

    if is_left_handed:
        deg= -deg
    return deg

def compute_wrist_flexion(points, marker_indices, frame, is_left_handed=False):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat= points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med= points[:3, marker_indices["Medial_Elbow"], frame]
    H= points[:3, marker_indices["Hand"], frame]

    wcent= (R+U)/2.0
    ecent= (E_lat+E_med)/2.0
    fore= wcent - ecent
    fore_n= fore/(np.linalg.norm(fore)+1e-9)

    vertical= np.array([0,0,1], dtype=float)
    h_vec= H - wcent
    h_proj= project_to_plane(h_vec, fore_n)
    v_proj= project_to_plane(vertical, fore_n)

    h_n= h_proj/(np.linalg.norm(h_proj)+1e-9)
    v_n= v_proj/(np.linalg.norm(v_proj)+1e-9)

    cross_= np.cross(v_n,h_n)
    dot_=   np.dot(v_n,h_n)
    rad_=   np.arctan2(np.linalg.norm(cross_), dot_)
    sign=   np.sign(np.dot(cross_, fore_n))
    deg=    np.degrees(rad_)*sign

    if is_left_handed:
        deg= -deg
    return deg


# ----------------------------------------------------------------
# 6) COMPUTE PITCH STABILITY SCORE
# ----------------------------------------------------------------
def compute_pitch_stability_score(ulnar_dev_series, pron_series, flex_series):
    """
    Combine absolute magnitude + consistency => 0..100
    """
    total_frames = len(ulnar_dev_series)
    if total_frames<20:
        return 0.0

    rel_idx = total_frames-20
    wstart= max(rel_idx-5,0)
    wend=   min(rel_idx+5, total_frames)

    uslice= ulnar_dev_series[wstart:wend]
    pslice= pron_series[wstart:wend]
    fslice= flex_series[wstart:wend]

    def angle_score(a, mx):
        sc= 100-(abs(a)/mx)*100
        return np.clip(sc,0,100)

    u_scores= [angle_score(a,40) for a in uslice]
    p_scores= [angle_score(a,80) for a in pslice]
    f_scores= [angle_score(a,70) for a in fslice]

    mu_u= np.mean(u_scores)
    mu_p= np.mean(p_scores)
    mu_f= np.mean(f_scores)

    # Variation penalty
    max_std_u=20
    max_std_p=30
    max_std_f=25
    std_u= np.std(uslice)
    std_p= np.std(pslice)
    std_f= np.std(fslice)
    def var_score(stdv, mxs):
        sc= 100-(stdv/mxs)*100
        return np.clip(sc,0,100)

    vs_u= var_score(std_u,max_std_u)
    vs_p= var_score(std_p,max_std_p)
    vs_f= var_score(std_f,max_std_f)

    mag_mean= np.mean([mu_u, mu_p, mu_f])
    var_mean= np.mean([vs_u, vs_p, vs_f])
    final= 0.6*mag_mean + 0.4*var_mean
    return round(final,2)


# ----------------------------------------------------------------
# 7) MAIN PROCESS: PROMPT FOLDER, LOAD c3d, INSERT INTO DB
# ----------------------------------------------------------------
def process_c3d_folder():
    # Prompt user for folder
    root= tk.Tk()
    root.withdraw()
    folder= filedialog.askdirectory(title="Select Data Folder")
    if not folder:
        raise ValueError("No folder selected")

    # gather c3d
    c3d_files= [os.path.join(folder,f) for f in os.listdir(folder) if f.lower().endswith(".c3d")]
    if not c3d_files:
        raise FileNotFoundError("No C3D found in the folder")

    # find static
    static_files= [f for f in c3d_files if "static" in f.lower()]
    if not static_files:
        raise FileNotFoundError("No static trial found in that folder.")
    static_path= static_files[0]

    # create table if needed
    conn= sqlite3.connect(db_path)
    cur= conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS pitch_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        participant_name TEXT,
        pitch_date TEXT,
        pitch_type TEXT,
        filename TEXT,
        pitch_stability_score REAL,
        mid_u_dev REAL,
        rel_u_dev REAL,
        frame1_u_dev REAL,
        frame2_u_dev REAL,
        frame3_u_dev REAL,
        frame4_u_dev REAL,
        frame5_u_dev REAL,
        frame6_u_dev REAL,
        frame7_u_dev REAL,
        frame8_u_dev REAL,
        frame9_u_dev REAL,
        frame10_u_dev REAL,
        mid_pronation REAL,
        rel_pronation REAL,
        frame1_pronation REAL,
        frame2_pronation REAL,
        frame3_pronation REAL,
        frame4_pronation REAL,
        frame5_pronation REAL,
        frame6_pronation REAL,
        frame7_pronation REAL,
        frame8_pronation REAL,
        frame9_pronation REAL,
        frame10_pronation REAL
    )
    """)
    conn.commit()

    # load static, filter, create offset
    sc3d= ezc3d.c3d(static_path)
    filter_marker_data(sc3d)
    static_offset= create_virtual_hand_offset(sc3d)

    # process each dynamic file
    for c3d_file in c3d_files:
        if c3d_file== static_path:
            continue
        c3d_obj= ezc3d.c3d(c3d_file)
        filter_marker_data(c3d_obj)

        points= c3d_obj["data"]["points"]
        marker_labels= c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        markers= resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file}, missing required markers.")
            continue

        # apply offset
        apply_virtual_hand_marker(points, markers, static_offset)

        frate= c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        totalf= points.shape[2]
        try:
            foot, rel= find_local_events(c3d_obj, frate, totalf)
        except Exception as e:
            print(f"Skipping {c3d_file}, event error: {e}")
            continue
        if rel+20>= totalf:
            print(f"Skipping {c3d_file}, not enough frames post-release.")
            continue

        # parse participant, date, pitch type
        npth= os.path.normpath(c3d_file)
        parts= npth.split(os.sep)
        if len(parts)<3:
            continue
        participant_folder= parts[-3]
        participant_name= participant_folder.rsplit("_",1)[0]
        date_folder= parts[-2].rstrip("_")
        pitch_date= date_folder
        file_only= parts[-1]
        file_noext= os.path.splitext(file_only)[0]
        pit_type= file_noext.split()[0].capitalize()

        is_left= detect_left_handed(file_noext)

        start_fr= foot
        end_fr= rel+20

        # compute angles w/ unwrapping
        u_arr, p_arr, f_arr= compute_deviation_angles(points, markers, start_fr, end_fr, is_left, do_median=False)

        # stability
        pitch_score= compute_pitch_stability_score(u_arr, p_arr, f_arr)

        mid_idx= len(u_arr)//2
        release_idx= len(u_arr)-20

        # store first 10 frames after foot_contact if exist
        n_frames= min(11, len(u_arr))

        insql= """
        INSERT INTO pitch_data (
            participant_name, pitch_date, pitch_type, filename,
            pitch_stability_score,
            mid_u_dev, rel_u_dev,
            frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
            frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
            mid_pronation, rel_pronation,
            frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
            frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
        )
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        """
        data_tuple=(
            participant_name,
            pitch_date,
            pit_type,
            file_noext,
            float(pitch_score),

            float(u_arr[mid_idx]) if mid_idx<len(u_arr) else None,
            float(u_arr[release_idx]) if release_idx<len(u_arr) else None,

            float(u_arr[1]) if n_frames>1 else None,
            float(u_arr[2]) if n_frames>2 else None,
            float(u_arr[3]) if n_frames>3 else None,
            float(u_arr[4]) if n_frames>4 else None,
            float(u_arr[5]) if n_frames>5 else None,
            float(u_arr[6]) if n_frames>6 else None,
            float(u_arr[7]) if n_frames>7 else None,
            float(u_arr[8]) if n_frames>8 else None,
            float(u_arr[9]) if n_frames>9 else None,
            float(u_arr[10])if n_frames>10 else None,

            float(p_arr[mid_idx]) if mid_idx<len(p_arr) else None,
            float(p_arr[release_idx]) if release_idx<len(p_arr) else None,

            float(p_arr[1]) if n_frames>1 else None,
            float(p_arr[2]) if n_frames>2 else None,
            float(p_arr[3]) if n_frames>3 else None,
            float(p_arr[4]) if n_frames>4 else None,
            float(p_arr[5]) if n_frames>5 else None,
            float(p_arr[6]) if n_frames>6 else None,
            float(p_arr[7]) if n_frames>7 else None,
            float(p_arr[8]) if n_frames>8 else None,
            float(p_arr[9]) if n_frames>9 else None,
            float(p_arr[10])if n_frames>10 else None
        )
        cur.execute(insql, data_tuple)
        conn.commit()
        print(f"Inserted {file_noext} => Score={pitch_score}  anglesrange=[{u_arr.min():.1f},{u_arr.max():.1f}]")

    conn.close()
    print("DONE PROCESSING FOLDER.")


# ----------------------------------------------------------------
# 8) GET FULL TIME-SERIES (FOR DASH PLOTTING)
# ----------------------------------------------------------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """
    EXACT same code used above, but does not re-insert into DB. 
    We also do the same unwrapping approach to match what's in DB.
    """
    import glob

    # For demonstration, we assume the user put these c3d in the same folder they picked originally
    # or a known root. If you want to prompt again, do so. Here we assume a known path.
    # If you want EXACT same folder, store it globally or in DB. We'll do a placeholder:
    data_root = r"D:\Some\Fixed\Data\Root"

    all_c3d= glob.glob(os.path.join(data_root,"**/*.c3d"), recursive=True)
    ts= {}

    for c3d_path in all_c3d:
        npth= os.path.normpath(c3d_path)
        parts= npth.split(os.sep)
        if len(parts)<3:
            continue
        participant_folder= parts[-3]
        part_name= participant_folder.rsplit("_",1)[0]
        date_folder= parts[-2].rstrip("_")
        pitch_date= date_folder
        fname_only= parts[-1]
        fname_noext= os.path.splitext(fname_only)[0]
        pit_type= fname_noext.split()[0].capitalize()

        if part_name!= selected_participant:
            continue
        if pitch_date!= selected_date:
            continue
        if selected_pitch_type!="All" and pit_type!= selected_pitch_type:
            continue
        if selected_filename!="All" and fname_noext!= selected_filename:
            continue

        c3d_obj= ezc3d.c3d(c3d_path)
        filter_marker_data(c3d_obj)

        marker_labels= c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        markers= resolve_marker_indices(marker_labels)
        if not markers:
            continue

        points= c3d_obj["data"]["points"]
        frate= c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        totalf= points.shape[2]
        try:
            foot, rel= find_local_events(c3d_obj, frate, totalf)
        except:
            continue
        if rel+20>= totalf:
            continue

        is_left= detect_left_handed(fname_noext)
        start_fr= foot
        end_fr= rel+20

        # compute angles w/ unwrapping
        u_arr, p_arr, f_arr= compute_deviation_angles(points, markers, start_fr, end_fr, is_left, do_median=False)

        if pit_type not in ts:
            ts[pit_type]= {
                "ulnar_dev_series":[],
                "pronation":[],
                "flexion":[]
            }
        ts[pit_type]["ulnar_dev_series"].append(u_arr)
        ts[pit_type]["pronation"].append(p_arr)
        ts[pit_type]["flexion"].append(f_arr)

    return ts


# ----------------------------------------------------------------
# 6) REFERENCE DATA
# ----------------------------------------------------------------
def get_curve_reference_on_the_fly():
    """
    Example reference function that uses the same plane-based angle approach
    for 'Curve' pitches. Adjust path as needed.
    Returns a dictionary of the form:
      {
         "Curve": {
            "ulnar_dev_series": [...],
            "pronation": [...],
            "flexion": [...]
         }
      }
    If no files found, returns {}.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        # Provide a simple notice, then return empty
        print(f"Reference folder not found: {REFERENCE_FOLDER}")
        return {}

    all_c3d = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d')
    ]

    reference_data = {
        "Curve": {
            "ulnar_dev_series": [],
            "pronation": [],
            "flexion": []
        }
    }

    for c3d_path in all_c3d:
        c3d_obj = ezc3d.c3d(c3d_path)
        filter_marker_data(c3d_obj)

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}, missing required markers.")
            continue

        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]
        try:
            foot, rel = find_local_events(c3d_obj, frame_rate, total_frames)
        except:
            continue

        if rel + 20 >= total_frames:
            continue

        # If reference is all right-handed or all left-handed, either fix is_left=False
        # or detect from filename
        filename_noext = os.path.splitext(os.path.basename(c3d_path))[0]
        is_left = detect_left_handed(filename_noext)

        start_fr = foot
        end_fr   = rel + 20
        u_arr, p_arr, f_arr = [], [], []
        for fr in range(start_fr, end_fr):
            u = compute_ulnar_deviation(points, markers, fr, is_left_handed=is_left)
            p = compute_pronation(points, markers, fr, is_left_handed=is_left)
            fx = compute_wrist_flexion(points, markers, fr, is_left_handed=is_left)
            u_arr.append(u)
            p_arr.append(p)
            f_arr.append(fx)

        reference_data["Curve"]["ulnar_dev_series"].append(u_arr)
        reference_data["Curve"]["pronation"].append(p_arr)
        reference_data["Curve"]["flexion"].append(f_arr)

    return reference_data


# ----------------------------------------------------------------
# 7) MAIN TIME-SERIES FUNCTION FOR PLOTS
# ----------------------------------------------------------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """
    Re-compute the angle arrays for each matching pitch .c3d,
    returning a dict: pitch_type => { 'ulnar_dev_series': [...], 'pronation': [...], 'flexion': [...] }
    """

    # Modify this path to where your .c3d data is stored:
    root_data_path = r"D:\Path\To\All\Your\Data"

    # Gather all .c3d files in subfolders:
    all_files = glob.glob(os.path.join(root_data_path, "**/*.c3d"), recursive=True)

    ts = {}

    for c3d_path in all_files:
        normalized_path = os.path.normpath(c3d_path)
        parts = normalized_path.split(os.sep)
        if len(parts) < 3:
            continue

        participant_folder = parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = parts[-2].rstrip("_")
        pitch_date  = date_folder
        filename_only = parts[-1]
        filename_noext= os.path.splitext(filename_only)[0]
        pitch_type_local = filename_noext.split()[0].capitalize()

        # Check if it matches the user's selection:
        if participant_name != selected_participant:
            continue
        if pitch_date != selected_date:
            continue
        if selected_pitch_type != "All" and pitch_type_local != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        # load & filter
        c3d_obj = ezc3d.c3d(c3d_path)
        filter_marker_data(c3d_obj)

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}, required markers missing.")
            continue

        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact, release = find_local_events(c3d_obj, frame_rate, total_frames)
        except:
            continue
        if release + 20 >= total_frames:
            continue

        # detect LH or RH
        is_left = detect_left_handed(filename_noext)

        # build local arrays
        start_frame = foot_contact
        end_frame   = release + 20
        ulnar_arr   = []
        pron_arr    = []
        flex_arr    = []

        for fr in range(start_frame, end_frame):
            u = compute_ulnar_deviation(points, markers, fr, is_left_handed=is_left)
            p = compute_pronation(points, markers, fr, is_left_handed=is_left)
            fx = compute_wrist_flexion(points, markers, fr, is_left_handed=is_left)
            ulnar_arr.append(u)
            pron_arr.append(p)
            flex_arr.append(fx)

        if pitch_type_local not in ts:
            ts[pitch_type_local] = {
                "ulnar_dev_series": [],
                "pronation": [],
                "flexion": []
            }

        ts[pitch_type_local]["ulnar_dev_series"].append(ulnar_arr)
        ts[pitch_type_local]["pronation"].append(pron_arr)
        ts[pitch_type_local]["flexion"].append(flex_arr)

    return ts

# ----------------------------------------------------------------
# 9) DASH + LAYOUT
# ----------------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# Optional: after we run insertion code, retrieve last participant/date:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()
if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=[],
                value=LAST_PARTICIPANT,
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                options=[],
                value=LAST_DATE,
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative",
                        "width": "250px", 
                        "height": "auto",
                        "padding": "10px",
                        "backgroundColor": "black"
                    }
                )
            ], style={"textAlign": "right"})
        ], width=2)
    ], className="mt-3", align="center"),

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[{"name": i, "id": i} for i in [
                            "pitch_type", "rel_u_dev", "frame2_u_dev", "frame4_u_dev",
                            "frame6_u_dev", "frame8_u_dev", "frame10_u_dev", "accel_u_dev"
                        ]],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=9
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Average Stability Score"),
                dbc.CardBody([
                    html.H3(id="average-score", style={
                        "fontSize": "82px",
                        "textAlign": "center",
                        "color": "lime"
                    }),
                    html.P("Higher = Better Wrist Stability", style={
                        "textAlign": "center",
                        "color": "white"
                    })
                ])
            ]),
            width=3
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody([
                    dcc.Graph(id="ulnar-dev-graph"),
                    html.P([
                        "Ulnar deviation: negative = radial dev, positive = ulnar dev."
                    ], style={
                        "marginTop": "10px",
                        "fontSize": "18px",
                        "lineHeight": "1.8"
                    })
                ])
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody([
                    dcc.Graph(id="acceleration-graph"),
                    html.P([
                        "Acceleration in the transverse/frontal planes around release."
                    ], style={
                        "marginTop": "10px",
                        "fontSize": "18px",
                        "lineHeight": "1.8"
                    })
                ])
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="pronation-graph")
                )
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="flexion-graph")
                )
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ---------------------------------------------------
# HELPER QUERIES FOR DROPDOWNS
# ---------------------------------------------------
def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].dropna().unique()
    return [{"label": p, "value": p} for p in sorted(participants)]

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].dropna().unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"""
        SELECT DISTINCT pitch_type 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}'
          AND pitch_date = '{selected_date}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    ptypes = df["pitch_type"].dropna().unique().tolist()
    opts = [{"label": pt, "value": pt} for pt in sorted(ptypes)]
    opts.insert(0, {"label":"All","value":"All"})
    return opts

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"""
        SELECT DISTINCT filename 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}'
          AND pitch_date = '{selected_date}'
    """
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"

    df = pd.read_sql_query(query, conn)
    conn.close()
    fnames = df["filename"].dropna().unique().tolist()
    opts = [{"label": f, "value": f} for f in sorted(fnames)]
    opts.insert(0, {"label":"All","value":"All"})
    return opts

# Example reference_data logic for comparison table:
def get_reference_data():
    """
    If you store reference data in a DB table called 'reference_data',
    load it here. Or return an empty df if not used.
    """
    conn = sqlite3.connect(db_path)
    try:
        ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    except:
        ref_df = pd.DataFrame()
    conn.close()
    return ref_df


# ---------------------------------------------------
# COMPARISON TABLE
# ---------------------------------------------------
def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """
    Summarize pitch_data vs. reference_data in a table of key angles.
    """
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"

    pitch_df = pd.read_sql_query(query, conn)
    ref_df   = get_reference_data()
    conn.close()

    if pitch_df.empty:
        return pd.DataFrame()

    pitch_summary = pitch_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    if ref_df.empty or "pitch_type" not in ref_df.columns:
        # If no reference data table, just return pitch_summary
        # but still format the columns
        pitch_summary["accel_u_dev_selected"] = 0
        pitch_summary["diff_accel_u_dev"] = 0
        pitch_summary["accel_pronation_selected"] = 0
        pitch_summary["diff_accel_pronation"] = 0
        # Return this as the final comparison
        out_cols = ["pitch_type", "rel_u_dev", "frame2_u_dev","frame4_u_dev","frame6_u_dev","frame8_u_dev","frame10_u_dev"]
        return pitch_summary[out_cols]

    # Else we have reference data
    ref_summary = ref_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    merged = pitch_summary.merge(ref_summary, on="pitch_type", suffixes=("_selected", "_reference"))

    # Simple 'acceleration' from frame2..frame10
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    rows = []
    for _, row in merged.iterrows():
        pitch = row["pitch_type"]

        # Ulnar row
        rows.append({
            "pitch_type": pitch,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 2)
        })

        # Ulnar Comparison
        rows.append({
            "pitch_type": f"{pitch} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 2)
        })

        # Pronation row
        rows.append({
            "pitch_type": f"{pitch} Pronation/Supination",
            "rel_u_dev": round(row["rel_pronation_selected"], 1),  # re-using 'rel_u_dev' col name in table
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 2)
        })

        # Pronation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Pronation/Supination Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 2)
        })

    return pd.DataFrame(rows)


# ---------------------------------------------------
# DASH CALLBACKS
# ---------------------------------------------------
@app.callback(
    Output("participant-dropdown", "options"),
    Output("participant-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def init_participant_dropdown(_):
    # Just populate participants on load
    opts = get_dropdown_options()
    val = None
    if opts:
        # pick first in sorted list or keep LAST_PARTICIPANT if present
        all_vals = [o['value'] for o in opts]
        val = LAST_PARTICIPANT if LAST_PARTICIPANT in all_vals else all_vals[0]
    return opts, val

@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    opts = get_date_options(selected_participant)
    val = None
    if opts:
        all_vals = [o['value'] for o in opts]
        # if LAST_DATE still valid, keep it
        val = LAST_DATE if LAST_DATE in all_vals else all_vals[0]
    return opts, val

@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None

    opts = get_pitch_type_options(selected_participant, selected_date)
    if not opts:
        return [], None

    # If 'Curve' is in there, default to that, else 'All' or first
    pitch_types = [o["value"] for o in opts]
    if "Curve" in pitch_types:
        default_val = "Curve"
    else:
        default_val = "All" if "All" in pitch_types else pitch_types[0]
    return opts, default_val

@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    opts = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    val = opts[0]["value"] if opts else None
    return opts, val

@app.callback(
    Output("average-score", "children"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value")
    ]
)
def update_average_score(selected_participant, selected_date, selected_pitch_type):
    """
    Quick DB fetch of the average pitch_stability_score for the chosen participant/date/type.
    """
    if not selected_participant or not selected_date:
        return "N/A"
    conn = sqlite3.connect(db_path)
    q = f"""
        SELECT AVG(pitch_stability_score)
        FROM pitch_data
        WHERE participant_name = '{selected_participant}'
          AND pitch_date = '{selected_date}'
    """
    if selected_pitch_type != "All":
        q += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(q, conn)
    conn.close()
    val = df.iloc[0,0]
    if val is None:
        return "N/A"
    return f"{val:.2f}"

@app.callback(
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    if not selected_participant or not selected_date:
        return [], go.Figure(), go.Figure(), go.Figure(), go.Figure()

    # 1) Build the comparison table
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    table_data = comp_df.to_dict("records")

    # 2) Build the time-series data from c3d files
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)

    # 3) Build the reference data
    ref_data = get_curve_reference_on_the_fly()

    # 4) Create the plots
    # compute average release index
    release_frames = []
    for pt, ddict in ts_data.items():
        for series in ddict["ulnar_dev_series"]:
            release_idx = len(series) - 20
            release_frames.append(release_idx)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # A) Ulnar Deviation figure
    ulnar_fig = go.Figure()
    plotted_pitch_types = set()

    # plot reference if relevant
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))

    # plot actual data
    for pt, ddict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types
        for series in ddict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
        plotted_pitch_types.add(pt)

    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline],
        autosize=True
    )

    # B) Acceleration figure
    accel_fig = go.Figure()
    def compute_acceleration(arr):
        return np.diff(arr, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    FRAMES_BEFORE = 30
    FRAMES_AFTER  = 40
    release_line = dict(
        type="line",
        x0=30, x1=30,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    plotted_pitch_types = set()

    for pt, ddict in ts_data.items():
        color_ulnar = "#bb6a74"
        color_pron  = "#2c99d4"
        show_legend = pt not in plotted_pitch_types

        # Ulnar dev acceleration
        for series in ddict["ulnar_dev_series"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub = series[start_i:end_i]
            acc_u = compute_acceleration(sub)
            rms_u = compute_rms(acc_u)
            x_vals = list(range(len(rms_u)))
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color=color_ulnar, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))

        # Pronation acceleration
        for series in ddict["pronation"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub = series[start_i:end_i]
            acc_p = compute_acceleration(sub)
            rms_p = compute_rms(acc_p)
            x_vals = list(range(len(rms_p)))
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color=color_pron, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))

        plotted_pitch_types.add(pt)

    accel_fig.update_layout(
        title="Acceleration: ±30 Frames Before to +40 After Release",
        xaxis_title="Index (local to sub-window)",
        yaxis_title="Accel (°/frame²)",
        shapes=[release_line],
        autosize=True
    )

    # C) Pronation figure
    pronation_fig = go.Figure()
    plotted_pitch_types = set()

    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))

    for pt, ddict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types

        for series in ddict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))

        plotted_pitch_types.add(pt)

    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    # D) Flexion figure
    flexion_fig = go.Figure()
    plotted_pitch_types = set()

    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))

    for pt, ddict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types

        for series in ddict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
        plotted_pitch_types.add(pt)

    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


# ---------------------------------------------------
# MAIN
# ---------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)


OperationalError: no such table: pitch_data

In [49]:
import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import medfilt
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

db_path = "pitch_analysis_v4.sqlite"

# ----------------------------------------------------------------
# 1) FILTERING
# ----------------------------------------------------------------
def lowpass_filter(data, cutoff, fs, order=2):
    """
    6 Hz Butterworth lowpass filter, forward-backward filtfilt.
    data shape is (nFrames,) for 1D signals.
    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

def filter_marker_data(c3d_obj):
    """Filter all marker coordinates in the c3d file at 6 Hz."""
    points = c3d_obj['data']['points']
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    nMarkers = points.shape[1]
    nFrames = points.shape[2]

    if nFrames < 10:
        print("Not enough frames to lowpass filter; skipping filtering for this file.")
        return

    for m in range(nMarkers):
        for coord in range(3):
            raw_signal = points[coord, m, :]
            filtered = lowpass_filter(raw_signal, cutoff=6.0, fs=frame_rate, order=2)
            points[coord, m, :] = filtered

# ----------------------------------------------------------------
# 2) MARKER MAPPING
# ----------------------------------------------------------------
def resolve_marker_indices(marker_labels):
    """
    Map marker names to indices. 
    We expect the following markers at minimum:
      - Lateral_Elbow
      - Medial_Elbow
      - Wrist_Radius
      - Wrist_Ulna
      - Hand
    Return a dict { "Lateral_Elbow": index, ... } if all found, else {}.
    """
    required = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    marker_indices = {}
    for mk in required:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}
    return marker_indices

def create_virtual_hand_offset(c3d_obj):
    """
    From the static trial, compute an offset vector from the wrist center to the Hand marker.
    We'll replicate that offset in dynamic trials for consistent hand positioning.
    """
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Required markers not found in static trial to create virtual hand offset.")
    nFrames = points.shape[2]

    offsets = []
    for f in range(nFrames):
        R = points[:3, markers["Wrist_Radius"], f]
        U = points[:3, markers["Wrist_Ulna"], f]
        H = points[:3, markers["Hand"], f]
        wrist_center = (R + U) / 2.0
        offset_vec = H - wrist_center
        offsets.append(offset_vec)

    mean_offset = np.mean(offsets, axis=0)
    return mean_offset

def apply_virtual_hand_marker(points, marker_indices, hand_offset):
    """
    Override the 'Hand' marker in dynamic data with a virtual location
    based on the static offset. 
    """
    nFrames = points.shape[2]
    for f in range(nFrames):
        R = points[:3, marker_indices["Wrist_Radius"], f]
        U = points[:3, marker_indices["Wrist_Ulna"], f]
        wrist_center = (R + U) / 2.0
        new_hand = wrist_center + hand_offset
        points[:3, marker_indices["Hand"], f] = new_hand

# ----------------------------------------------------------------
# 3) EVENT DETECTION
# ----------------------------------------------------------------
def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]

    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")

    idx_foot    = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec      = event_times[idx_release]

    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global      = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)

    foot_local    = foot_contact_global - earliest
    release_local = release_global      - earliest

    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")

    return foot_local, release_local

# ----------------------------------------------------------------
# 4) ANGLE CALCULATIONS: Direct Ulnar Deviation, Pronation, Flexion
# ----------------------------------------------------------------
def compute_ulnar_deviation(points, marker_indices, frame, is_left_handed=False):
    """
    Computes the signed wrist deviation angle in the frontal plane.
    Range typically ~ -40° (radial) to +40° (ulnar).
    
    Strategy:
      1) Forearm axis from elbow center -> wrist center.
      2) Project the hand vector onto the FRONTAL plane (the plane perpendicular to the forearm axis).
      3) Determine the angle between the 'neutral' vector (e.g. a line from wrist center -> radius side, or a known reference) and the actual hand projection.
      4) The sign is determined by cross-product direction.
    is_left_handed: if True, we flip the sign so that 'ulnar' is still + for a lefty.
    """
    # Markers
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    wrist_center = (R + U) / 2.0
    elbow_center = (E_lat + E_med) / 2.0

    forearm_vec  = wrist_center - elbow_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    # We'll define a 'reference axis' in the frontal plane: from wrist_center -> radius marker
    # That means if the hand is exactly over the radius, angle = 0, 
    # deviate ulnarly -> positive, deviate radially -> negative.
    #  - For a right-handed forearm, the radius side is typically lateral. 
    #  - For a left-handed forearm, we might flip sign. But let's do that after the angle calc.
    ref_vec = R - wrist_center

    # Project both ref_vec and (hand - wrist_center) into the plane perpendicular to forearm.
    def project_to_plane(vec, normal):
        return vec - np.dot(vec, normal) * normal

    ref_proj = project_to_plane(ref_vec, forearm_unit)
    hand_vec = H - wrist_center
    hand_proj = project_to_plane(hand_vec, forearm_unit)

    # Normalize
    ref_norm  = np.linalg.norm(ref_proj) + 1e-9
    hand_norm = np.linalg.norm(hand_proj) + 1e-9
    ref_unit  = ref_proj / ref_norm
    hand_unit = hand_proj / hand_norm

    # Signed angle in the plane
    cross_2d = np.cross(ref_unit, hand_unit)
    dot_2d   = np.dot(ref_unit, hand_unit)

    angle_rad = np.arctan2(np.linalg.norm(cross_2d), dot_2d)
    # sign: if cross_2d is 'above' or 'below' the plane w.r.t forearm_unit
    # But simpler might be sign = np.sign(np.dot(cross_2d, forearm_unit)) 
    sign = np.sign(np.dot(cross_2d, forearm_unit))

    angle_deg = np.degrees(angle_rad) * sign

    # So angle_deg ~ +30 => 30 deg ulnar dev, -20 => 20 deg radial dev, etc.
    # If left-handed, we can flip the sign so that "ulnar" is still positive.
    if is_left_handed:
        angle_deg = -angle_deg

    return angle_deg

def compute_pronation(points, marker_indices, frame, is_left_handed=False):
    """
    Similar approach to track pronation/supination around the forearm axis.
    Positive => pronation, negative => supination.
    Flip sign for lefty if you want consistent reporting.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]

    wrist_center = (R + U) / 2.0
    elbow_center = (E_lat + E_med) / 2.0

    forearm_vec  = wrist_center - elbow_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    # For pronation, we measure the rotation of the R->U vector about the forearm axis.
    # Vector from R->U
    RU = U - R

    # We'll define a reference as RU in the neutral supination position, but we 
    # can use E_lat->E_med if that helps. For simplicity, let's just use frames 0..some baseline,
    # or do the angle relative to some global orientation. 
    # For now, do the angle relative to 'vertical cross' or something. 
    # This is a simpler approach, though not a standard jcs. 
    # We'll do a cross approach: cross RU with forearm => measure sign. 
    # Or do an explicit plane approach. We'll keep it simpler:

    # Let's define a local orientation:
    #  - Project RU onto plane perpendicular to forearm_unit
    #  - Compare it to a "neutral" that points 'lateral' in that plane
    # Actually, let's do a direct angle between RU and 'elbow_lat -> elbow_med' or something 
    # you had in your code. We'll keep the sign approach. 
    # To keep it simpler, I'll do a cross/dot:

    # Project RU onto plane:
    RU_proj = RU - np.dot(RU, forearm_unit) * forearm_unit
    # Also define an 'elbow lat->med' for reference:
    E_vec = E_med - E_lat
    E_proj = E_vec - np.dot(E_vec, forearm_unit)*forearm_unit

    RU_n = RU_proj / (np.linalg.norm(RU_proj)+1e-9)
    E_n  = E_proj  / (np.linalg.norm(E_proj)+1e-9)

    cross_val = np.cross(E_n, RU_n)
    dot_val   = np.dot(E_n, RU_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))
    sign      = np.sign(np.dot(cross_val, forearm_unit))

    angle_deg = np.degrees(angle_rad) * sign

    # Now, + => pronation, - => supination, presumably
    # If left-handed, flip sign if we want the same convention for LH
    if is_left_handed:
        angle_deg = -angle_deg

    return angle_deg

def compute_wrist_flexion(points, marker_indices, frame, is_left_handed=False):
    """
    Similar approach to measure flexion/extension around the mediolateral axis. 
    We define the forearm axis, then define a 'vertical' or a 'rest' orientation 
    to measure how much the hand flexes up/down. 
    For consistency, we can define: 
      + => extension, - => flexion
    or vice versa. Then apply is_left_handed sign flip if desired.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    wrist_center = (R + U) / 2.0
    elbow_center = (E_lat + E_med) / 2.0

    forearm_vec  = wrist_center - elbow_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec)+1e-9)

    # We want the plane orthonormal basis for flex/extension. 
    # Let's define 'vertical' = global Z axis = [0,0,1], project that onto plane 
    # perpendicular to forearm_unit, then measure angle from that to the hand vector.
    vertical = np.array([0,0,1], dtype=float)
    def project_to_plane(vec, normal):
        return vec - np.dot(vec, normal)*normal

    hand_vec  = H - wrist_center
    hand_proj = project_to_plane(hand_vec, forearm_unit)

    vert_proj = project_to_plane(vertical, forearm_unit)

    # Normalize
    hp_n = hand_proj / (np.linalg.norm(hand_proj)+1e-9)
    vp_n = vert_proj / (np.linalg.norm(vert_proj)+1e-9)

    cross_val = np.cross(vp_n, hp_n)
    dot_val   = np.dot(vp_n, hp_n)
    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign      = np.sign(np.dot(cross_val, forearm_unit))

    angle_deg = np.degrees(angle_rad)*sign
    # + => extension, - => flexion (arbitrary choice)

    if is_left_handed:
        # If we want the same sign convention for lefties, we can flip sign
        angle_deg = -angle_deg

    return angle_deg


# ----------------------------------------------------------------
# 5) PITCH STABILITY SCORE
# ----------------------------------------------------------------
def compute_pitch_stability_score(ulnar_dev_series, pronation_series, flexion_series):
    """
    Updated approach to incorporate absolute magnitude and consistency (variation).
    1) Baseline angle at foot contact is assumed ~ 0 or small. We measure how far from that 
       the angles get around release.
    2) Also measure how quickly they change (flick).
    3) Weighted combination => final score (0..100).
    """
    total_frames = len(ulnar_dev_series)
    if total_frames < 20:
        return 0.0

    release_idx = total_frames - 20  # 'release' is 20 frames before end

    # We define a window ~ release_idx +/- 5 for measuring angle
    window_start = max(release_idx - 5, 0)
    window_end   = min(release_idx + 5, total_frames)

    # Slices
    ulnar_slice = ulnar_dev_series[window_start:window_end]
    pron_slice  = pronation_series[window_start:window_end]
    flex_slice  = flexion_series[window_start:window_end]

    # 1) absolute magnitude penalty => if angle is large in the slice, reduce score
    # typical feasible ranges: 
    #   - ulnar dev ~ +/- 40 
    #   - pronation ~ +/- 80 
    #   - flexion ~ +/- 70 
    # We'll define a function that gives 100 if angle ~ 0, and 0 if angle hits the max range.
    def angle_score(angle_val, max_allowed):
        # e.g. if angle_val= 20, max_allowed= 40 => 50% used => 50 left => 50 on scale => 50 => 50. 
        # simpler: sc = 100 - (abs(angle_val)/max_allowed * 100)
        sc = 100 - (abs(angle_val)/max_allowed)*100
        return np.clip(sc, 0, 100)

    # We'll compute an average magnitude penalty across frames in that slice
    # then average it with a variability penalty.
    # Let's do separate weighting for each angle:
    #   - Ulnar dev max ~ 40
    #   - Pronation max ~ 80
    #   - Flexion max ~ 70
    # Then we average them
    ulnar_scores = [angle_score(a, 40) for a in ulnar_slice]
    pron_scores  = [angle_score(a, 80) for a in pron_slice]
    flex_scores  = [angle_score(a, 70) for a in flex_slice]

    mean_ulnar_mag_score = np.mean(ulnar_scores)  # 0..100
    mean_pron_mag_score  = np.mean(pron_scores)
    mean_flex_mag_score  = np.mean(flex_scores)

    # 2) Variation penalty => if angles swing widely in that slice, reduce score
    # We'll do the standard deviation in the slice or range.
    # e.g. if there's a 40° swing in the slice for ulnar dev, that's large flick => big penalty
    # Let's define a function that returns a 0..100 penalty for a stdev or range.
    # We'll define 'max_stdev' for each angle. If stdev is that or more => 0 score, 
    # if stdev is 0 => 100 score
    max_stdev_ulnar = 20  # if stdev ~ 20 => 0
    max_stdev_pron  = 30
    max_stdev_flex  = 25

    ulnar_std = np.std(ulnar_slice)
    pron_std  = np.std(pron_slice)
    flex_std  = np.std(flex_slice)

    def variation_score(std_val, max_stdev):
        vs = 100 - (std_val / max_stdev)*100
        return np.clip(vs, 0, 100)

    ulnar_var_score = variation_score(ulnar_std, max_stdev_ulnar)
    pron_var_score  = variation_score(pron_std, max_stdev_pron)
    flex_var_score  = variation_score(flex_std, max_stdev_flex)

    # Weighted combination
    # example weighting:
    #  - For magnitude:  60% 
    #  - For variation:  40%
    #  - Among angles: each angle could weigh equally or not
    # We'll do a simple equal weighting among angles (ulnar/pron/flex => 3 ways).
    # Then combine magnitude and variation as 60/40.
    mag_ulnar = mean_ulnar_mag_score
    mag_pron  = mean_pron_mag_score
    mag_flex  = mean_flex_mag_score

    var_ulnar = ulnar_var_score
    var_pron  = pron_var_score
    var_flex  = flex_var_score

    mean_mag_score = np.mean([mag_ulnar, mag_pron, mag_flex])
    mean_var_score = np.mean([var_ulnar, var_pron, var_flex])

    final_score = 0.6 * mean_mag_score + 0.4 * mean_var_score
    return round(final_score, 2)

# ----------------------------------------------------------------
# 6) HAND PREFERENCE DETECTION
# ----------------------------------------------------------------
def detect_left_handed(filename_noext):
    """
    Simple approach:
     - if 'LH' or 'Left' in the filename (case-insensitive), return True
     - otherwise False
    """
    fname_lower = filename_noext.lower()
    if 'lh' in fname_lower or 'left' in fname_lower:
        return True
    return False

# ----------------------------------------------------------------
# (7) The MAIN PROCESSING LOOP
#     (like your original but with the new angle calculations)
# ----------------------------------------------------------------
def process_c3d_folder(selected_folder):
    # 1) find c3d
    c3d_files = [os.path.join(selected_folder, f)
                 for f in os.listdir(selected_folder)
                 if f.lower().endswith('.c3d')]

    # 2) find static
    static_files = [f for f in c3d_files if "static" in f.lower()]
    if not static_files:
        raise FileNotFoundError("No static trial found in the folder.")
    static_path = static_files[0]  # pick one

    # 3) create db table if not exist
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS pitch_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        participant_name TEXT,
        pitch_date TEXT,
        pitch_type TEXT,
        filename TEXT,
        pitch_stability_score REAL,
        mid_u_dev REAL,
        rel_u_dev REAL,
        frame1_u_dev REAL,
        frame2_u_dev REAL,
        frame3_u_dev REAL,
        frame4_u_dev REAL,
        frame5_u_dev REAL,
        frame6_u_dev REAL,
        frame7_u_dev REAL,
        frame8_u_dev REAL,
        frame9_u_dev REAL,
        frame10_u_dev REAL,
        mid_pronation REAL,
        rel_pronation REAL,
        frame1_pronation REAL,
        frame2_pronation REAL,
        frame3_pronation REAL,
        frame4_pronation REAL,
        frame5_pronation REAL,
        frame6_pronation REAL,
        frame7_pronation REAL,
        frame8_pronation REAL,
        frame9_pronation REAL,
        frame10_pronation REAL
    )
    """)
    conn.commit()

    # 4) load static, filter, compute offset
    static_c3d = ezc3d.c3d(static_path)
    filter_marker_data(static_c3d)
    static_offset = create_virtual_hand_offset(static_c3d)

    # 5) loop dynamic
    for c3d_file in c3d_files:
        if c3d_file == static_path:
            continue

        c3d_obj = ezc3d.c3d(c3d_file)
        filter_marker_data(c3d_obj)

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file}, missing required markers.")
            continue

        apply_virtual_hand_marker(points, markers, static_offset)

        frame_rate  = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames= points.shape[2]
        try:
            foot_contact, release = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file}, event issue: {e}")
            continue

        if release + 20 >= total_frames:
            print(f"Skipping {c3d_file}, not enough frames post-release.")
            continue

        # parse participant, date, pitch type
        normalized_path = os.path.normpath(c3d_file)
        path_parts = normalized_path.split(os.sep)
        # adapt if your folder structure differs
        if len(path_parts) < 3:
            continue

        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder      = path_parts[-2].rstrip("_")
        pitch_date       = date_folder
        filename_only    = path_parts[-1]
        filename_noext   = os.path.splitext(filename_only)[0]
        pitch_type       = filename_noext.split()[0].capitalize()

        is_left = detect_left_handed(filename_noext)

        # extract time-series
        start_frame = foot_contact
        end_frame   = release + 20
        ulnar_series = []
        pron_series  = []
        flex_series  = []

        for fr in range(start_frame, end_frame):
            u_dev = compute_ulnar_deviation(points, markers, fr, is_left_handed=is_left)
            p_dev = compute_pronation(points, markers, fr, is_left_handed=is_left)
            f_dev = compute_wrist_flexion(points, markers, fr, is_left_handed=is_left)
            ulnar_series.append(u_dev)
            pron_series.append(p_dev)
            flex_series.append(f_dev)

        ulnar_series = np.array(ulnar_series)
        pron_series  = np.array(pron_series)
        flex_series  = np.array(flex_series)

        pitch_stability_score = compute_pitch_stability_score(
            ulnar_series, pron_series, flex_series
        )

        # For DB insertion, let's store these new angles directly (–40..+40)
        # no more storing 180-angle
        mid_idx = len(ulnar_series)//2
        release_idx = len(ulnar_series)-20

        # we want 1..10 frames after foot_contact for e.g. 
        # (But foot_contact->foot_contact+10 might not be valid if not enough frames?)
        # We'll assume we have at least 10 frames
        # Just do min(10, len(ulnar_series)) 
        n_for_frames = min(11, len(ulnar_series))  # index 0..10 => 11 points

        # Build insert
        insert_sql = """
            INSERT INTO pitch_data (
                participant_name, pitch_date, pitch_type, filename,
                pitch_stability_score,
                mid_u_dev, rel_u_dev,
                frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
                frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
                mid_pronation, rel_pronation,
                frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
                frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
            )
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        """

        data_tuple = (
            participant_name, 
            pitch_date,
            pitch_type,
            filename_noext,

            float(pitch_stability_score),

            float(ulnar_series[mid_idx]) if mid_idx<len(ulnar_series) else None,
            float(ulnar_series[release_idx]) if release_idx<len(ulnar_series) else None,

            float(ulnar_series[1]) if n_for_frames>1 else None,
            float(ulnar_series[2]) if n_for_frames>2 else None,
            float(ulnar_series[3]) if n_for_frames>3 else None,
            float(ulnar_series[4]) if n_for_frames>4 else None,
            float(ulnar_series[5]) if n_for_frames>5 else None,
            float(ulnar_series[6]) if n_for_frames>6 else None,
            float(ulnar_series[7]) if n_for_frames>7 else None,
            float(ulnar_series[8]) if n_for_frames>8 else None,
            float(ulnar_series[9]) if n_for_frames>9 else None,
            float(ulnar_series[10]) if n_for_frames>10 else None,

            float(pron_series[mid_idx]) if mid_idx<len(pron_series) else None,
            float(pron_series[release_idx]) if release_idx<len(pron_series) else None,

            float(pron_series[1]) if n_for_frames>1 else None,
            float(pron_series[2]) if n_for_frames>2 else None,
            float(pron_series[3]) if n_for_frames>3 else None,
            float(pron_series[4]) if n_for_frames>4 else None,
            float(pron_series[5]) if n_for_frames>5 else None,
            float(pron_series[6]) if n_for_frames>6 else None,
            float(pron_series[7]) if n_for_frames>7 else None,
            float(pron_series[8]) if n_for_frames>8 else None,
            float(pron_series[9]) if n_for_frames>9 else None,
            float(pron_series[10]) if n_for_frames>10 else None
        )

        cursor.execute(insert_sql, data_tuple)
        conn.commit()
        print(f"Inserted pitch => {filename_noext} for {participant_name}: Score={pitch_stability_score}")

    conn.close()
    print("DONE processing folder.")

# If you want to run it directly:
if __name__ == "__main__":
    # Prompt user for folder
    root = tk.Tk()
    root.withdraw()
    folder = filedialog.askdirectory(title="Select Data Folder")
    if not folder:
        raise ValueError("No folder selected.")
    process_c3d_folder(folder)
    

# ------ OPTIONAL: Retrieve last inserted participant/date for your Dash defaults ------
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

# ----------------- HELPER FUNCTIONS -----------------
def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    options = [{"label": p, "value": p} for p in sorted(participants)]
    return options

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_type FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": pt, "value": pt} for pt in sorted(df["pitch_type"].unique())]
    # Insert "All" at the top
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT filename FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": fn, "value": fn} for fn in sorted(df["filename"].unique())]
    # Insert "All"
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """Shows a table that includes both Ulnar Deviation and Pronation comparisons."""
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    pitch_df = pd.read_sql_query(query, conn)

    # If you have a reference_data table, fetch it here:
    ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    conn.close()

    # Group for selected data
    pitch_summary = pitch_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    # Group for reference data
    ref_summary = ref_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    merged = pitch_summary.merge(ref_summary, on="pitch_type", suffixes=("_selected", "_reference"))

    # Simple "acceleration" estimate from frame2..frame10
    # For Ulnar Dev
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    # For Pronation
    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    rows = []
    for _, row in merged.iterrows():
        pitch = row["pitch_type"]
        # Ulnar Deviation row
        rows.append({
            "pitch_type": pitch,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 1)
        })
        # Ulnar Deviation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 1)
        })
        # Pronation row (store actual pronation at release in rel_u_dev column)
        rows.append({
            "pitch_type": f"{pitch} Pronation/Supination",
            # Instead of "Pronation", place the numeric pronation at release:
            "rel_u_dev": round(row["rel_pronation_selected"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 1)
        })
        # Pronation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Pronation/Supination Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 1)
        })

    comp_df = pd.DataFrame(rows)
    return comp_df


# ----------------- DASH APP SETUP -----------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# We define global defaults from the last inserted row:
default_participant = LAST_PARTICIPANT
default_date = LAST_DATE

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=get_dropdown_options(),
                value=default_participant,  # Use last processed participant
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                # We leave options empty; the callback will populate them
                options=[],
                value=default_date,  # Use last processed date
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative", 
                        "width": "440px",
                        "height": "auto",
                        "padding": "10px",  
                        "margin-left": "140px",
                        "backgroundColor": "black"  
                    }
                )
            ], style={"text-align": "right"})
        ], width=2)
    ], className="mt-3", align="center"),       

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[{"name": i, "id": i} for i in [
                            "pitch_type", "rel_u_dev",
                            "frame2_u_dev", "frame4_u_dev",
                            "frame6_u_dev", "frame8_u_dev",
                            "frame10_u_dev", "accel_u_dev"
                        ]],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=9
        ),
            # Average Score Display (1/3 width)
    dbc.Col(
        dbc.Card([
            dbc.CardHeader("Average Stability Score"),
            dbc.CardBody([
                html.H3(id="average-score", style={
                    "fontSize": "82px", 
                    "textAlign": "center", 
                    "color": "lime"
                }),
                html.P("Higher = Better Wrist Stability", style={
                    "textAlign": "center", 
                    "color": "white"
                })
            ])
        ]),
        width=3  # 1/3 of the frame
    )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody([
                    dcc.Graph(id="ulnar-dev-graph"),
                    html.P([
                        "Ulnar deviation measures how much the wrist flexes toward the ulnar side of the forearm or 'flicks' as we release the ball.",
                        html.Br(),
                        html.Strong("Moving in the negative (-) direction represents ulnar deviation "),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "24px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody([
                    dcc.Graph(id="acceleration-graph"),
                    html.P([
                        "Acceleration in the transverse and frontal planes shows how "
                            "rapidly the wrist angles are changing around release, "
                            "which can correlate with injury risk. ",
                        html.Br(),
                        html.Strong("Ideally, it’s kept minimal "),
                        "to reduce stress during throwing."
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="pronation-graph")),
                    html.P([
                        "Pronation and Supination measure how much you 'twist' the wrist. Supination through ball release is associated with the same 'flick' motion we are trying to avoid",
                        html.Br(),
                        html.Strong("Negative (-) values correspond with supination"),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="flexion-graph")),
                    html.P([
                        "Flexion at the wrist can be associated with the same 'flick' that occurs with excessive ulnar deviation"
                        "",
                        html.Br(),
                        html.Strong("Negative (-) values correspond with flexion"),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ----------------- DASH CALLBACKS -----------------
@app.callback(
    Output("average-score", "children"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value")
    ]
)
def update_average_score(selected_participant, selected_date, selected_pitch_type):
    """ Fetches and calculates the average stability score from the database. """
    conn = sqlite3.connect(db_path)
    query = f"""
        SELECT AVG(pitch_stability_score) 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}' 
        AND pitch_date = '{selected_date}'
        AND pitch_type = '{selected_pitch_type}'
    """
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"

    df = pd.read_sql_query(query, conn)
    conn.close()

    avg_score = df.iloc[0, 0]  # Extracts the single value
    if avg_score is None:
        return "N/A"

    return f"{avg_score:.2f}"

def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    options = get_date_options(selected_participant)
    # If the default_date is in the new options, keep it, else use the first
    possible_values = [o["value"] for o in options]
    if default_date in possible_values:
        return options, default_date
    return options, (options[0]["value"] if options else None)

@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None
    options = get_pitch_type_options(selected_participant, selected_date)
    pitch_types = [o["value"] for o in options]

    # Default to "Curve" if available, else "All" or the first
    if "Curve" in pitch_types:
        default_value = "Curve"
    else:
        default_value = "All" if "All" in pitch_types else (pitch_types[0] if pitch_types else None)

    return options, default_value

@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    options = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    value = options[0]["value"] if options else None
    return options, value

# ... your existing imports and code

@app.callback(
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    # 1) Table & 2) main time-series
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)
    table_data = comp_df.to_dict("records")

    # 3) reference data for "Curve"
    ref_data = get_curve_reference_on_the_fly()

    # Compute average release index
    release_frames = []
    for pt, data_dict in ts_data.items():
        for series in data_dict["ulnar_dev_series"]:
            # The last 20 frames in each series are post-release
            release_idx = len(series) - 20
            release_frames.append(release_idx)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # --------------- Ulnar Deviation ---------------
    ulnar_fig = go.Figure()
    plotted_pitch_types = set()  # Track pitch types that have been added to the legend

    # Plot reference if relevant
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # Plot actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Show legend only once per pitch type
    
        for series in data_dict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,  # Only show legend entry for the first trace
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)

    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline],
        autosize=True  # Let it expand
    )

    # --------------- Acceleration ---------------
    accel_fig = go.Figure()

    def compute_acceleration(angle_series):
        return np.diff(angle_series, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    # We'll do 30 frames before release, 40 frames after
    FRAMES_BEFORE = 30
    FRAMES_AFTER  = 40
    
    # The release point is at the 30th index in the new zoomed-in window
    release_line = dict(
        type="line",
        x0=30, x1=30,  # Middle of the new range
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    plotted_pitch_types = set()
    
    for pt, data_dict in ts_data.items():
        color_ulnar = "#bb6a74"
        color_pron  = "#2c99d4"
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["ulnar_dev_series"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]
    
            acc_u = compute_acceleration(sub_series)
            rms_u = compute_rms(acc_u)
            x_vals = list(range(len(rms_u)))
            
    
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color=color_ulnar, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        for series in data_dict["pronation"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]
    
            acc_p = compute_acceleration(sub_series)
            rms_p = compute_rms(acc_p)
            x_vals = list(range(len(rms_p)))
    
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color=color_pron, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    accel_fig.update_layout(
        title="Acceleration: ±30 Frames Before to +40 After Release",
        xaxis_title="Index (local to sub-window)",
        yaxis_title="Accel (°/frame²)",
        shapes=[release_line],
        autosize=True
    )
    
    # --------------- Pronation ---------------
    pronation_fig = go.Figure()
    plotted_pitch_types = set()
    
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )
    
    # --------------- Flexion ---------------
    flexion_fig = go.Figure()
    plotted_pitch_types = set()
    
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    print("Dash Flexion Data:", ts_data)  # See what is being plotted

    for pt, data_dict in ts_data.items():
        for series in data_dict["flexion"]:
            print(f"Pitch: {pt} | Min: {np.min(series):.2f} | Max: {np.max(series):.2f}")

    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


if __name__ == '__main__':
    app.run_server(debug=True)



Inserted pitch => Curve RH 5 for Caleb Sasser: Score=49.58
Inserted pitch => Curve RH 4 for Caleb Sasser: Score=46.95
Inserted pitch => Curve RH 3 for Caleb Sasser: Score=50.68
Inserted pitch => Curve RH 1 for Caleb Sasser: Score=46.66
Inserted pitch => Fastball RH 2 for Caleb Sasser: Score=24.31
Inserted pitch => Fastball RH 1 for Caleb Sasser: Score=37.86
Inserted pitch => Curve RH 2 for Caleb Sasser: Score=32.06
DONE processing folder.


In [47]:
import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import medfilt
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

db_path = "pitch_analysis_v4.sqlite"

# ----------------------------------------------------------------
# 1) FILTERING
# ----------------------------------------------------------------
def lowpass_filter(data, cutoff, fs, order=2):
    """
    6 Hz Butterworth lowpass filter, forward-backward filtfilt.
    data shape is (nFrames,) for 1D signals.
    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

def filter_marker_data(c3d_obj):
    """Filter all marker coordinates in the c3d file at 6 Hz."""
    points = c3d_obj['data']['points']
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    nMarkers = points.shape[1]
    nFrames = points.shape[2]

    if nFrames < 10:
        print("Not enough frames to lowpass filter; skipping filtering for this file.")
        return

    for m in range(nMarkers):
        for coord in range(3):
            raw_signal = points[coord, m, :]
            filtered = lowpass_filter(raw_signal, cutoff=6.0, fs=frame_rate, order=2)
            points[coord, m, :] = filtered

# ----------------------------------------------------------------
# 2) MARKER MAPPING
# ----------------------------------------------------------------
def resolve_marker_indices(marker_labels):
    """
    Map marker names to indices. 
    We expect the following markers at minimum:
      - Lateral_Elbow
      - Medial_Elbow
      - Wrist_Radius
      - Wrist_Ulna
      - Hand
    Return a dict { "Lateral_Elbow": index, ... } if all found, else {}.
    """
    required = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    marker_indices = {}
    for mk in required:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}
    return marker_indices

def create_virtual_hand_offset(c3d_obj):
    """
    From the static trial, compute an offset vector from the wrist center to the Hand marker.
    We'll replicate that offset in dynamic trials for consistent hand positioning.
    """
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Required markers not found in static trial to create virtual hand offset.")
    nFrames = points.shape[2]

    offsets = []
    for f in range(nFrames):
        R = points[:3, markers["Wrist_Radius"], f]
        U = points[:3, markers["Wrist_Ulna"], f]
        H = points[:3, markers["Hand"], f]
        wrist_center = (R + U) / 2.0
        offset_vec = H - wrist_center
        offsets.append(offset_vec)

    mean_offset = np.mean(offsets, axis=0)
    return mean_offset

def apply_virtual_hand_marker(points, marker_indices, hand_offset):
    """
    Override the 'Hand' marker in dynamic data with a virtual location
    based on the static offset. 
    """
    nFrames = points.shape[2]
    for f in range(nFrames):
        R = points[:3, marker_indices["Wrist_Radius"], f]
        U = points[:3, marker_indices["Wrist_Ulna"], f]
        wrist_center = (R + U) / 2.0
        new_hand = wrist_center + hand_offset
        points[:3, marker_indices["Hand"], f] = new_hand

# ----------------------------------------------------------------
# 3) EVENT DETECTION
# ----------------------------------------------------------------
def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]

    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")

    idx_foot    = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec      = event_times[idx_release]

    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global      = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)

    foot_local    = foot_contact_global - earliest
    release_local = release_global      - earliest

    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")

    return foot_local, release_local

# ----------------------------------------------------------------
# 4) ANGLE CALCULATIONS: Direct Ulnar Deviation, Pronation, Flexion
# ----------------------------------------------------------------
def compute_ulnar_deviation(points, marker_indices, frame, is_left_handed=False):
    """
    Computes the signed wrist deviation angle in the frontal plane.
    Range typically ~ -40° (radial) to +40° (ulnar).
    
    Strategy:
      1) Forearm axis from elbow center -> wrist center.
      2) Project the hand vector onto the FRONTAL plane (the plane perpendicular to the forearm axis).
      3) Determine the angle between the 'neutral' vector (e.g. a line from wrist center -> radius side, or a known reference) and the actual hand projection.
      4) The sign is determined by cross-product direction.
    is_left_handed: if True, we flip the sign so that 'ulnar' is still + for a lefty.
    """
    # Markers
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    wrist_center = (R + U) / 2.0
    elbow_center = (E_lat + E_med) / 2.0

    forearm_vec  = wrist_center - elbow_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    # We'll define a 'reference axis' in the frontal plane: from wrist_center -> radius marker
    # That means if the hand is exactly over the radius, angle = 0, 
    # deviate ulnarly -> positive, deviate radially -> negative.
    #  - For a right-handed forearm, the radius side is typically lateral. 
    #  - For a left-handed forearm, we might flip sign. But let's do that after the angle calc.
    ref_vec = R - wrist_center

    # Project both ref_vec and (hand - wrist_center) into the plane perpendicular to forearm.
    def project_to_plane(vec, normal):
        return vec - np.dot(vec, normal) * normal

    ref_proj = project_to_plane(ref_vec, forearm_unit)
    hand_vec = H - wrist_center
    hand_proj = project_to_plane(hand_vec, forearm_unit)

    # Normalize
    ref_norm  = np.linalg.norm(ref_proj) + 1e-9
    hand_norm = np.linalg.norm(hand_proj) + 1e-9
    ref_unit  = ref_proj / ref_norm
    hand_unit = hand_proj / hand_norm

    # Signed angle in the plane
    cross_2d = np.cross(ref_unit, hand_unit)
    dot_2d   = np.dot(ref_unit, hand_unit)

    angle_rad = np.arctan2(np.linalg.norm(cross_2d), dot_2d)
    # sign: if cross_2d is 'above' or 'below' the plane w.r.t forearm_unit
    # But simpler might be sign = np.sign(np.dot(cross_2d, forearm_unit)) 
    sign = np.sign(np.dot(cross_2d, forearm_unit))

    angle_deg = np.degrees(angle_rad) * sign

    # So angle_deg ~ +30 => 30 deg ulnar dev, -20 => 20 deg radial dev, etc.
    # If left-handed, we can flip the sign so that "ulnar" is still positive.
    if is_left_handed:
        angle_deg = -angle_deg

    return angle_deg

def compute_pronation(points, marker_indices, frame, is_left_handed=False):
    """
    Similar approach to track pronation/supination around the forearm axis.
    Positive => pronation, negative => supination.
    Flip sign for lefty if you want consistent reporting.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]

    wrist_center = (R + U) / 2.0
    elbow_center = (E_lat + E_med) / 2.0

    forearm_vec  = wrist_center - elbow_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    # For pronation, we measure the rotation of the R->U vector about the forearm axis.
    # Vector from R->U
    RU = U - R

    # We'll define a reference as RU in the neutral supination position, but we 
    # can use E_lat->E_med if that helps. For simplicity, let's just use frames 0..some baseline,
    # or do the angle relative to some global orientation. 
    # For now, do the angle relative to 'vertical cross' or something. 
    # This is a simpler approach, though not a standard jcs. 
    # We'll do a cross approach: cross RU with forearm => measure sign. 
    # Or do an explicit plane approach. We'll keep it simpler:

    # Let's define a local orientation:
    #  - Project RU onto plane perpendicular to forearm_unit
    #  - Compare it to a "neutral" that points 'lateral' in that plane
    # Actually, let's do a direct angle between RU and 'elbow_lat -> elbow_med' or something 
    # you had in your code. We'll keep the sign approach. 
    # To keep it simpler, I'll do a cross/dot:

    # Project RU onto plane:
    RU_proj = RU - np.dot(RU, forearm_unit) * forearm_unit
    # Also define an 'elbow lat->med' for reference:
    E_vec = E_med - E_lat
    E_proj = E_vec - np.dot(E_vec, forearm_unit)*forearm_unit

    RU_n = RU_proj / (np.linalg.norm(RU_proj)+1e-9)
    E_n  = E_proj  / (np.linalg.norm(E_proj)+1e-9)

    cross_val = np.cross(E_n, RU_n)
    dot_val   = np.dot(E_n, RU_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))
    sign      = np.sign(np.dot(cross_val, forearm_unit))

    angle_deg = np.degrees(angle_rad) * sign

    # Now, + => pronation, - => supination, presumably
    # If left-handed, flip sign if we want the same convention for LH
    if is_left_handed:
        angle_deg = -angle_deg

    return angle_deg

def compute_wrist_flexion(points, marker_indices, frame, is_left_handed=False):
    """
    Similar approach to measure flexion/extension around the mediolateral axis. 
    We define the forearm axis, then define a 'vertical' or a 'rest' orientation 
    to measure how much the hand flexes up/down. 
    For consistency, we can define: 
      + => extension, - => flexion
    or vice versa. Then apply is_left_handed sign flip if desired.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    wrist_center = (R + U) / 2.0
    elbow_center = (E_lat + E_med) / 2.0

    forearm_vec  = wrist_center - elbow_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec)+1e-9)

    # We want the plane orthonormal basis for flex/extension. 
    # Let's define 'vertical' = global Z axis = [0,0,1], project that onto plane 
    # perpendicular to forearm_unit, then measure angle from that to the hand vector.
    vertical = np.array([0,0,1], dtype=float)
    def project_to_plane(vec, normal):
        return vec - np.dot(vec, normal)*normal

    hand_vec  = H - wrist_center
    hand_proj = project_to_plane(hand_vec, forearm_unit)

    vert_proj = project_to_plane(vertical, forearm_unit)

    # Normalize
    hp_n = hand_proj / (np.linalg.norm(hand_proj)+1e-9)
    vp_n = vert_proj / (np.linalg.norm(vert_proj)+1e-9)

    cross_val = np.cross(vp_n, hp_n)
    dot_val   = np.dot(vp_n, hp_n)
    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign      = np.sign(np.dot(cross_val, forearm_unit))

    angle_deg = np.degrees(angle_rad)*sign
    # + => extension, - => flexion (arbitrary choice)

    if is_left_handed:
        # If we want the same sign convention for lefties, we can flip sign
        angle_deg = -angle_deg

    return angle_deg

# ----------------------------------------------------------------
# 5) PITCH STABILITY SCORE
# ----------------------------------------------------------------
def compute_pitch_stability_score(ulnar_dev_series, pronation_series, flexion_series):
    """
    Updated approach to incorporate absolute magnitude and consistency (variation).
    1) Baseline angle at foot contact is assumed ~ 0 or small. We measure how far from that 
       the angles get around release.
    2) Also measure how quickly they change (flick).
    3) Weighted combination => final score (0..100).
    """
    total_frames = len(ulnar_dev_series)
    if total_frames < 20:
        return 0.0

    release_idx = total_frames - 20  # 'release' is 20 frames before end

    # We define a window ~ release_idx +/- 5 for measuring angle
    window_start = max(release_idx - 5, 0)
    window_end   = min(release_idx + 5, total_frames)

    # Slices
    ulnar_slice = ulnar_dev_series[window_start:window_end]
    pron_slice  = pronation_series[window_start:window_end]
    flex_slice  = flexion_series[window_start:window_end]

    # 1) absolute magnitude penalty => if angle is large in the slice, reduce score
    # typical feasible ranges: 
    #   - ulnar dev ~ +/- 40 
    #   - pronation ~ +/- 80 
    #   - flexion ~ +/- 70 
    # We'll define a function that gives 100 if angle ~ 0, and 0 if angle hits the max range.
    def angle_score(angle_val, max_allowed):
        # e.g. if angle_val= 20, max_allowed= 40 => 50% used => 50 left => 50 on scale => 50 => 50. 
        # simpler: sc = 100 - (abs(angle_val)/max_allowed * 100)
        sc = 100 - (abs(angle_val)/max_allowed)*100
        return np.clip(sc, 0, 100)

    # We'll compute an average magnitude penalty across frames in that slice
    # then average it with a variability penalty.
    # Let's do separate weighting for each angle:
    #   - Ulnar dev max ~ 40
    #   - Pronation max ~ 80
    #   - Flexion max ~ 70
    # Then we average them
    ulnar_scores = [angle_score(a, 40) for a in ulnar_slice]
    pron_scores  = [angle_score(a, 80) for a in pron_slice]
    flex_scores  = [angle_score(a, 70) for a in flex_slice]

    mean_ulnar_mag_score = np.mean(ulnar_scores)  # 0..100
    mean_pron_mag_score  = np.mean(pron_scores)
    mean_flex_mag_score  = np.mean(flex_scores)

    # 2) Variation penalty => if angles swing widely in that slice, reduce score
    # We'll do the standard deviation in the slice or range.
    # e.g. if there's a 40° swing in the slice for ulnar dev, that's large flick => big penalty
    # Let's define a function that returns a 0..100 penalty for a stdev or range.
    # We'll define 'max_stdev' for each angle. If stdev is that or more => 0 score, 
    # if stdev is 0 => 100 score
    max_stdev_ulnar = 20  # if stdev ~ 20 => 0
    max_stdev_pron  = 30
    max_stdev_flex  = 25

    ulnar_std = np.std(ulnar_slice)
    pron_std  = np.std(pron_slice)
    flex_std  = np.std(flex_slice)

    def variation_score(std_val, max_stdev):
        vs = 100 - (std_val / max_stdev)*100
        return np.clip(vs, 0, 100)

    ulnar_var_score = variation_score(ulnar_std, max_stdev_ulnar)
    pron_var_score  = variation_score(pron_std, max_stdev_pron)
    flex_var_score  = variation_score(flex_std, max_stdev_flex)

    # Weighted combination
    # example weighting:
    #  - For magnitude:  60% 
    #  - For variation:  40%
    #  - Among angles: each angle could weigh equally or not
    # We'll do a simple equal weighting among angles (ulnar/pron/flex => 3 ways).
    # Then combine magnitude and variation as 60/40.
    mag_ulnar = mean_ulnar_mag_score
    mag_pron  = mean_pron_mag_score
    mag_flex  = mean_flex_mag_score

    var_ulnar = ulnar_var_score
    var_pron  = pron_var_score
    var_flex  = flex_var_score

    mean_mag_score = np.mean([mag_ulnar, mag_pron, mag_flex])
    mean_var_score = np.mean([var_ulnar, var_pron, var_flex])

    final_score = 0.6 * mean_mag_score + 0.4 * mean_var_score
    return round(final_score, 2)

# ----------------------------------------------------------------
# 6) HAND PREFERENCE DETECTION
# ----------------------------------------------------------------
def detect_left_handed(filename_noext):
    """
    Simple approach:
     - if 'LH' or 'Left' in the filename (case-insensitive), return True
     - otherwise False
    """
    fname_lower = filename_noext.lower()
    if 'lh' in fname_lower or 'left' in fname_lower:
        return True
    return False

# ----------------------------------------------------------------
# (7) The MAIN PROCESSING LOOP
#     (like your original but with the new angle calculations)
# ----------------------------------------------------------------
def process_c3d_folder(selected_folder):
    # 1) find c3d
    c3d_files = [os.path.join(selected_folder, f)
                 for f in os.listdir(selected_folder)
                 if f.lower().endswith('.c3d')]

    # 2) find static
    static_files = [f for f in c3d_files if "static" in f.lower()]
    if not static_files:
        raise FileNotFoundError("No static trial found in the folder.")
    static_path = static_files[0]  # pick one

    # 3) create db table if not exist
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS reference_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        participant_name TEXT,
        pitch_date TEXT,
        pitch_type TEXT,
        filename TEXT,
        pitch_stability_score REAL,
        mid_u_dev REAL,
        rel_u_dev REAL,
        frame1_u_dev REAL,
        frame2_u_dev REAL,
        frame3_u_dev REAL,
        frame4_u_dev REAL,
        frame5_u_dev REAL,
        frame6_u_dev REAL,
        frame7_u_dev REAL,
        frame8_u_dev REAL,
        frame9_u_dev REAL,
        frame10_u_dev REAL,
        mid_pronation REAL,
        rel_pronation REAL,
        frame1_pronation REAL,
        frame2_pronation REAL,
        frame3_pronation REAL,
        frame4_pronation REAL,
        frame5_pronation REAL,
        frame6_pronation REAL,
        frame7_pronation REAL,
        frame8_pronation REAL,
        frame9_pronation REAL,
        frame10_pronation REAL
    )
    """)
    conn.commit()

    # 4) load static, filter, compute offset
    static_c3d = ezc3d.c3d(static_path)
    filter_marker_data(static_c3d)
    static_offset = create_virtual_hand_offset(static_c3d)

    # 5) loop dynamic
    for c3d_file in c3d_files:
        if c3d_file == static_path:
            continue

        c3d_obj = ezc3d.c3d(c3d_file)
        filter_marker_data(c3d_obj)

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file}, missing required markers.")
            continue

        apply_virtual_hand_marker(points, markers, static_offset)

        frame_rate  = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames= points.shape[2]
        try:
            foot_contact, release = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file}, event issue: {e}")
            continue

        if release + 20 >= total_frames:
            print(f"Skipping {c3d_file}, not enough frames post-release.")
            continue

        # parse participant, date, pitch type
        normalized_path = os.path.normpath(c3d_file)
        path_parts = normalized_path.split(os.sep)
        # adapt if your folder structure differs
        if len(path_parts) < 3:
            continue

        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder      = path_parts[-2].rstrip("_")
        pitch_date       = date_folder
        filename_only    = path_parts[-1]
        filename_noext   = os.path.splitext(filename_only)[0]
        pitch_type       = filename_noext.split()[0].capitalize()

        is_left = detect_left_handed(filename_noext)

        # extract time-series
        start_frame = foot_contact
        end_frame   = release + 20
        ulnar_series = []
        pron_series  = []
        flex_series  = []

        for fr in range(start_frame, end_frame):
            u_dev = compute_ulnar_deviation(points, markers, fr, is_left_handed=is_left)
            p_dev = compute_pronation(points, markers, fr, is_left_handed=is_left)
            f_dev = compute_wrist_flexion(points, markers, fr, is_left_handed=is_left)
            ulnar_series.append(u_dev)
            pron_series.append(p_dev)
            flex_series.append(f_dev)

        ulnar_series = np.array(ulnar_series)
        pron_series  = np.array(pron_series)
        flex_series  = np.array(flex_series)

        pitch_stability_score = compute_pitch_stability_score(
            ulnar_series, pron_series, flex_series
        )

        # For DB insertion, let's store these new angles directly (–40..+40)
        # no more storing 180-angle
        mid_idx = len(ulnar_series)//2
        release_idx = len(ulnar_series)-20

        # we want 1..10 frames after foot_contact for e.g. 
        # (But foot_contact->foot_contact+10 might not be valid if not enough frames?)
        # We'll assume we have at least 10 frames
        # Just do min(10, len(ulnar_series)) 
        n_for_frames = min(11, len(ulnar_series))  # index 0..10 => 11 points

        # Build insert
        insert_sql = """
            INSERT INTO reference_data (
                participant_name, pitch_date, pitch_type, filename,
                pitch_stability_score,
                mid_u_dev, rel_u_dev,
                frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
                frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
                mid_pronation, rel_pronation,
                frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
                frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
            )
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        """

        data_tuple = (
            participant_name, 
            pitch_date,
            pitch_type,
            filename_noext,

            float(pitch_stability_score),

            float(ulnar_series[mid_idx]) if mid_idx<len(ulnar_series) else None,
            float(ulnar_series[release_idx]) if release_idx<len(ulnar_series) else None,

            float(ulnar_series[1]) if n_for_frames>1 else None,
            float(ulnar_series[2]) if n_for_frames>2 else None,
            float(ulnar_series[3]) if n_for_frames>3 else None,
            float(ulnar_series[4]) if n_for_frames>4 else None,
            float(ulnar_series[5]) if n_for_frames>5 else None,
            float(ulnar_series[6]) if n_for_frames>6 else None,
            float(ulnar_series[7]) if n_for_frames>7 else None,
            float(ulnar_series[8]) if n_for_frames>8 else None,
            float(ulnar_series[9]) if n_for_frames>9 else None,
            float(ulnar_series[10]) if n_for_frames>10 else None,

            float(pron_series[mid_idx]) if mid_idx<len(pron_series) else None,
            float(pron_series[release_idx]) if release_idx<len(pron_series) else None,

            float(pron_series[1]) if n_for_frames>1 else None,
            float(pron_series[2]) if n_for_frames>2 else None,
            float(pron_series[3]) if n_for_frames>3 else None,
            float(pron_series[4]) if n_for_frames>4 else None,
            float(pron_series[5]) if n_for_frames>5 else None,
            float(pron_series[6]) if n_for_frames>6 else None,
            float(pron_series[7]) if n_for_frames>7 else None,
            float(pron_series[8]) if n_for_frames>8 else None,
            float(pron_series[9]) if n_for_frames>9 else None,
            float(pron_series[10]) if n_for_frames>10 else None
        )

        cursor.execute(insert_sql, data_tuple)
        conn.commit()
        print(f"Inserted pitch => {filename_noext} for {participant_name}: Score={pitch_stability_score}")

    conn.close()
    print("DONE processing folder.")

# If you want to run it directly:
if __name__ == "__main__":
    # Prompt user for folder
    root = tk.Tk()
    root.withdraw()
    folder = filedialog.askdirectory(title="Select Data Folder")
    if not folder:
        raise ValueError("No folder selected.")
    process_c3d_folder(folder)


Inserted pitch => Slider RH 3 for Reference Data: Score=59.62
Inserted pitch => Slider RH 2 for Reference Data: Score=57.16
Inserted pitch => Slider RH 1 for Reference Data: Score=58.84
Inserted pitch => Changeup RH 2 for Reference Data: Score=54.17
Inserted pitch => Curve RH 3 for Reference Data: Score=55.82
Inserted pitch => Curve RH 2 for Reference Data: Score=58.79
Inserted pitch => Fastball RH 3 for Reference Data: Score=58.61
Inserted pitch => Fastball RH 2 for Reference Data: Score=56.4
Skipping D:/Youth Pitch Design/Data/Reference Data_RD/2025-02-21_\Fastball RH 1.c3d, event issue: Required events (Foot Contact, Release) not found in C3D events.
Inserted pitch => Changeup RH 1 for Reference Data: Score=58.28
Inserted pitch => Curve RH 1 for Reference Data: Score=50.74
DONE processing folder.


In [43]:
# This is it. Does it all for processing data. 

import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import medfilt
import matplotlib.pyplot as plt


# Dash and Plotly imports
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis_v3.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    pitch_stability_score REAL,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL

)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")


# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [os.path.join(selected_folder, file)
             for file in os.listdir(selected_folder)
             if file.lower().endswith('.c3d')]

if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- IDENTIFY STATIC TRIAL -----------------
static_files = [f for f in c3d_files if "static" in f.lower()]
if not static_files:
    raise FileNotFoundError("No static trial found in the dataset.")


# ----------------- HELPER FUNCTIONS -----------------

def lowpass_filter(data, cutoff, fs, order=2):
    """
    6 Hz Butterworth lowpass filter, forward-backward filtfilt.
    data shape is (nFrames,) for 1D signals.
    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

# ----------------- GET TIME SERIES (for graphs) -----------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    
    for c3d_file_path in c3d_files:
        normalized_path = os.path.normpath(c3d_file_path)
        path_parts = normalized_path.split(os.sep)
        if len(path_parts) < 3:
            continue
        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = path_parts[-2].rstrip("_")

        if participant_name != selected_participant or date_folder != selected_date:
            continue

        filename_only = path_parts[-1]
        filename_noext = os.path.splitext(filename_only)[0]
        pitch_type = filename_noext.split()[0].capitalize()

        if selected_pitch_type != "All" and pitch_type != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        try:
            c3d_obj = ezc3d.c3d(c3d_file_path)
        except Exception as e:
            print(f"Error reading {c3d_file_path}: {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame = release_frame + 20
        ulnar_series = []
        pronation_series = []
        flexion_series = []

        #SINGLE LOOP - Avoiding Double Processing
        prev_flexion_angle = None  # Track previous angle for smooth transitions
        for frame in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame)
            p_angle = compute_pronation(points, markers, frame)
            f_angle = compute_wrist_flexion(points, markers, frame, prev_angle=prev_flexion_angle)

            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)
            prev_flexion_angle = f_angle  # Update previous angle

        # Convert lists to numpy arrays **AFTER THE LOOP**
        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        # # Apply np.unwrap to ensure smooth transitions
        # flexion_series = np.unwrap(np.radians(flexion_series), discont=np.radians(180))
        # flexion_series = np.degrees(flexion_series)
        # 
        # # Apply median filter to remove single-frame spikes
        # flexion_series = medfilt(flexion_series, kernel_size=5)

        if pitch_type not in ts:
            ts[pitch_type] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}

        ts[pitch_type]["ulnar_dev_series"].append(ulnar_series)
        ts[pitch_type]["pronation"].append(pronation_series)
        ts[pitch_type]["flexion"].append(flexion_series)
        
        print(f"Dash route => {filename_noext}: foot_contact={foot_contact_frame}, release={release_frame}")
        print(f"Dash route => flexion length={len(flexion_series)}, min={np.min(flexion_series)}, max={np.max(flexion_series)}")

    return ts


def get_curve_reference_on_the_fly():
    """
    Only loads 'Curve' .c3d files from a known reference folder and averages them.
    Adjust as needed for your actual reference approach.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        return {}

    all_files = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d') and "curve" in f.lower()
    ]
    if not all_files:
        print("No valid 'Curve' .c3d files found in reference folder.")
        return {}

    all_ulnar_arrays = []
    all_pronation_arrays = []
    all_flexion_arrays = []

    for c3d_path in all_files:
        try:
            c3d_obj = ezc3d.c3d(c3d_path)
        except Exception as e:
            print(f"Skipping {c3d_path}: read error {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame   = release_frame + 20

        ulnar_series = []
        pronation_series = []
        flexion_series = []
        for frame_idx in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame_idx)
            p_angle = compute_pronation(points, markers, frame_idx)
            f_angle = compute_wrist_flexion(points, markers, frame_idx)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        all_ulnar_arrays.append(ulnar_series)
        all_pronation_arrays.append(pronation_series)
        all_flexion_arrays.append(flexion_series)

    if not all_ulnar_arrays:
        return {}

    # Trim to min length across all files
    min_len = min([arr.shape[0] for arr in all_ulnar_arrays + all_pronation_arrays + all_flexion_arrays])
    all_ulnar_arrays = [arr[:min_len] for arr in all_ulnar_arrays]
    all_pronation_arrays = [arr[:min_len] for arr in all_pronation_arrays]
    all_flexion_arrays = [arr[:min_len] for arr in all_flexion_arrays]

    stacked_ulnar = np.vstack(all_ulnar_arrays)
    stacked_pron  = np.vstack(all_pronation_arrays)
    stacked_flex  = np.vstack(all_flexion_arrays)

    mean_ulnar = np.mean(stacked_ulnar, axis=0)
    mean_pron  = np.mean(stacked_pron,  axis=0)
    mean_flex  = np.mean(stacked_flex,  axis=0)

    # Return only "Curve" references
    ref_data = {
        "Curve": {
            "ulnar_dev_series": [mean_ulnar],
            "pronation": [mean_pron],
            "flexion": [mean_flex]
        }
    }
    return ref_data

# For a table-based reference (if you have a 'reference_data' table), you'd have a function like:
# def get_reference_time_series():
#     ... your table approach ...
#     return reference_ts

def resolve_marker_indices(marker_labels):
    """
    Map marker names to indices, removing 'Right_'/'Left_' prefix.
    Returns a dict of required markers if all are present.
    """
    # This is the set we need
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    # Strip 'Right_'/'Left_' from each label to unify
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}  # required marker not found
    return marker_indices

def filter_marker_data(c3d_obj):
    points = c3d_obj['data']['points']
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    nMarkers = points.shape[1]
    nFrames = points.shape[2]

    # If too few frames for filtfilt’s default pad, skip filtering
    if nFrames < 10:
        print("Not enough frames to lowpass filter; skipping filtering for this file.")
        return

    for m in range(nMarkers):
        for coord in range(3):
            raw_signal = points[coord, m, :]  # shape (nFrames,)
            filtered = lowpass_filter(raw_signal, cutoff=6.0, fs=frame_rate, order=2)
            points[coord, m, :] = filtered


def create_virtual_hand_offset(c3d_obj):
    """
    From the static trial, compute an offset vector to define a stable hand marker.
    For example, compute the vector from the wrist center to the 'Hand' marker.
    We'll replicate that offset in dynamic trials to help define the hand segment.
    """
    points = c3d_obj["data"]["points"]  # shape: (4, nMarkers, nFrames)
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Required markers not found in static trial to create virtual hand offset.")

    nFrames = points.shape[2]

    # We'll average across all static frames
    wrist_centers = []
    hand_vectors  = []

    for f in range(nFrames):
        R = points[:3, markers["Wrist_Radius"], f]
        U = points[:3, markers["Wrist_Ulna"], f]
        H = points[:3, markers["Hand"], f]
        wrist_center = (R + U) / 2.0
        # Vector from wrist_center to the 'Hand' marker
        offset_vec = H - wrist_center

        wrist_centers.append(wrist_center)
        hand_vectors.append(offset_vec)

    # Mean offset vector from wrist center to hand
    mean_hand_offset = np.mean(hand_vectors, axis=0)
    return mean_hand_offset

def apply_virtual_hand_marker(points, marker_indices, hand_offset):
    """
    Override the 'Hand' marker in dynamic data with a virtual location
    based on the static offset. For each frame:
      Hand_virtual = wrist_center + hand_offset
    This ensures the hand marker is in a consistent location relative to the wrist center.
    """
    nFrames = points.shape[2]
    for f in range(nFrames):
        R = points[:3, marker_indices["Wrist_Radius"], f]
        U = points[:3, marker_indices["Wrist_Ulna"], f]
        wrist_center = (R + U) / 2.0

        # new hand location
        new_hand = wrist_center + hand_offset
        # override the original 'Hand' marker
        points[:3, marker_indices["Hand"], f] = new_hand

def compute_ulnar_deviation(points, marker_indices, frame):
    """
    Computes radial/ulnar deviation angle relative to the forearm axis.
    Higher angle => more radial dev if sign is +, or more ulnar dev if sign is -,
    depending on final logic. We'll just keep your existing formula but note
    we subtract from 180 at the end.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = E_center - W_center
    radial_vec = R - U
    hand_vec = H - W_center

    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit) * forearm_unit
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0

    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val = np.dot(radial_proj_n, hand_proj_n)

    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    # In your existing code, you do: return 180 - abs(angle_deg)
    return 180.0 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit) * forearm_unit

    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0

    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)

    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))

    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180.0 - abs(raw_angle)

def compute_wrist_flexion(points, marker_indices, frame, prev_angle=None):
    """
    Computes wrist flexion/extension relative to the global vertical axis.
    Includes fixes for discontinuities using np.unwrap and median filtering.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit) * forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)

    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)

    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))

    raw_angle = np.degrees(angle_rad) * sign

    # 🔹 Ensure smooth transitions using previous angle
    if prev_angle is not None:
        angle_diff = raw_angle - prev_angle
        if abs(angle_diff) > 90:  # If the jump is more than 90°, fix it
            raw_angle -= np.sign(angle_diff) * 180  # Flip to maintain continuity

    return 180.0 - abs(raw_angle)

def plot_wrist_flexion(flexion_series, participant_name, pitch_type, filename_noext, foot_contact_frame, release_frame):
    """
    Plots the wrist flexion time series for a given participant.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(flexion_series, label="Wrist Flexion", color="blue", linewidth=2)
    plt.axvline(x=len(flexion_series) - 20, color="red", linestyle="--", label="Release Point")
    
    plt.xlabel("Frame")
    plt.ylabel("Flexion Angle (°)")
    plt.title(f"Wrist Flexion Over Time ({participant_name} - {pitch_type} - {filename_noext})")
    plt.legend()
    plt.grid()

    # Force display
    plt.show(block=True)  # Ensures it stays open
    plt.savefig(f"{participant_name}_{pitch_type}_{filename_noext}_flexion.png")
    print(f"Flexion plot saved as {participant_name}_{pitch_type}_{filename_noext}_flexion.png")
    print(f"Matplotlib route => {filename_noext}: foot_contact={foot_contact_frame}, release={release_frame}, len={len(flexion_series)}")
    print(f"Matplotlib route => flexion min={flexion_series.min()}, max={flexion_series.max()}")


    
def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]

    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")

    idx_foot = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec = event_times[idx_release]

    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)

    foot_local = foot_contact_global - earliest
    release_local = release_global - earliest

    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")
    return foot_local, release_local

def compute_pitch_stability_score(ulnar_dev_series, pronation_series, flexion_series, accel_ulnar_series, accel_pron_series):
    """
    Computes a wrist stability score (0-100), where higher = better.

    The joint angle differences are computed as the difference between the baseline
    (frame 0, at foot contact) and the average over a 10-frame window centered on ball release.
    
    Inputs:
    - ulnar_dev_series: Time-series of ulnar deviation (frames)
    - pronation_series: Time-series of pronation angles (frames)
    - flexion_series: Time-series of flexion angles (frames)
    - accel_ulnar_series: Time-series of ulnar deviation acceleration
    - accel_pron_series: Time-series of pronation acceleration

    Returns:
    - Stability score (0-100)
    """
    # 1. Frame Indexing (define key moments)
    # Baseline is at frame 0 (foot contact)
    baseline_idx = 0
    # Define the release frame (ball release is assumed to be at index: total_frames - 20)
    release_idx = len(ulnar_dev_series) - 20

    # Define a window for release measurements: 5 frames before and 5 frames after release_idx
    window_start = max(release_idx - 5, 0)
    window_end   = min(release_idx + 5, len(ulnar_dev_series))
    
    # Compute the mean joint angles in that 10-frame window
    mean_ulnar_release = np.median(ulnar_dev_series[window_start:window_end])
    mean_pronation_release = np.median(pronation_series[window_start:window_end])
    mean_flexion_release = np.median(flexion_series[window_start:window_end])
    
    # Use the baseline at frame 0 (or you could also average a few frames at the start if desired)
    baseline_ulnar = ulnar_dev_series[baseline_idx]
    baseline_pronation = pronation_series[baseline_idx]
    baseline_flexion = flexion_series[baseline_idx]

    # 2. Compute absolute movement (penalizes excessive motion)
    max_ulnar_range = 30   # Expected max deviation (in degrees)
    max_pronation_range = 45 # Supination shouldn't be more than 45°
    max_flexion_range = 40   # Wrist flick shouldn't be excessive

    ulnar_stability = 100 - (abs(mean_ulnar_release - baseline_ulnar) / max_ulnar_range) * 100
    pronation_stability = 100 - (abs(mean_pronation_release - baseline_pronation) / max_pronation_range) * 100
    flexion_stability = 100 - (abs(mean_flexion_release - baseline_flexion) / max_flexion_range) * 100

    # 3. Compute acceleration penalty (using a window around release as before)
    accel_start = max(0, release_idx - 5)
    accel_end   = min(len(ulnar_dev_series), release_idx + 5)
    rms_accel_ulnar = np.sqrt(np.mean(np.square(accel_ulnar_series[accel_start:accel_end])))
    rms_accel_pron  = np.sqrt(np.mean(np.square(accel_pron_series[accel_start:accel_end])))

    max_expected_accel = 10  # Expected max acceleration in deg/frame²
    accel_stability = 100 - ((rms_accel_ulnar + rms_accel_pron) / max_expected_accel) * 100

    # 4. Ensure values are within [0, 100] range
    ulnar_stability = np.clip(ulnar_stability, 0, 100)
    pronation_stability = np.clip(pronation_stability, 0, 100)
    flexion_stability = np.clip(flexion_stability, 0, 100)
    accel_stability = np.clip(accel_stability, 0, 100)

    # 5. Weighted score (adjust weights if needed)
    # (Note: the weights in your current code sum to more than 1, but they can be adjusted as desired.)
    weights = [0.45, 0.20, 0.05, 0.30]
    final_score = (
        weights[0] * ulnar_stability +
        weights[1] * pronation_stability +
        weights[2] * flexion_stability +
        weights[3] * accel_stability
    )

    return round(final_score, 2)



# ---------------------- PROCESS STATIC TRIAL ----------------------
print("Processing static trial:", static_files[0])
static_c3d = ezc3d.c3d(static_files[0])

# 1) Filter static data at 6 Hz
filter_marker_data(static_c3d)

# 2) Create a consistent 'hand offset' from the static trial
static_hand_offset = create_virtual_hand_offset(static_c3d)

# 3) Compute baseline angles from the filtered, original static trial
def compute_static_baseline(c3d_obj):
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Static trial missing required markers.")
    nFrames = points.shape[2]

    ulnar_vals = []
    pron_vals  = []
    flex_vals  = []

    for f in range(nFrames):
        u = compute_ulnar_deviation(points, markers, f)
        p = compute_pronation(points, markers, f)
        x = compute_wrist_flexion(points, markers, f)
        ulnar_vals.append(u)
        pron_vals.append(p)
        flex_vals.append(x)

    return {
        "ulnar_dev": np.mean(ulnar_vals),
        "pronation": np.mean(pron_vals),
        "flexion":   np.mean(flex_vals)
    }

static_baseline_vals = compute_static_baseline(static_c3d)
print("Static baseline angles:", static_baseline_vals)


# ----------------- APPLY TO PITCH DATA -----------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file in c3d_files:
    if "static" in c3d_file.lower():
        continue  # Skip static trial

    c3d_obj = ezc3d.c3d(c3d_file)

    # 1) Filter dynamic data at 6 Hz
    filter_marker_data(c3d_obj)

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file}: required markers not found.")
        continue

    # 2) Apply our "virtual hand marker" override so it remains consistent
    apply_virtual_hand_marker(points, markers, static_hand_offset)

    # 3) Identify events
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]
    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file}: {e}")
        continue

    # Ensure enough frames post-release
    if release_frame + 20 >= total_frames:
        print(f"Skipping {c3d_file}: Not enough frames for release+20.")
        continue

    # 4) Basic info for DB
    normalized_path = os.path.normpath(c3d_file)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2].rstrip("_")
    pitch_date = date_folder
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]
    pitch_type = filename_noext.split()[0].capitalize()

    # 5) Extract Time-Series Data
    start_frame = foot_contact_frame
    end_frame = release_frame + 20
    ulnar_series = []
    pronation_series = []
    flexion_series = []

    for frame in range(start_frame, end_frame):
        ulnar_series.append(compute_ulnar_deviation(points, markers, frame))
        pronation_series.append(compute_pronation(points, markers, frame))
        flexion_series.append(compute_wrist_flexion(points, markers, frame))

    # Convert lists to NumPy arrays for processing
    ulnar_series = np.array(ulnar_series)
    pronation_series = np.array(pronation_series)
    flexion_series = np.array(flexion_series)
    print(f"Dash route => All Flexion Angles: {flexion_series}")
    print(f"Matplotlib route => All Flexion Angles: {flexion_series}")

    # 6) Compute Acceleration
    def compute_acceleration(angle_series):
        return np.gradient(np.gradient(angle_series))  # Second derivative

    accel_ulnar_series = compute_acceleration(ulnar_series)
    accel_pron_series = compute_acceleration(pronation_series)

    # 7) Compute Stability Score
    pitch_stability_score = compute_pitch_stability_score(
        ulnar_series, 
        pronation_series, 
        flexion_series, 
        accel_ulnar_series,  
        accel_pron_series
    )
    
    print(f"Processing {filename_noext} for {participant_name} - {pitch_type}")
    # plot_wrist_flexion(flexion_series, participant_name, pitch_type, filename_noext)

    print(f"Inserted {filename_noext} ({pitch_type}) - Stability Score: {pitch_stability_score}")

    # 8) Insert into Database
    insert_sql = """
        INSERT INTO pitch_data (
            participant_name, pitch_date, pitch_type, filename,
            pitch_stability_score,
            mid_u_dev, rel_u_dev,
            frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
            frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
            mid_pronation, rel_pronation,
            frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
            frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
        )
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(pitch_stability_score),  # Store computed stability score
        
        float(ulnar_series[len(ulnar_series) // 2]),  # Mid-frame ulnar deviation
        float(ulnar_series[-20]),  # Ulnar deviation at release

        float(ulnar_series[1]),
        float(ulnar_series[2]),
        float(ulnar_series[3]),
        float(ulnar_series[4]),
        float(ulnar_series[5]),
        float(ulnar_series[6]),
        float(ulnar_series[7]),
        float(ulnar_series[8]),
        float(ulnar_series[9]),
        float(ulnar_series[10]),

        float(pronation_series[len(pronation_series) // 2]),  # Mid-frame pronation
        float(pronation_series[-20]),  # Pronation at release

        float(pronation_series[1]),
        float(pronation_series[2]),
        float(pronation_series[3]),
        float(pronation_series[4]),
        float(pronation_series[5]),
        float(pronation_series[6]),
        float(pronation_series[7]),
        float(pronation_series[8]),
        float(pronation_series[9]),
        float(pronation_series[10])

    )

    cursor.execute(insert_sql, data_tuple)

conn.commit()
conn.close()
print("All data inserted into pitch_data!")


# ------ OPTIONAL: Retrieve last inserted participant/date for your Dash defaults ------
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

# ----------------- HELPER FUNCTIONS -----------------
def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    options = [{"label": p, "value": p} for p in sorted(participants)]
    return options

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_type FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": pt, "value": pt} for pt in sorted(df["pitch_type"].unique())]
    # Insert "All" at the top
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT filename FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": fn, "value": fn} for fn in sorted(df["filename"].unique())]
    # Insert "All"
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """Shows a table that includes both Ulnar Deviation and Pronation comparisons."""
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    pitch_df = pd.read_sql_query(query, conn)

    # If you have a reference_data table, fetch it here:
    ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    conn.close()

    # Group for selected data
    pitch_summary = pitch_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    # Group for reference data
    ref_summary = ref_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    merged = pitch_summary.merge(ref_summary, on="pitch_type", suffixes=("_selected", "_reference"))

    # Simple "acceleration" estimate from frame2..frame10
    # For Ulnar Dev
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    # For Pronation
    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    rows = []
    for _, row in merged.iterrows():
        pitch = row["pitch_type"]
        # Ulnar Deviation row
        rows.append({
            "pitch_type": pitch,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 1)
        })
        # Ulnar Deviation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 1)
        })
        # Pronation row (store actual pronation at release in rel_u_dev column)
        rows.append({
            "pitch_type": f"{pitch} Pronation/Supination",
            # Instead of "Pronation", place the numeric pronation at release:
            "rel_u_dev": round(row["rel_pronation_selected"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 1)
        })
        # Pronation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Pronation/Supination Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 1)
        })

    comp_df = pd.DataFrame(rows)
    return comp_df


# ----------------- DASH APP SETUP -----------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# We define global defaults from the last inserted row:
default_participant = LAST_PARTICIPANT
default_date = LAST_DATE

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=get_dropdown_options(),
                value=default_participant,  # Use last processed participant
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                # We leave options empty; the callback will populate them
                options=[],
                value=default_date,  # Use last processed date
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative", 
                        "width": "440px",
                        "height": "auto",
                        "padding": "10px",  
                        "margin-left": "140px",
                        "backgroundColor": "black"  
                    }
                )
            ], style={"text-align": "right"})
        ], width=2)
    ], className="mt-3", align="center"),       

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[{"name": i, "id": i} for i in [
                            "pitch_type", "rel_u_dev",
                            "frame2_u_dev", "frame4_u_dev",
                            "frame6_u_dev", "frame8_u_dev",
                            "frame10_u_dev", "accel_u_dev"
                        ]],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=9
        ),
            # Average Score Display (1/3 width)
    dbc.Col(
        dbc.Card([
            dbc.CardHeader("Average Stability Score"),
            dbc.CardBody([
                html.H3(id="average-score", style={
                    "fontSize": "82px", 
                    "textAlign": "center", 
                    "color": "lime"
                }),
                html.P("Higher = Better Wrist Stability", style={
                    "textAlign": "center", 
                    "color": "white"
                })
            ])
        ]),
        width=3  # 1/3 of the frame
    )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody([
                    dcc.Graph(id="ulnar-dev-graph"),
                    html.P([
                        "Ulnar deviation measures how much the wrist flexes toward the ulnar side of the forearm or 'flicks' as we release the ball.",
                        html.Br(),
                        html.Strong("Moving in the negative (-) direction represents ulnar deviation "),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "24px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody([
                    dcc.Graph(id="acceleration-graph"),
                    html.P([
                        "Acceleration in the transverse and frontal planes shows how "
                            "rapidly the wrist angles are changing around release, "
                            "which can correlate with injury risk. ",
                        html.Br(),
                        html.Strong("Ideally, it’s kept minimal "),
                        "to reduce stress during throwing."
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="pronation-graph")),
                    html.P([
                        "Pronation and Supination measure how much you 'twist' the wrist. Supination through ball release is associated with the same 'flick' motion we are trying to avoid",
                        html.Br(),
                        html.Strong("Negative (-) values correspond with supination"),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="flexion-graph")),
                    html.P([
                        "Flexion at the wrist can be associated with the same 'flick' that occurs with excessive ulnar deviation"
                        "",
                        html.Br(),
                        html.Strong("Negative (-) values correspond with flexion"),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ----------------- DASH CALLBACKS -----------------
@app.callback(
    Output("average-score", "children"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value")
    ]
)
def update_average_score(selected_participant, selected_date, selected_pitch_type):
    """ Fetches and calculates the average stability score from the database. """
    conn = sqlite3.connect(db_path)
    query = f"""
        SELECT AVG(pitch_stability_score) 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}' 
        AND pitch_date = '{selected_date}'
        AND pitch_type = '{selected_pitch_type}'
    """
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"

    df = pd.read_sql_query(query, conn)
    conn.close()

    avg_score = df.iloc[0, 0]  # Extracts the single value
    if avg_score is None:
        return "N/A"

    return f"{avg_score:.2f}"

def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    options = get_date_options(selected_participant)
    # If the default_date is in the new options, keep it, else use the first
    possible_values = [o["value"] for o in options]
    if default_date in possible_values:
        return options, default_date
    return options, (options[0]["value"] if options else None)

@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None
    options = get_pitch_type_options(selected_participant, selected_date)
    pitch_types = [o["value"] for o in options]

    # Default to "Curve" if available, else "All" or the first
    if "Curve" in pitch_types:
        default_value = "Curve"
    else:
        default_value = "All" if "All" in pitch_types else (pitch_types[0] if pitch_types else None)

    return options, default_value

@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    options = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    value = options[0]["value"] if options else None
    return options, value

# ... your existing imports and code

@app.callback(
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    # 1) Table & 2) main time-series
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)
    table_data = comp_df.to_dict("records")

    # 3) reference data for "Curve"
    ref_data = get_curve_reference_on_the_fly()

    # Compute average release index
    release_frames = []
    for pt, data_dict in ts_data.items():
        for series in data_dict["ulnar_dev_series"]:
            # The last 20 frames in each series are post-release
            release_idx = len(series) - 20
            release_frames.append(release_idx)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # --------------- Ulnar Deviation ---------------
    ulnar_fig = go.Figure()
    plotted_pitch_types = set()  # Track pitch types that have been added to the legend

    # Plot reference if relevant
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # Plot actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Show legend only once per pitch type
    
        for series in data_dict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,  # Only show legend entry for the first trace
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)

    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline],
        autosize=True  # Let it expand
    )

    # --------------- Acceleration ---------------
    accel_fig = go.Figure()

    def compute_acceleration(angle_series):
        return np.diff(angle_series, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    # We'll do 30 frames before release, 40 frames after
    FRAMES_BEFORE = 30
    FRAMES_AFTER  = 40
    
    # The release point is at the 30th index in the new zoomed-in window
    release_line = dict(
        type="line",
        x0=30, x1=30,  # Middle of the new range
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    plotted_pitch_types = set()
    
    for pt, data_dict in ts_data.items():
        color_ulnar = "#bb6a74"
        color_pron  = "#2c99d4"
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["ulnar_dev_series"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]
    
            acc_u = compute_acceleration(sub_series)
            rms_u = compute_rms(acc_u)
            x_vals = list(range(len(rms_u)))
            
    
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color=color_ulnar, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        for series in data_dict["pronation"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]
    
            acc_p = compute_acceleration(sub_series)
            rms_p = compute_rms(acc_p)
            x_vals = list(range(len(rms_p)))
    
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color=color_pron, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    accel_fig.update_layout(
        title="Acceleration: ±30 Frames Before to +40 After Release",
        xaxis_title="Index (local to sub-window)",
        yaxis_title="Accel (°/frame²)",
        shapes=[release_line],
        autosize=True
    )
    
    # --------------- Pronation ---------------
    pronation_fig = go.Figure()
    plotted_pitch_types = set()
    
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )
    
    # --------------- Flexion ---------------
    flexion_fig = go.Figure()
    plotted_pitch_types = set()
    
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    print("Dash Flexion Data:", ts_data)  # See what is being plotted

    for pt, data_dict in ts_data.items():
        for series in data_dict["flexion"]:
            print(f"Pitch: {pt} | Min: {np.min(series):.2f} | Max: {np.max(series):.2f}")

    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


if __name__ == '__main__':
    app.run_server(debug=True)


Processing static trial: D:/Youth Pitch Design/Data/Matt Solter_MS/2025-03-05_\Pitch Design Static RH 1.c3d
Static baseline angles: {'ulnar_dev': np.float64(19.430699334632923), 'pronation': np.float64(78.54913064282889), 'flexion': np.float64(152.59566509365473)}
Dash route => All Flexion Angles: [139.12117662 139.36553082 139.61893708 139.88346257 140.16280567
 140.46285343 140.79236054 141.16379668 141.59442013 142.10764936
 142.73481153 143.51730962 144.50915538 145.77963278 147.41545689
 149.52099233 152.21369972 155.61000339 159.79542203 164.77601108
 170.42216812 176.43977045 177.58783266 172.08533924 167.36399339
 163.56611143 160.68586289 158.62567719 157.249278   156.41629779
 155.9997665  155.89251577 156.00772033 156.27683786 156.64668175
 157.07645296 157.53507351 157.99892567 158.44999438 158.87436847
 159.26104412 159.60097301 159.88629878 160.10973144 160.26401814
 160.34147421 160.33354188 160.23034705 160.02022538 159.68918662
 159.22028328 158.59284514 157.78153877 1

In [38]:
# Add to reference table in db

import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import medfilt
import matplotlib.pyplot as plt


# Dash and Plotly imports
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis_v3.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS reference_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    pitch_stability_score REAL,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL

)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")


# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [os.path.join(selected_folder, file)
             for file in os.listdir(selected_folder)
             if file.lower().endswith('.c3d')]

if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- IDENTIFY STATIC TRIAL -----------------
static_files = [f for f in c3d_files if "static" in f.lower()]
if not static_files:
    raise FileNotFoundError("No static trial found in the dataset.")


# ----------------- HELPER FUNCTIONS -----------------

def lowpass_filter(data, cutoff, fs, order=2):
    """
    6 Hz Butterworth lowpass filter, forward-backward filtfilt.
    data shape is (nFrames,) for 1D signals.
    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

# ----------------- GET TIME SERIES (for graphs) -----------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    
    for c3d_file_path in c3d_files:
        normalized_path = os.path.normpath(c3d_file_path)
        path_parts = normalized_path.split(os.sep)
        if len(path_parts) < 3:
            continue
        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = path_parts[-2].rstrip("_")

        if participant_name != selected_participant or date_folder != selected_date:
            continue

        filename_only = path_parts[-1]
        filename_noext = os.path.splitext(filename_only)[0]
        pitch_type = filename_noext.split()[0].capitalize()

        if selected_pitch_type != "All" and pitch_type != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        try:
            c3d_obj = ezc3d.c3d(c3d_file_path)
        except Exception as e:
            print(f"Error reading {c3d_file_path}: {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame = release_frame + 20
        ulnar_series = []
        pronation_series = []
        flexion_series = []

        #SINGLE LOOP - Avoiding Double Processing
        prev_flexion_angle = None  # Track previous angle for smooth transitions
        for frame in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame)
            p_angle = compute_pronation(points, markers, frame)
            f_angle = compute_wrist_flexion(points, markers, frame, prev_angle=prev_flexion_angle)

            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)
            prev_flexion_angle = f_angle  # Update previous angle

        # Convert lists to numpy arrays **AFTER THE LOOP**
        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        # # Apply np.unwrap to ensure smooth transitions
        # flexion_series = np.unwrap(np.radians(flexion_series), discont=np.radians(180))
        # flexion_series = np.degrees(flexion_series)
        # 
        # # Apply median filter to remove single-frame spikes
        # flexion_series = medfilt(flexion_series, kernel_size=5)

        if pitch_type not in ts:
            ts[pitch_type] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}

        ts[pitch_type]["ulnar_dev_series"].append(ulnar_series)
        ts[pitch_type]["pronation"].append(pronation_series)
        ts[pitch_type]["flexion"].append(flexion_series)
        
        print(f"Dash route => {filename_noext}: foot_contact={foot_contact_frame}, release={release_frame}")
        print(f"Dash route => flexion length={len(flexion_series)}, min={np.min(flexion_series)}, max={np.max(flexion_series)}")

    return ts


def get_curve_reference_on_the_fly():
    """
    Only loads 'Curve' .c3d files from a known reference folder and averages them.
    Adjust as needed for your actual reference approach.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        return {}

    all_files = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d') and "curve" in f.lower()
    ]
    if not all_files:
        print("No valid 'Curve' .c3d files found in reference folder.")
        return {}

    all_ulnar_arrays = []
    all_pronation_arrays = []
    all_flexion_arrays = []

    for c3d_path in all_files:
        try:
            c3d_obj = ezc3d.c3d(c3d_path)
        except Exception as e:
            print(f"Skipping {c3d_path}: read error {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame   = release_frame + 20

        ulnar_series = []
        pronation_series = []
        flexion_series = []
        for frame_idx in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame_idx)
            p_angle = compute_pronation(points, markers, frame_idx)
            f_angle = compute_wrist_flexion(points, markers, frame_idx)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        all_ulnar_arrays.append(ulnar_series)
        all_pronation_arrays.append(pronation_series)
        all_flexion_arrays.append(flexion_series)

    if not all_ulnar_arrays:
        return {}

    # Trim to min length across all files
    min_len = min([arr.shape[0] for arr in all_ulnar_arrays + all_pronation_arrays + all_flexion_arrays])
    all_ulnar_arrays = [arr[:min_len] for arr in all_ulnar_arrays]
    all_pronation_arrays = [arr[:min_len] for arr in all_pronation_arrays]
    all_flexion_arrays = [arr[:min_len] for arr in all_flexion_arrays]

    stacked_ulnar = np.vstack(all_ulnar_arrays)
    stacked_pron  = np.vstack(all_pronation_arrays)
    stacked_flex  = np.vstack(all_flexion_arrays)

    mean_ulnar = np.mean(stacked_ulnar, axis=0)
    mean_pron  = np.mean(stacked_pron,  axis=0)
    mean_flex  = np.mean(stacked_flex,  axis=0)

    # Return only "Curve" references
    ref_data = {
        "Curve": {
            "ulnar_dev_series": [mean_ulnar],
            "pronation": [mean_pron],
            "flexion": [mean_flex]
        }
    }
    return ref_data

# For a table-based reference (if you have a 'reference_data' table), you'd have a function like:
# def get_reference_time_series():
#     ... your table approach ...
#     return reference_ts

def resolve_marker_indices(marker_labels):
    """
    Map marker names to indices, removing 'Right_'/'Left_' prefix.
    Returns a dict of required markers if all are present.
    """
    # This is the set we need
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    # Strip 'Right_'/'Left_' from each label to unify
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}  # required marker not found
    return marker_indices

def filter_marker_data(c3d_obj):
    points = c3d_obj['data']['points']
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    nMarkers = points.shape[1]
    nFrames = points.shape[2]

    # If too few frames for filtfilt’s default pad, skip filtering
    if nFrames < 10:
        print("Not enough frames to lowpass filter; skipping filtering for this file.")
        return

    for m in range(nMarkers):
        for coord in range(3):
            raw_signal = points[coord, m, :]  # shape (nFrames,)
            filtered = lowpass_filter(raw_signal, cutoff=6.0, fs=frame_rate, order=2)
            points[coord, m, :] = filtered


def create_virtual_hand_offset(c3d_obj):
    """
    From the static trial, compute an offset vector to define a stable hand marker.
    For example, compute the vector from the wrist center to the 'Hand' marker.
    We'll replicate that offset in dynamic trials to help define the hand segment.
    """
    points = c3d_obj["data"]["points"]  # shape: (4, nMarkers, nFrames)
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Required markers not found in static trial to create virtual hand offset.")

    nFrames = points.shape[2]

    # We'll average across all static frames
    wrist_centers = []
    hand_vectors  = []

    for f in range(nFrames):
        R = points[:3, markers["Wrist_Radius"], f]
        U = points[:3, markers["Wrist_Ulna"], f]
        H = points[:3, markers["Hand"], f]
        wrist_center = (R + U) / 2.0
        # Vector from wrist_center to the 'Hand' marker
        offset_vec = H - wrist_center

        wrist_centers.append(wrist_center)
        hand_vectors.append(offset_vec)

    # Mean offset vector from wrist center to hand
    mean_hand_offset = np.mean(hand_vectors, axis=0)
    return mean_hand_offset

def apply_virtual_hand_marker(points, marker_indices, hand_offset):
    """
    Override the 'Hand' marker in dynamic data with a virtual location
    based on the static offset. For each frame:
      Hand_virtual = wrist_center + hand_offset
    This ensures the hand marker is in a consistent location relative to the wrist center.
    """
    nFrames = points.shape[2]
    for f in range(nFrames):
        R = points[:3, marker_indices["Wrist_Radius"], f]
        U = points[:3, marker_indices["Wrist_Ulna"], f]
        wrist_center = (R + U) / 2.0

        # new hand location
        new_hand = wrist_center + hand_offset
        # override the original 'Hand' marker
        points[:3, marker_indices["Hand"], f] = new_hand

def compute_ulnar_deviation(points, marker_indices, frame):
    """
    Computes radial/ulnar deviation angle relative to the forearm axis.
    Higher angle => more radial dev if sign is +, or more ulnar dev if sign is -,
    depending on final logic. We'll just keep your existing formula but note
    we subtract from 180 at the end.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = E_center - W_center
    radial_vec = R - U
    hand_vec = H - W_center

    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit) * forearm_unit
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0

    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val = np.dot(radial_proj_n, hand_proj_n)

    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    # In your existing code, you do: return 180 - abs(angle_deg)
    return 180.0 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit) * forearm_unit

    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0

    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)

    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))

    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180.0 - abs(raw_angle)

def compute_wrist_flexion(points, marker_indices, frame, prev_angle=None):
    """
    Computes wrist flexion/extension relative to the global vertical axis.
    Includes fixes for discontinuities using np.unwrap and median filtering.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit) * forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)

    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)

    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))

    raw_angle = np.degrees(angle_rad) * sign

    # 🔹 Ensure smooth transitions using previous angle
    if prev_angle is not None:
        angle_diff = raw_angle - prev_angle
        if abs(angle_diff) > 90:  # If the jump is more than 90°, fix it
            raw_angle -= np.sign(angle_diff) * 180  # Flip to maintain continuity

    return 180.0 - abs(raw_angle)

def plot_wrist_flexion(flexion_series, participant_name, pitch_type, filename_noext, foot_contact_frame, release_frame):
    """
    Plots the wrist flexion time series for a given participant.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(flexion_series, label="Wrist Flexion", color="blue", linewidth=2)
    plt.axvline(x=len(flexion_series) - 20, color="red", linestyle="--", label="Release Point")
    
    plt.xlabel("Frame")
    plt.ylabel("Flexion Angle (°)")
    plt.title(f"Wrist Flexion Over Time ({participant_name} - {pitch_type} - {filename_noext})")
    plt.legend()
    plt.grid()

    # Force display
    plt.show(block=True)  # Ensures it stays open
    plt.savefig(f"{participant_name}_{pitch_type}_{filename_noext}_flexion.png")
    print(f"Flexion plot saved as {participant_name}_{pitch_type}_{filename_noext}_flexion.png")
    print(f"Matplotlib route => {filename_noext}: foot_contact={foot_contact_frame}, release={release_frame}, len={len(flexion_series)}")
    print(f"Matplotlib route => flexion min={flexion_series.min()}, max={flexion_series.max()}")


    
def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]

    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")

    idx_foot = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec = event_times[idx_release]

    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)

    foot_local = foot_contact_global - earliest
    release_local = release_global - earliest

    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")
    return foot_local, release_local

def compute_pitch_stability_score(ulnar_dev_series, pronation_series, flexion_series, accel_ulnar_series, accel_pron_series):
    """
    Computes a wrist stability score (0-100), where higher = better.

    The joint angle differences are computed as the difference between the baseline
    (frame 0, at foot contact) and the average over a 10-frame window centered on ball release.
    
    Inputs:
    - ulnar_dev_series: Time-series of ulnar deviation (frames)
    - pronation_series: Time-series of pronation angles (frames)
    - flexion_series: Time-series of flexion angles (frames)
    - accel_ulnar_series: Time-series of ulnar deviation acceleration
    - accel_pron_series: Time-series of pronation acceleration

    Returns:
    - Stability score (0-100)
    """
    # 1. Frame Indexing (define key moments)
    # Baseline is at frame 0 (foot contact)
    baseline_idx = 0
    # Define the release frame (ball release is assumed to be at index: total_frames - 20)
    release_idx = len(ulnar_dev_series) - 20

    # Define a window for release measurements: 5 frames before and 5 frames after release_idx
    window_start = max(release_idx - 5, 0)
    window_end   = min(release_idx + 5, len(ulnar_dev_series))
    
    # Compute the mean joint angles in that 10-frame window
    mean_ulnar_release = np.mean(ulnar_dev_series[window_start:window_end])
    mean_pronation_release = np.mean(pronation_series[window_start:window_end])
    mean_flexion_release = np.mean(flexion_series[window_start:window_end])
    
    # Use the baseline at frame 0 (or you could also average a few frames at the start if desired)
    baseline_ulnar = ulnar_dev_series[baseline_idx]
    baseline_pronation = pronation_series[baseline_idx]
    baseline_flexion = flexion_series[baseline_idx]

    # 2. Compute absolute movement (penalizes excessive motion)
    max_ulnar_range = 30   # Expected max deviation (in degrees)
    max_pronation_range = 45 # Supination shouldn't be more than 45°
    max_flexion_range = 40   # Wrist flick shouldn't be excessive

    ulnar_stability = 100 - (abs(mean_ulnar_release - baseline_ulnar) / max_ulnar_range) * 100
    pronation_stability = 100 - (abs(mean_pronation_release - baseline_pronation) / max_pronation_range) * 100
    flexion_stability = 100 - (abs(mean_flexion_release - baseline_flexion) / max_flexion_range) * 100

    # 3. Compute acceleration penalty (using a window around release as before)
    accel_start = max(0, release_idx - 5)
    accel_end   = min(len(ulnar_dev_series), release_idx + 5)
    rms_accel_ulnar = np.sqrt(np.mean(np.square(accel_ulnar_series[accel_start:accel_end])))
    rms_accel_pron  = np.sqrt(np.mean(np.square(accel_pron_series[accel_start:accel_end])))

    max_expected_accel = 10  # Expected max acceleration in deg/frame²
    accel_stability = 100 - ((rms_accel_ulnar + rms_accel_pron) / max_expected_accel) * 100

    # 4. Ensure values are within [0, 100] range
    ulnar_stability = np.clip(ulnar_stability, 0, 100)
    pronation_stability = np.clip(pronation_stability, 0, 100)
    flexion_stability = np.clip(flexion_stability, 0, 100)
    accel_stability = np.clip(accel_stability, 0, 100)

    # 5. Weighted score (adjust weights if needed)
    # (Note: the weights in your current code sum to more than 1, but they can be adjusted as desired.)
    weights = [0.45, 0.20, 0.05, 0.30]
    final_score = (
        weights[0] * ulnar_stability +
        weights[1] * pronation_stability +
        weights[2] * flexion_stability +
        weights[3] * accel_stability
    )

    return round(final_score, 2)

    return round(final_score, 2)


# ---------------------- PROCESS STATIC TRIAL ----------------------
print("Processing static trial:", static_files[0])
static_c3d = ezc3d.c3d(static_files[0])

# 1) Filter static data at 6 Hz
filter_marker_data(static_c3d)

# 2) Create a consistent 'hand offset' from the static trial
static_hand_offset = create_virtual_hand_offset(static_c3d)

# 3) Compute baseline angles from the filtered, original static trial
def compute_static_baseline(c3d_obj):
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Static trial missing required markers.")
    nFrames = points.shape[2]

    ulnar_vals = []
    pron_vals  = []
    flex_vals  = []

    for f in range(nFrames):
        u = compute_ulnar_deviation(points, markers, f)
        p = compute_pronation(points, markers, f)
        x = compute_wrist_flexion(points, markers, f)
        ulnar_vals.append(u)
        pron_vals.append(p)
        flex_vals.append(x)

    return {
        "ulnar_dev": np.mean(ulnar_vals),
        "pronation": np.mean(pron_vals),
        "flexion":   np.mean(flex_vals)
    }

static_baseline_vals = compute_static_baseline(static_c3d)
print("Static baseline angles:", static_baseline_vals)


# ----------------- APPLY TO PITCH DATA -----------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file in c3d_files:
    if "static" in c3d_file.lower():
        continue  # Skip static trial

    c3d_obj = ezc3d.c3d(c3d_file)

    # 1) Filter dynamic data at 6 Hz
    filter_marker_data(c3d_obj)

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file}: required markers not found.")
        continue

    # 2) Apply our "virtual hand marker" override so it remains consistent
    apply_virtual_hand_marker(points, markers, static_hand_offset)

    # 3) Identify events
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]
    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file}: {e}")
        continue

    # Ensure enough frames post-release
    if release_frame + 20 >= total_frames:
        print(f"Skipping {c3d_file}: Not enough frames for release+20.")
        continue

    # 4) Basic info for DB
    normalized_path = os.path.normpath(c3d_file)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2].rstrip("_")
    pitch_date = date_folder
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]
    pitch_type = filename_noext.split()[0].capitalize()

    # 5) Extract Time-Series Data
    start_frame = foot_contact_frame
    end_frame = release_frame + 20
    ulnar_series = []
    pronation_series = []
    flexion_series = []

    for frame in range(start_frame, end_frame):
        ulnar_series.append(compute_ulnar_deviation(points, markers, frame))
        pronation_series.append(compute_pronation(points, markers, frame))
        flexion_series.append(compute_wrist_flexion(points, markers, frame))

    # Convert lists to NumPy arrays for processing
    ulnar_series = np.array(ulnar_series)
    pronation_series = np.array(pronation_series)
    flexion_series = np.array(flexion_series)
    print(f"Dash route => All Flexion Angles: {flexion_series}")
    print(f"Matplotlib route => All Flexion Angles: {flexion_series}")

    # 6) Compute Acceleration
    def compute_acceleration(angle_series):
        return np.gradient(np.gradient(angle_series))  # Second derivative

    accel_ulnar_series = compute_acceleration(ulnar_series)
    accel_pron_series = compute_acceleration(pronation_series)

    # 7) Compute Stability Score
    pitch_stability_score = compute_pitch_stability_score(
        ulnar_series, 
        pronation_series, 
        flexion_series, 
        accel_ulnar_series,  
        accel_pron_series
    )
    
    print(f"Processing {filename_noext} for {participant_name} - {pitch_type}")
    # plot_wrist_flexion(flexion_series, participant_name, pitch_type, filename_noext)

    print(f"Inserted {filename_noext} ({pitch_type}) - Stability Score: {pitch_stability_score}")

    # 8) Insert into Database
    insert_sql = """
        INSERT INTO reference_data (
            participant_name, pitch_date, pitch_type, filename,
            pitch_stability_score,
            mid_u_dev, rel_u_dev,
            frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
            frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
            mid_pronation, rel_pronation,
            frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
            frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
        )
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(pitch_stability_score),  # Store computed stability score
        
        float(ulnar_series[len(ulnar_series) // 2]),  # Mid-frame ulnar deviation
        float(ulnar_series[-20]),  # Ulnar deviation at release

        float(ulnar_series[1]),
        float(ulnar_series[2]),
        float(ulnar_series[3]),
        float(ulnar_series[4]),
        float(ulnar_series[5]),
        float(ulnar_series[6]),
        float(ulnar_series[7]),
        float(ulnar_series[8]),
        float(ulnar_series[9]),
        float(ulnar_series[10]),

        float(pronation_series[len(pronation_series) // 2]),  # Mid-frame pronation
        float(pronation_series[-20]),  # Pronation at release

        float(pronation_series[1]),
        float(pronation_series[2]),
        float(pronation_series[3]),
        float(pronation_series[4]),
        float(pronation_series[5]),
        float(pronation_series[6]),
        float(pronation_series[7]),
        float(pronation_series[8]),
        float(pronation_series[9]),
        float(pronation_series[10])

    )

    cursor.execute(insert_sql, data_tuple)

conn.commit()
conn.close()
print("All data inserted into pitch_data!")


# ------ OPTIONAL: Retrieve last inserted participant/date for your Dash defaults ------
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM reference_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

Processing static trial: D:/Youth Pitch Design/Data/Reference Data_RD/2025-02-21_\Pitch Design Static RH 2.c3d
Static baseline angles: {'ulnar_dev': np.float64(5.281115642286362), 'pronation': np.float64(92.95433319543125), 'flexion': np.float64(176.8432856558699)}
Dash route => All Flexion Angles: [179.52351204 179.50959151 179.49528238 179.48062007 179.46563761
 179.45036537 179.43483097 179.41905933 179.40307275 179.38689091
 179.37053059 179.35400548 179.33732596 179.3204989  179.30352751
 179.28641123 179.26914573 179.25172294 179.23413112 179.21635504
 179.19837607 179.18017247 179.16171955 179.14298995 179.12395387
 179.10457929 179.08483209 179.06467614 179.04407332 179.02298347
 179.00136428 178.97917107 178.95635657 178.93287059 178.90865964
 178.8836665  178.85782969 178.83108287 178.8033542  178.77456561
 178.74463192 178.71345997 178.68094767 178.64698301 178.61144309
 178.57419304 178.53508494 178.49395666 178.45063054 178.40491202
 178.35658822 178.30542636 178.25117214 

In [15]:
# Debug for discontinuities

import ezc3d
import numpy as np
from scipy.signal import medfilt

# Load the C3D file
file_path = "D:\\Youth Pitch Design\\Data\Keenan Aldridge_KA\\2025-02-17_\\Curve RH 6.c3d"  # Adjust path as needed
c3d_data = ezc3d.c3d(file_path)

# Extract marker data
points = c3d_data["data"]["points"]  # Shape: (4, nMarkers, nFrames)
marker_labels = c3d_data["parameters"]["POINT"]["LABELS"]["value"]
frame_rate = c3d_data["parameters"]["POINT"]["RATE"]["value"][0]
total_frames = points.shape[2]

# Resolve marker indices (removing "Right_" or "Left_" prefixes)
def resolve_marker_indices(marker_labels):
    req_markers = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]
    marker_indices = {mk: clean_labels.index(mk) for mk in req_markers if mk in clean_labels}
    return marker_indices if len(marker_indices) == len(req_markers) else {}

marker_indices = resolve_marker_indices(marker_labels)

# Compute wrist flexion angle for each frame
def compute_wrist_flexion(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit) * forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)

    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)

    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))

    return np.degrees(angle_rad) * sign

# Compute wrist flexion angles
flexion_angles = np.array([compute_wrist_flexion(points, marker_indices, f) for f in range(total_frames)])

flexion_angles = np.unwrap(np.radians(flexion_angles), discont=np.radians(180))
flexion_angles = np.degrees(flexion_angles)
flexion_angles = medfilt(flexion_angles, kernel_size=5)

for frame in [41, 42, 43, 87, 88, 89]:
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    hand_vec = H - W_center

    print(f"Frame {frame}:")
    print(f"  Forearm Vec: {forearm_vec}")
    print(f"  Hand Vec: {hand_vec}")
    print(f"  Flexion Angle: {flexion_angles[frame]}")
    print("-" * 30)


def compute_wrist_flexion_debug(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit) * forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)

    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)

    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))

    print(f"Frame {frame}: dot_val={dot_val}, cross_val={cross_val}, sign={sign}")

    return np.degrees(angle_rad) * sign

# Identify discontinuities (where the jump exceeds 90 degrees)
jumps = np.abs(np.diff(flexion_angles))
discontinuity_frames = np.where(jumps > 90)[0]

# Output the frames where the jump occurs
print("Discontinuity frames:", discontinuity_frames)


Frame 41:
  Forearm Vec: [-47.22039795 -54.87731934 216.29882812]
  Hand Vec: [-37.73713684 -36.58886719  21.75775146]
  Flexion Angle: 173.67074587807855
------------------------------
Frame 42:
  Forearm Vec: [-50.72917175 -53.52624512 215.96160889]
  Hand Vec: [-37.19075012 -37.72622681  21.58306885]
  Flexion Angle: 178.61632258519018
------------------------------
Frame 43:
  Forearm Vec: [-54.3719635  -52.38867188 215.32763672]
  Hand Vec: [-36.50727844 -38.72662354  21.0032959 ]
  Flexion Angle: 183.3863420978099
------------------------------
Frame 87:
  Forearm Vec: [126.21737671 -73.65985107 193.34466553]
  Hand Vec: [ 59.43716431 -20.75994873  22.85186768]
  Flexion Angle: 198.70806230615224
------------------------------
Frame 88:
  Forearm Vec: [134.74237061 -42.40441895 197.99420166]
  Hand Vec: [ 62.38543701 -14.44604492  27.11999512]
  Flexion Angle: 187.77828060264494
------------------------------
Frame 89:
  Forearm Vec: [140.90957642  -8.57757568 199.29858398]
  Han

<>:6: SyntaxWarning:

invalid escape sequence '\K'

<>:6: SyntaxWarning:

invalid escape sequence '\K'

C:\Users\q\AppData\Local\Temp\ipykernel_3724\2298614488.py:6: SyntaxWarning:

invalid escape sequence '\K'



In [3]:
import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Dash and Plotly imports
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL
)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [os.path.join(selected_folder, file)
             for file in os.listdir(selected_folder)
             if file.lower().endswith('.c3d')]
if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- COMPUTE FUNCTIONS -----------------
def lowpass_filter(data, cutoff, fs, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=-1)

def compute_ulnar_deviation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = E_center - W_center
    radial_vec  = R - U
    hand_vec    = H - W_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit)*forearm_unit
    hand_proj   = hand_vec   - np.dot(hand_vec,   forearm_unit)*forearm_unit
    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n   = hand_proj   / np.linalg.norm(hand_proj)
    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val   = np.dot(radial_proj_n, hand_proj_n)
    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    return 180 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit)*forearm_unit
    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit)*forearm_unit
    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0
    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)
    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))
    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180 - abs(raw_angle)

def compute_wrist_flexion(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit)*forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)
    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit)*forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)
    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)
    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180 - abs(raw_angle)

def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]
    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")
    idx_foot = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec = event_times[idx_release]
    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)
    foot_local = foot_contact_global - earliest
    release_local = release_global - earliest
    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")
    return foot_local, release_local

def resolve_marker_indices(marker_labels):
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}
    return marker_indices

# ------------------------------------------------------------
#  MAIN PROCESSING LOOP: Insert results into pitch_data
# ------------------------------------------------------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file_path in c3d_files:
    normalized_path = os.path.normpath(c3d_file_path)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2]
    pitch_date = date_folder.rstrip("_")
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]

    # pitch_type is the first word in the filename (split by space)
    pitch_type = filename_noext.split()[0].capitalize()

    try:
        c3d_obj = ezc3d.c3d(c3d_file_path)
    except Exception as e:
        print(f"Error reading {c3d_file_path}: {e}")
        continue

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]

    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file_path}: {e}")
        continue

    # ensure we have enough frames
    if release_frame + 20 >= total_frames:
        print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
        continue

    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file_path}: Required markers not found.")
        continue

    num_frames = release_frame - foot_contact_frame
    if num_frames < 1:
        print(f"Skipping {c3d_file_path}: release <= foot_contact.")
        continue

    mid_frame = foot_contact_frame + (num_frames // 2)

    # 11 equally spaced indexes from foot_contact to release
    equally_spaced = np.linspace(foot_contact_frame, release_frame, num=11, dtype=int)

    # Ulnar Deviation
    mid_u_dev = compute_ulnar_deviation(points, markers, mid_frame)
    rel_u_dev = compute_ulnar_deviation(points, markers, release_frame)
    frame_u_dev = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_ulnar_deviation(points, markers, f_idx)
        frame_u_dev.append(angle)

    # Pronation
    mid_pronation = compute_pronation(points, markers, mid_frame)
    rel_pronation = compute_pronation(points, markers, release_frame)
    frame_pronation = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_pronation(points, markers, f_idx)
        frame_pronation.append(angle)

    # Insert row into pitch_data
    insert_sql = """
    INSERT INTO pitch_data (
        participant_name, pitch_date, pitch_type, filename,
        mid_u_dev, rel_u_dev,
        frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
        frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
        mid_pronation, rel_pronation,
        frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
        frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
    )
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(mid_u_dev),
        float(rel_u_dev),
        float(frame_u_dev[0]),
        float(frame_u_dev[1]),
        float(frame_u_dev[2]),
        float(frame_u_dev[3]),
        float(frame_u_dev[4]),
        float(frame_u_dev[5]),
        float(frame_u_dev[6]),
        float(frame_u_dev[7]),
        float(frame_u_dev[8]),
        float(frame_u_dev[9]),

        float(mid_pronation),
        float(rel_pronation),
        float(frame_pronation[0]),
        float(frame_pronation[1]),
        float(frame_pronation[2]),
        float(frame_pronation[3]),
        float(frame_pronation[4]),
        float(frame_pronation[5]),
        float(frame_pronation[6]),
        float(frame_pronation[7]),
        float(frame_pronation[8]),
        float(frame_pronation[9])
    )

    cursor.execute(insert_sql, data_tuple)
    print(f"Inserted data for {filename_noext} ({pitch_type})")

conn.commit()
conn.close()
print("All data inserted into pitch_data!")

# Retrieve the *last inserted* participant/date to set as defaults in Dash.
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

# ----------------- GET TIME SERIES (for graphs) -----------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    for c3d_file_path in c3d_files:
        normalized_path = os.path.normpath(c3d_file_path)
        path_parts = normalized_path.split(os.sep)
        if len(path_parts) < 3:
            continue
        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = path_parts[-2].rstrip("_")

        if participant_name != selected_participant or date_folder != selected_date:
            continue

        filename_only = path_parts[-1]
        filename_noext = os.path.splitext(filename_only)[0]
        pitch_type = filename_noext.split()[0].capitalize()

        if selected_pitch_type != "All" and pitch_type != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        try:
            c3d_obj = ezc3d.c3d(c3d_file_path)
        except Exception as e:
            print(f"Error reading {c3d_file_path}: {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame = release_frame + 20
        ulnar_series = []
        pronation_series = []
        flexion_series = []

        # Collect angles from foot_contact..release+20
        for frame in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame)
            p_angle = compute_pronation(points, markers, frame)
            f_angle = compute_wrist_flexion(points, markers, frame)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        # Convert to numpy
        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        # # Subtract baseline so it starts near 0 (optional). 
        # # This can cause a mismatch vs. reference data if the reference isn't also zeroed.
        # pronation_series = pronation_series - pronation_series[0]
        # flexion_series = flexion_series - flexion_series[0]

        if pitch_type not in ts:
            ts[pitch_type] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}

        ts[pitch_type]["ulnar_dev_series"].append(ulnar_series)
        ts[pitch_type]["pronation"].append(pronation_series)
        ts[pitch_type]["flexion"].append(flexion_series)

    return ts

def get_curve_reference_on_the_fly():
    """
    Only loads 'Curve' .c3d files from a known reference folder and averages them.
    Adjust as needed for your actual reference approach.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        return {}

    all_files = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d') and "curve" in f.lower()
    ]
    if not all_files:
        print("No valid 'Curve' .c3d files found in reference folder.")
        return {}

    all_ulnar_arrays = []
    all_pronation_arrays = []
    all_flexion_arrays = []

    for c3d_path in all_files:
        try:
            c3d_obj = ezc3d.c3d(c3d_path)
        except Exception as e:
            print(f"Skipping {c3d_path}: read error {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame   = release_frame + 20

        ulnar_series = []
        pronation_series = []
        flexion_series = []
        for frame_idx in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame_idx)
            p_angle = compute_pronation(points, markers, frame_idx)
            f_angle = compute_wrist_flexion(points, markers, frame_idx)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        all_ulnar_arrays.append(ulnar_series)
        all_pronation_arrays.append(pronation_series)
        all_flexion_arrays.append(flexion_series)

    if not all_ulnar_arrays:
        return {}

    # Trim to min length across all files
    min_len = min([arr.shape[0] for arr in all_ulnar_arrays + all_pronation_arrays + all_flexion_arrays])
    all_ulnar_arrays = [arr[:min_len] for arr in all_ulnar_arrays]
    all_pronation_arrays = [arr[:min_len] for arr in all_pronation_arrays]
    all_flexion_arrays = [arr[:min_len] for arr in all_flexion_arrays]

    stacked_ulnar = np.vstack(all_ulnar_arrays)
    stacked_pron  = np.vstack(all_pronation_arrays)
    stacked_flex  = np.vstack(all_flexion_arrays)

    mean_ulnar = np.mean(stacked_ulnar, axis=0)
    mean_pron  = np.mean(stacked_pron,  axis=0)
    mean_flex  = np.mean(stacked_flex,  axis=0)

    # Return only "Curve" references
    ref_data = {
        "Curve": {
            "ulnar_dev_series": [mean_ulnar],
            "pronation": [mean_pron],
            "flexion": [mean_flex]
        }
    }
    return ref_data

# For a table-based reference (if you have a 'reference_data' table), you'd have a function like:
# def get_reference_time_series():
#     ... your table approach ...
#     return reference_ts

# ----------------- HELPER FUNCTIONS -----------------
def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    options = [{"label": p, "value": p} for p in sorted(participants)]
    return options

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_type FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": pt, "value": pt} for pt in sorted(df["pitch_type"].unique())]
    # Insert "All" at the top
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT filename FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": fn, "value": fn} for fn in sorted(df["filename"].unique())]
    # Insert "All"
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """Shows a table that includes both Ulnar Deviation and Pronation comparisons."""
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    pitch_df = pd.read_sql_query(query, conn)

    # If you have a reference_data table, fetch it here:
    ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    conn.close()

    # Group for selected data
    pitch_summary = pitch_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    # Group for reference data
    ref_summary = ref_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    merged = pitch_summary.merge(ref_summary, on="pitch_type", suffixes=("_selected", "_reference"))

    # Simple "acceleration" estimate from frame2..frame10
    # For Ulnar Dev
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    # For Pronation
    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    rows = []
    for _, row in merged.iterrows():
        pitch = row["pitch_type"]
        # Ulnar Deviation row
        rows.append({
            "pitch_type": pitch,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 1)
        })
        # Ulnar Deviation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 1)
        })
        # Pronation row (store actual pronation at release in rel_u_dev column)
        rows.append({
            "pitch_type": f"{pitch} Pronation",
            # Instead of "Pronation", place the numeric pronation at release:
            "rel_u_dev": round(row["rel_pronation_selected"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 1)
        })
        # Pronation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Pronation Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 1)
        })

    comp_df = pd.DataFrame(rows)
    return comp_df


# ----------------- DASH APP SETUP -----------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# We define global defaults from the last inserted row:
default_participant = LAST_PARTICIPANT
default_date = LAST_DATE

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=get_dropdown_options(),
                value=default_participant,  # Use last processed participant
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                # We leave options empty; the callback will populate them
                options=[],
                value=default_date,  # Use last processed date
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative", 
                        "width": "440px",
                        "height": "auto",
                        "padding": "10px",  
                        "margin-left": "140px",
                        "backgroundColor": "black"  
                    }
                )
            ], style={"text-align": "right"})
        ], width=2)
    ], className="mt-3", align="center"),       

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[{"name": i, "id": i} for i in [
                            "pitch_type", "rel_u_dev",
                            "frame2_u_dev", "frame4_u_dev",
                            "frame6_u_dev", "frame8_u_dev",
                            "frame10_u_dev", "accel_u_dev"
                        ]],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=12
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody(dcc.Graph(id="ulnar-dev-graph"))
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody(dcc.Graph(id="acceleration-graph"))
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(dcc.Graph(id="pronation-graph"))
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(dcc.Graph(id="flexion-graph"))
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ----------------- DASH CALLBACKS -----------------
@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    options = get_date_options(selected_participant)
    # If the default_date is in the new options, keep it, else use the first
    possible_values = [o["value"] for o in options]
    if default_date in possible_values:
        return options, default_date
    return options, (options[0]["value"] if options else None)

@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None
    options = get_pitch_type_options(selected_participant, selected_date)
    pitch_types = [o["value"] for o in options]

    # Default to "Curve" if available, else "All" or the first
    if "Curve" in pitch_types:
        default_value = "Curve"
    else:
        default_value = "All" if "All" in pitch_types else (pitch_types[0] if pitch_types else None)

    return options, default_value

@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    options = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    value = options[0]["value"] if options else None
    return options, value

@app.callback(
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    # 1) Table
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    table_data = comp_df.to_dict("records")

    # 2) main time-series
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)

    # 3) reference data (only "Curve" for example)
    ref_data = get_curve_reference_on_the_fly()  # or your reference table approach

    # Vertical line at average release
    release_frames = []
    for pt, data_dict in ts_data.items():
        for series in data_dict["ulnar_dev_series"]:
            # release is at index = len(series) - 20
            release_frames.append(len(series) - 20)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # --------------------------------
    # Ulnar Deviation
    # --------------------------------
    ulnar_fig = go.Figure()
    # (a) reference
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # (b) actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        for series in data_dict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt
            ))
    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    # --------------------------------
    # Acceleration
    # --------------------------------
    accel_fig = go.Figure()

    def compute_acceleration(angle_series):
        # 2nd difference = approximate acceleration
        # Adjust as needed (frame rate, etc.)
        return np.diff(angle_series, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    for pt, data_dict in ts_data.items():
        # Ulnar deviation acceleration
        for series in data_dict["ulnar_dev_series"]:
            accel_u = compute_acceleration(series)
            rms_u = compute_rms(accel_u)
            x_vals = list(range(len(rms_u)))
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color="#bb6a74", width=2),
                name=f"{pt} Ulnar RMS"
            ))

        # Pronation acceleration
        for series in data_dict["pronation"]:
            accel_p = compute_acceleration(series)
            rms_p = compute_rms(accel_p)
            x_vals = list(range(len(rms_p)))
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color="#2c99d4", width=2),
                name=f"{pt} Pronation RMS"
            ))

    accel_fig.update_layout(
        title="Acceleration: Transverse & Frontal",
        xaxis_title="Frame",
        yaxis_title="Accel (°/frame²)",
        shapes=[vline]
    )

    # --------------------------------
    # Pronation
    # --------------------------------
    pronation_fig = go.Figure()
    # (a) reference
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # (b) actual data (FIX: use ts_data, not ref_data again!)
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        for series in data_dict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt
            ))
    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    # --------------------------------
    # Flexion
    # --------------------------------
    flexion_fig = go.Figure()
    # (a) reference
    # No pitch-type filter if your ref_data only has "Curve", 
    # or do the same check as above:
    for pt, data_dict in ref_data.items():
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{pt} Reference"
            ))
    # (b) actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt
            ))
    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


if __name__ == '__main__':
    app.run_server(debug=True)


Inserted data for Slider RH 3 (Slider)
Inserted data for Slider RH 2 (Slider)
Inserted data for Slider RH 1 (Slider)
Inserted data for Changeup RH 2 (Changeup)
Inserted data for Changeup RH 1 (Changeup)
Inserted data for Curve RH 6 (Curve)
Inserted data for Curve RH 5 (Curve)
Inserted data for Curve RH 4 (Curve)
Inserted data for Curve RH 3 (Curve)
Inserted data for Curve RH 1 (Curve)
Inserted data for Fastball RH 3 (Fastball)
Inserted data for Slider RH 5 (Slider)
Inserted data for Fastball RH 2 (Fastball)
Inserted data for Slider RH 4 (Slider)
Skipping D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Fastball RH 1.c3d: Required events (Foot Contact, Release) not found in C3D events.
Inserted data for Curve RH 2 (Curve)
Skipping D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Pitch Design Static RH 1.c3d: No EVENT data in C3D.
All data inserted into pitch_data!


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [46]:
import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import medfilt
import matplotlib.pyplot as plt


# Dash and Plotly imports
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis_v3.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    pitch_stability_score REAL,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL

)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")


# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [os.path.join(selected_folder, file)
             for file in os.listdir(selected_folder)
             if file.lower().endswith('.c3d')]

if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- IDENTIFY STATIC TRIAL -----------------
static_files = [f for f in c3d_files if "static" in f.lower()]
if not static_files:
    raise FileNotFoundError("No static trial found in the dataset.")


# ----------------- HELPER FUNCTIONS -----------------

def lowpass_filter(data, cutoff, fs, order=2):
    """
    6 Hz Butterworth lowpass filter, forward-backward filtfilt.
    data shape is (nFrames,) for 1D signals.
    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

# ----------------- GET TIME SERIES (for graphs) -----------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    
    for c3d_file_path in c3d_files:
        normalized_path = os.path.normpath(c3d_file_path)
        path_parts = normalized_path.split(os.sep)
        if len(path_parts) < 3:
            continue
        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = path_parts[-2].rstrip("_")

        if participant_name != selected_participant or date_folder != selected_date:
            continue

        filename_only = path_parts[-1]
        filename_noext = os.path.splitext(filename_only)[0]
        pitch_type = filename_noext.split()[0].capitalize()

        if selected_pitch_type != "All" and pitch_type != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        try:
            c3d_obj = ezc3d.c3d(c3d_file_path)
        except Exception as e:
            print(f"Error reading {c3d_file_path}: {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame = release_frame + 20
        ulnar_series = []
        pronation_series = []
        flexion_series = []

        #SINGLE LOOP - Avoiding Double Processing
        prev_flexion_angle = None  # Track previous angle for smooth transitions
        for frame in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame)
            p_angle = compute_pronation(points, markers, frame)
            f_angle = compute_wrist_flexion(points, markers, frame, prev_angle=prev_flexion_angle)

            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)
            prev_flexion_angle = f_angle  # Update previous angle

        # Convert lists to numpy arrays **AFTER THE LOOP**
        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        # # Apply np.unwrap to ensure smooth transitions
        # flexion_series = np.unwrap(np.radians(flexion_series), discont=np.radians(180))
        # flexion_series = np.degrees(flexion_series)
        # 
        # # Apply median filter to remove single-frame spikes
        # flexion_series = medfilt(flexion_series, kernel_size=5)

        if pitch_type not in ts:
            ts[pitch_type] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}

        ts[pitch_type]["ulnar_dev_series"].append(ulnar_series)
        ts[pitch_type]["pronation"].append(pronation_series)
        ts[pitch_type]["flexion"].append(flexion_series)
        
        print(f"Dash route => {filename_noext}: foot_contact={foot_contact_frame}, release={release_frame}")
        print(f"Dash route => flexion length={len(flexion_series)}, min={np.min(flexion_series)}, max={np.max(flexion_series)}")

    return ts


def get_curve_reference_on_the_fly():
    """
    Only loads 'Curve' .c3d files from a known reference folder and averages them.
    Adjust as needed for your actual reference approach.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        return {}

    all_files = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d') and "curve" in f.lower()
    ]
    if not all_files:
        print("No valid 'Curve' .c3d files found in reference folder.")
        return {}

    all_ulnar_arrays = []
    all_pronation_arrays = []
    all_flexion_arrays = []

    for c3d_path in all_files:
        try:
            c3d_obj = ezc3d.c3d(c3d_path)
        except Exception as e:
            print(f"Skipping {c3d_path}: read error {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame   = release_frame + 20

        ulnar_series = []
        pronation_series = []
        flexion_series = []
        for frame_idx in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame_idx)
            p_angle = compute_pronation(points, markers, frame_idx)
            f_angle = compute_wrist_flexion(points, markers, frame_idx)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        all_ulnar_arrays.append(ulnar_series)
        all_pronation_arrays.append(pronation_series)
        all_flexion_arrays.append(flexion_series)

    if not all_ulnar_arrays:
        return {}

    # Trim to min length across all files
    min_len = min([arr.shape[0] for arr in all_ulnar_arrays + all_pronation_arrays + all_flexion_arrays])
    all_ulnar_arrays = [arr[:min_len] for arr in all_ulnar_arrays]
    all_pronation_arrays = [arr[:min_len] for arr in all_pronation_arrays]
    all_flexion_arrays = [arr[:min_len] for arr in all_flexion_arrays]

    stacked_ulnar = np.vstack(all_ulnar_arrays)
    stacked_pron  = np.vstack(all_pronation_arrays)
    stacked_flex  = np.vstack(all_flexion_arrays)

    mean_ulnar = np.mean(stacked_ulnar, axis=0)
    mean_pron  = np.mean(stacked_pron,  axis=0)
    mean_flex  = np.mean(stacked_flex,  axis=0)

    # Return only "Curve" references
    ref_data = {
        "Curve": {
            "ulnar_dev_series": [mean_ulnar],
            "pronation": [mean_pron],
            "flexion": [mean_flex]
        }
    }
    return ref_data

# For a table-based reference (if you have a 'reference_data' table), you'd have a function like:
# def get_reference_time_series():
#     ... your table approach ...
#     return reference_ts

def resolve_marker_indices(marker_labels):
    """
    Map marker names to indices, removing 'Right_'/'Left_' prefix.
    Returns a dict of required markers if all are present.
    """
    # This is the set we need
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    # Strip 'Right_'/'Left_' from each label to unify
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]

    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}  # required marker not found
    return marker_indices

def filter_marker_data(c3d_obj):
    points = c3d_obj['data']['points']
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    nMarkers = points.shape[1]
    nFrames = points.shape[2]

    # If too few frames for filtfilt’s default pad, skip filtering
    if nFrames < 10:
        print("Not enough frames to lowpass filter; skipping filtering for this file.")
        return

    for m in range(nMarkers):
        for coord in range(3):
            raw_signal = points[coord, m, :]  # shape (nFrames,)
            filtered = lowpass_filter(raw_signal, cutoff=6.0, fs=frame_rate, order=2)
            points[coord, m, :] = filtered


def create_virtual_hand_offset(c3d_obj):
    """
    From the static trial, compute an offset vector to define a stable hand marker.
    For example, compute the vector from the wrist center to the 'Hand' marker.
    We'll replicate that offset in dynamic trials to help define the hand segment.
    """
    points = c3d_obj["data"]["points"]  # shape: (4, nMarkers, nFrames)
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Required markers not found in static trial to create virtual hand offset.")

    nFrames = points.shape[2]

    # We'll average across all static frames
    wrist_centers = []
    hand_vectors  = []

    for f in range(nFrames):
        R = points[:3, markers["Wrist_Radius"], f]
        U = points[:3, markers["Wrist_Ulna"], f]
        H = points[:3, markers["Hand"], f]
        wrist_center = (R + U) / 2.0
        # Vector from wrist_center to the 'Hand' marker
        offset_vec = H - wrist_center

        wrist_centers.append(wrist_center)
        hand_vectors.append(offset_vec)

    # Mean offset vector from wrist center to hand
    mean_hand_offset = np.mean(hand_vectors, axis=0)
    return mean_hand_offset

def apply_virtual_hand_marker(points, marker_indices, hand_offset):
    """
    Override the 'Hand' marker in dynamic data with a virtual location
    based on the static offset. For each frame:
      Hand_virtual = wrist_center + hand_offset
    This ensures the hand marker is in a consistent location relative to the wrist center.
    """
    nFrames = points.shape[2]
    for f in range(nFrames):
        R = points[:3, marker_indices["Wrist_Radius"], f]
        U = points[:3, marker_indices["Wrist_Ulna"], f]
        wrist_center = (R + U) / 2.0

        # new hand location
        new_hand = wrist_center + hand_offset
        # override the original 'Hand' marker
        points[:3, marker_indices["Hand"], f] = new_hand

def compute_ulnar_deviation(points, marker_indices, frame):
    """
    Computes radial/ulnar deviation angle relative to the forearm axis.
    Higher angle => more radial dev if sign is +, or more ulnar dev if sign is -,
    depending on final logic. We'll just keep your existing formula but note
    we subtract from 180 at the end.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = E_center - W_center
    radial_vec = R - U
    hand_vec = H - W_center

    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit) * forearm_unit
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0

    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val = np.dot(radial_proj_n, hand_proj_n)

    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    # In your existing code, you do: return 180 - abs(angle_deg)
    return 180.0 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit) * forearm_unit

    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0

    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)

    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))

    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180.0 - abs(raw_angle)

def compute_wrist_flexion(points, marker_indices, frame, prev_angle=None):
    """
    Computes wrist flexion/extension relative to the global vertical axis.
    Includes fixes for discontinuities using np.unwrap and median filtering.
    """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit) * forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)

    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)

    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))

    raw_angle = np.degrees(angle_rad) * sign

    # 🔹 Ensure smooth transitions using previous angle
    if prev_angle is not None:
        angle_diff = raw_angle - prev_angle
        if abs(angle_diff) > 90:  # If the jump is more than 90°, fix it
            raw_angle -= np.sign(angle_diff) * 180  # Flip to maintain continuity

    return 180.0 - abs(raw_angle)

def plot_wrist_flexion(flexion_series, participant_name, pitch_type, filename_noext, foot_contact_frame, release_frame):
    """
    Plots the wrist flexion time series for a given participant.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(flexion_series, label="Wrist Flexion", color="blue", linewidth=2)
    plt.axvline(x=len(flexion_series) - 20, color="red", linestyle="--", label="Release Point")
    
    plt.xlabel("Frame")
    plt.ylabel("Flexion Angle (°)")
    plt.title(f"Wrist Flexion Over Time ({participant_name} - {pitch_type} - {filename_noext})")
    plt.legend()
    plt.grid()

    # Force display
    plt.show(block=True)  # Ensures it stays open
    plt.savefig(f"{participant_name}_{pitch_type}_{filename_noext}_flexion.png")
    print(f"Flexion plot saved as {participant_name}_{pitch_type}_{filename_noext}_flexion.png")
    print(f"Matplotlib route => {filename_noext}: foot_contact={foot_contact_frame}, release={release_frame}, len={len(flexion_series)}")
    print(f"Matplotlib route => flexion min={flexion_series.min()}, max={flexion_series.max()}")


    
def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]

    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")

    idx_foot = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec = event_times[idx_release]

    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)

    foot_local = foot_contact_global - earliest
    release_local = release_global - earliest

    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")
    return foot_local, release_local

def compute_pitch_stability_score(ulnar_dev_series, pronation_series, flexion_series, accel_ulnar_series, accel_pron_series):
    """
    Computes a wrist stability score (0-100), where higher = better.

    The joint angle differences are computed as the difference between the baseline
    (frame 0, at foot contact) and the average over a 10-frame window centered on ball release.
    
    Inputs:
    - ulnar_dev_series: Time-series of ulnar deviation (frames)
    - pronation_series: Time-series of pronation angles (frames)
    - flexion_series: Time-series of flexion angles (frames)
    - accel_ulnar_series: Time-series of ulnar deviation acceleration
    - accel_pron_series: Time-series of pronation acceleration

    Returns:
    - Stability score (0-100)
    """
    # 1. Frame Indexing (define key moments)
    # Baseline is at frame 0 (foot contact)
    baseline_idx = 0
    # Define the release frame (ball release is assumed to be at index: total_frames - 20)
    release_idx = len(ulnar_dev_series) - 20

    # Define a window for release measurements: 5 frames before and 5 frames after release_idx
    window_start = max(release_idx - 5, 0)
    window_end   = min(release_idx + 5, len(ulnar_dev_series))
    
    # Compute the mean joint angles in that 10-frame window
    mean_ulnar_release = np.median(ulnar_dev_series[window_start:window_end])
    mean_pronation_release = np.median(pronation_series[window_start:window_end])
    mean_flexion_release = np.median(flexion_series[window_start:window_end])
    
    # Use the baseline at frame 0 (or you could also average a few frames at the start if desired)
    baseline_ulnar = ulnar_dev_series[baseline_idx]
    baseline_pronation = pronation_series[baseline_idx]
    baseline_flexion = flexion_series[baseline_idx]

    # 2. Compute absolute movement (penalizes excessive motion)
    max_ulnar_range = 30   # Expected max deviation (in degrees)
    max_pronation_range = 45 # Supination shouldn't be more than 45°
    max_flexion_range = 40   # Wrist flick shouldn't be excessive

    ulnar_stability = 100 - (abs(mean_ulnar_release - baseline_ulnar) / max_ulnar_range) * 100
    pronation_stability = 100 - (abs(mean_pronation_release - baseline_pronation) / max_pronation_range) * 100
    flexion_stability = 100 - (abs(mean_flexion_release - baseline_flexion) / max_flexion_range) * 100

    # 3. Compute acceleration penalty (using a window around release as before)
    accel_start = max(0, release_idx - 5)
    accel_end   = min(len(ulnar_dev_series), release_idx + 5)
    rms_accel_ulnar = np.sqrt(np.mean(np.square(accel_ulnar_series[accel_start:accel_end])))
    rms_accel_pron  = np.sqrt(np.mean(np.square(accel_pron_series[accel_start:accel_end])))

    max_expected_accel = 10  # Expected max acceleration in deg/frame²
    accel_stability = 100 - ((rms_accel_ulnar + rms_accel_pron) / max_expected_accel) * 100

    # 4. Ensure values are within [0, 100] range
    ulnar_stability = np.clip(ulnar_stability, 0, 100)
    pronation_stability = np.clip(pronation_stability, 0, 100)
    flexion_stability = np.clip(flexion_stability, 0, 100)
    accel_stability = np.clip(accel_stability, 0, 100)

    # 5. Weighted score (adjust weights if needed)
    # (Note: the weights in your current code sum to more than 1, but they can be adjusted as desired.)
    weights = [0.45, 0.20, 0.05, 0.30]
    final_score = (
        weights[0] * ulnar_stability +
        weights[1] * pronation_stability +
        weights[2] * flexion_stability +
        weights[3] * accel_stability
    )

    return round(final_score, 2)



# ---------------------- PROCESS STATIC TRIAL ----------------------
print("Processing static trial:", static_files[0])
static_c3d = ezc3d.c3d(static_files[0])

# 1) Filter static data at 6 Hz
filter_marker_data(static_c3d)

# 2) Create a consistent 'hand offset' from the static trial
static_hand_offset = create_virtual_hand_offset(static_c3d)

# 3) Compute baseline angles from the filtered, original static trial
def compute_static_baseline(c3d_obj):
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError("Static trial missing required markers.")
    nFrames = points.shape[2]

    ulnar_vals = []
    pron_vals  = []
    flex_vals  = []

    for f in range(nFrames):
        u = compute_ulnar_deviation(points, markers, f)
        p = compute_pronation(points, markers, f)
        x = compute_wrist_flexion(points, markers, f)
        ulnar_vals.append(u)
        pron_vals.append(p)
        flex_vals.append(x)

    return {
        "ulnar_dev": np.mean(ulnar_vals),
        "pronation": np.mean(pron_vals),
        "flexion":   np.mean(flex_vals)
    }

static_baseline_vals = compute_static_baseline(static_c3d)
print("Static baseline angles:", static_baseline_vals)


# ----------------- APPLY TO PITCH DATA -----------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file in c3d_files:
    if "static" in c3d_file.lower():
        continue  # Skip static trial

    c3d_obj = ezc3d.c3d(c3d_file)

    # 1) Filter dynamic data at 6 Hz
    filter_marker_data(c3d_obj)

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file}: required markers not found.")
        continue

    # 2) Apply our "virtual hand marker" override so it remains consistent
    apply_virtual_hand_marker(points, markers, static_hand_offset)

    # 3) Identify events
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]
    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file}: {e}")
        continue

    # Ensure enough frames post-release
    if release_frame + 20 >= total_frames:
        print(f"Skipping {c3d_file}: Not enough frames for release+20.")
        continue

    # 4) Basic info for DB
    normalized_path = os.path.normpath(c3d_file)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2].rstrip("_")
    pitch_date = date_folder
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]
    pitch_type = filename_noext.split()[0].capitalize()

    # 5) Extract Time-Series Data
    start_frame = foot_contact_frame
    end_frame = release_frame + 20
    ulnar_series = []
    pronation_series = []
    flexion_series = []

    for frame in range(start_frame, end_frame):
        ulnar_series.append(compute_ulnar_deviation(points, markers, frame))
        pronation_series.append(compute_pronation(points, markers, frame))
        flexion_series.append(compute_wrist_flexion(points, markers, frame))

    # Convert lists to NumPy arrays for processing
    ulnar_series = np.array(ulnar_series)
    pronation_series = np.array(pronation_series)
    flexion_series = np.array(flexion_series)
    print(f"Dash route => All Flexion Angles: {flexion_series}")
    print(f"Matplotlib route => All Flexion Angles: {flexion_series}")

    # 6) Compute Acceleration
    def compute_acceleration(angle_series):
        return np.gradient(np.gradient(angle_series))  # Second derivative

    accel_ulnar_series = compute_acceleration(ulnar_series)
    accel_pron_series = compute_acceleration(pronation_series)

    # 7) Compute Stability Score
    pitch_stability_score = compute_pitch_stability_score(
        ulnar_series, 
        pronation_series, 
        flexion_series, 
        accel_ulnar_series,  
        accel_pron_series
    )
    
    print(f"Processing {filename_noext} for {participant_name} - {pitch_type}")
    # plot_wrist_flexion(flexion_series, participant_name, pitch_type, filename_noext)

    print(f"Inserted {filename_noext} ({pitch_type}) - Stability Score: {pitch_stability_score}")

    # 8) Insert into Database
    insert_sql = """
        INSERT INTO pitch_data (
            participant_name, pitch_date, pitch_type, filename,
            pitch_stability_score,
            mid_u_dev, rel_u_dev,
            frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
            frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
            mid_pronation, rel_pronation,
            frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
            frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
        )
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(pitch_stability_score),  # Store computed stability score
        
        float(ulnar_series[len(ulnar_series) // 2]),  # Mid-frame ulnar deviation
        float(ulnar_series[-20]),  # Ulnar deviation at release

        float(ulnar_series[1]),
        float(ulnar_series[2]),
        float(ulnar_series[3]),
        float(ulnar_series[4]),
        float(ulnar_series[5]),
        float(ulnar_series[6]),
        float(ulnar_series[7]),
        float(ulnar_series[8]),
        float(ulnar_series[9]),
        float(ulnar_series[10]),

        float(pronation_series[len(pronation_series) // 2]),  # Mid-frame pronation
        float(pronation_series[-20]),  # Pronation at release

        float(pronation_series[1]),
        float(pronation_series[2]),
        float(pronation_series[3]),
        float(pronation_series[4]),
        float(pronation_series[5]),
        float(pronation_series[6]),
        float(pronation_series[7]),
        float(pronation_series[8]),
        float(pronation_series[9]),
        float(pronation_series[10])

    )

    cursor.execute(insert_sql, data_tuple)

conn.commit()
conn.close()
print("All data inserted into pitch_data!")


# ------ OPTIONAL: Retrieve last inserted participant/date for your Dash defaults ------
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)
    
import streamlit as st
import sqlite3
import pandas as pd
import plotly.graph_objects as go
from scipy.signal import medfilt

# =============================================================================
# Helper Functions (using your code)
# =============================================================================

db_path = "pitch_analysis_v3.sqlite"  # or your current db file

def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    # Return a list of participant names
    return sorted(participants)

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return sorted(dates)

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_type FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = sorted(df["pitch_type"].unique())
    # Insert "All" at the beginning if desired
    return ["All"] + options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT filename FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = sorted(df["filename"].unique())
    return ["All"] + options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def update_average_score(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"""
        SELECT AVG(pitch_stability_score) 
        FROM pitch_data 
        WHERE participant_name = '{selected_participant}' 
          AND pitch_date = '{selected_date}' 
          AND pitch_type = '{selected_pitch_type}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    avg_score = df.iloc[0, 0]
    return avg_score if avg_score is not None else 0

# For the time-series and plotting, we assume your existing functions are used.
# (You might import these from a separate module in your project.)
# For brevity, here we assume get_time_series() returns a dictionary with keys:
#   { pitch_type: { "ulnar_dev_series": [np.array, ...],
#                   "pronation": [np.array, ...],
#                   "flexion": [np.array, ...] } }
# and that your plotting code using Plotly is similar to the one in your Dash callback.
#
# I will replicate a simplified version for one graph; you can replicate for the others.

def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    conn = sqlite3.connect(db_path)
    # For demonstration, we simply read all rows that match the selection
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    # Convert stored string/numeric columns into time-series arrays.
    # Here we assume the time-series are stored in columns like:
    # mid_u_dev, rel_u_dev, frame1_u_dev, frame2_u_dev, ..., frame10_u_dev
    # We can aggregate by pitch_type.
    for _, row in df.iterrows():
        pt = row["pitch_type"]
        if pt not in ts:
            ts[pt] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}
        # For demonstration, we construct arrays from frame1 to frame10 for ulnar dev, etc.
        # (Adjust if your DB schema is different.)
        u_series = [row[f"frame{i}_u_dev"] for i in range(1, 11)]
        p_series = [row[f"frame{i}_pronation"] for i in range(1, 11)]
        f_series = [row[f"frame{i}_pronation"] for i in range(1, 11)]  # Change if flexion stored differently
        
        ts[pt]["ulnar_dev_series"].append(np.array(u_series))
        ts[pt]["pronation"].append(np.array(p_series))
        ts[pt]["flexion"].append(np.array(f_series))
    return ts

def plot_time_series(series_list, title, yaxis_title):
    # Plot all series from the list on one figure
    fig = go.Figure()
    for s in series_list:
        fig.add_trace(go.Scatter(x=list(range(len(s))), y=s, mode="lines"))
    fig.update_layout(title=title, xaxis_title="Frame", yaxis_title=yaxis_title)
    return fig

# =============================================================================
# Streamlit Layout
# =============================================================================

st.set_page_config(layout="wide", page_title="Pitch Analysis Report")

# Sidebar - Dropdowns
st.sidebar.header("Select Data")
participants = get_dropdown_options()
selected_participant = st.sidebar.selectbox("Select Participant", participants)

if selected_participant:
    dates = get_date_options(selected_participant)
    selected_date = st.sidebar.selectbox("Select Test Date", dates)
    
    if selected_date:
        pitch_types = get_pitch_type_options(selected_participant, selected_date)
        selected_pitch_type = st.sidebar.selectbox("Select Pitch Type", pitch_types)
        
        filenames = get_filename_options(selected_participant, selected_date, selected_pitch_type)
        selected_filename = st.sidebar.selectbox("Select Pitch Number", filenames)
    else:
        selected_date = None
        selected_pitch_type = None
        selected_filename = None
else:
    selected_date = selected_pitch_type = selected_filename = None

# Main Dashboard Area
st.title("Pitch Analysis Report")

if selected_participant and selected_date and selected_pitch_type and selected_filename:
    avg_score = update_average_score(selected_participant, selected_date, selected_pitch_type)
    st.metric("Average Stability Score", f"{avg_score:.2f}")

    st.header("Comparison Table")
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    st.dataframe(comp_df)

    st.header("Time Series Plots")
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)

    # For each pitch type (or if only one, use that), plot the figures.
    for pt, data in ts_data.items():
        st.subheader(f"Pitch Type: {pt}")

        # Ulnar Deviation Plot
        fig_ud = plot_time_series(data["ulnar_dev_series"], "Ulnar Deviation Time Series", "Angle (°)")
        st.plotly_chart(fig_ud, use_container_width=True)

        # Acceleration Plot (if you compute acceleration separately)
        # For simplicity, we can show the acceleration of the first ulnar series:
        if data["ulnar_dev_series"]:
            acc = np.diff(data["ulnar_dev_series"][0], n=2)
            fig_acc = go.Figure()
            fig_acc.add_trace(go.Scatter(x=list(range(len(acc))), y=acc, mode="lines"))
            fig_acc.update_layout(title="Acceleration", xaxis_title="Frame", yaxis_title="Acceleration (°/frame²)")
            st.plotly_chart(fig_acc, use_container_width=True)

        # Pronation Plot
        fig_pr = plot_time_series(data["pronation"], "Pronation Time Series", "Angle (°)")
        st.plotly_chart(fig_pr, use_container_width=True)

        # Flexion Plot
        fig_fl = plot_time_series(data["flexion"], "Flexion Time Series", "Angle (°)")
        st.plotly_chart(fig_fl, use_container_width=True)
else:
    st.info("Please select a participant, test date, pitch type, and pitch number from the sidebar.")


Processing static trial: D:/Youth Pitch Design/Data/Matt Solter_MS/2025-03-05_\Pitch Design Static RH 1.c3d
Static baseline angles: {'ulnar_dev': np.float64(19.430699334632923), 'pronation': np.float64(78.54913064282889), 'flexion': np.float64(152.59566509365473)}
Dash route => All Flexion Angles: [139.12117662 139.36553082 139.61893708 139.88346257 140.16280567
 140.46285343 140.79236054 141.16379668 141.59442013 142.10764936
 142.73481153 143.51730962 144.50915538 145.77963278 147.41545689
 149.52099233 152.21369972 155.61000339 159.79542203 164.77601108
 170.42216812 176.43977045 177.58783266 172.08533924 167.36399339
 163.56611143 160.68586289 158.62567719 157.249278   156.41629779
 155.9997665  155.89251577 156.00772033 156.27683786 156.64668175
 157.07645296 157.53507351 157.99892567 158.44999438 158.87436847
 159.26104412 159.60097301 159.88629878 160.10973144 160.26401814
 160.34147421 160.33354188 160.23034705 160.02022538 159.68918662
 159.22028328 158.59284514 157.78153877 1

2025-03-06 13:16:29.911 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:16:29.912 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:16:30.711 
  command:

    streamlit run C:\Users\q\miniconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-06 13:16:30.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:16:30.714 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:16:30.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:16:30.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running

In [62]:
import pdfkit
import tkinter as tk

root = tk.Tk()
root.withdraw()
target_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

config = pdfkit.configuration(wkhtmltopdf=r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe")

pdfkit.from_url(
    "http://127.0.0.1:8050",
    "C:/SomeFolder/dashboard.pdf",
    configuration=config
)


OSError: No wkhtmltopdf executable found: "C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"
If this file exists please check that this process can read it or you can pass path to it manually in method call, check README. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf

In [ ]:
######################################################
# All below this is drafts
######################################################